In [2]:
import requests as r
import json
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import random
import time
import pandas as pd
import re
import numpy as np
import csv


In [3]:
# Начальный URL для дальнейшего парсинга
# URL = 'https://auto.ru/cars/used/' 
URL = 'https://auto.ru/cars/bmw/used/'
# element copied from start page to avoid flexible classes, copied: 13.10.2021
BRAND_SOURCE = "brands_soup.txt"

pd.set_option('display.max_columns', None)

In [4]:
HEADERS = {
    'Connection': 'keep-alive',
    'sec-ch-ua': '"Chromium";v="94", "Google Chrome";v="94", ";Not A Brand";v="99"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.71 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-User': '?1',
    'Sec-Fetch-Dest': 'document',
    'Referer': 'https://auto.ru/cars/used/',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'ru'
}

cookies = dict({
                'autoru_gdpr':'1',
                'suid':'d623ff3ea8568230f7365727199435bb.f86fd98831fdbc8f7cc83e6bc734fb90',
                'yandexuid':'625342111625395404',
                'counter_ga_all7':'2',
                '_csrf_token':'cbf3d961f93743c21f8b4a70feedb03997ae9354dfc79bae',
                'autoru_sid':'a%3Ag615dfe3a2oedoori3cl0hbgl2n5vbn0.9d2cb71f02ff80ffb82c172756d4884f%7C1633549882248.604800.EZNqR3FCBHVvD6N4ivq4sw.iR14jacJziqqnOk2TDS-6h8U9C5bJMUlo4wsdbtScio',
                'autoruuid':'g615dfe3a2oedoori3cl0hbgl2n5vbn0.9d2cb71f02ff80ffb82c172756d4884f',
                'yuidlt':'1',
                'bltsr':'1', 
                'X-Vertis-DC':'sas',
                'autoru-visits-count':'18',
                'autoru-visits-session-unexpired':'1',
                'from_lifetime':'1633641453855',
                'from':'direct',
                'los':'1'
                })


In [5]:
def from_lifetime(resp):
    lst = resp.request.headers['Cookie'].split('; ')
    dict1 = {}
    for elem in lst:
        key, value = elem.split('=')
        dict1[key] = value
        return dict1['from_lifetime']
    
    
def get_pages_url_list(brand):
    # get urls from csv file "brand_links.csv"
    brand = brand.upper()
    file_name = "LINKS/%s_links.csv"%brand
    pages_url_list = pd.read_csv(file_name, sep = '\n', header=None)[0].unique()
    return pages_url_list


def get_content(brand, start_at=0):
    brand = brand.upper()
    pages_url_list = get_pages_url_list(brand)
    print(brand + ": " + str(len(pages_url_list)) + " urls to parse")
    content_csv = 'CONTENT/%s_content.csv'%brand
    cars_list = []
    link_counter = 0
    portion_size = 100
    
    if start_at==0:
        print("starting from the beginning...")
        print("creating " + content_csv)
        open(content_csv, mode='w').close()
    else:
        print("starting from ..." + str(start_at))
        content_csv = 'CONTENT/%s_%s_content.csv'%(brand, start_at)
        print("creating " + content_csv)
        open(content_csv, mode='w').close()
    
    # идем по линкам:
    for item in pages_url_list:
        # link counter:
        link_counter += 1
        if link_counter > start_at:
            # add a small portion to csv file:
            if link_counter%portion_size == 0: # next portion_size entries
                print(str(link_counter) + " links parsed") 
                print("adding to " + content_csv)
                df_temp = pd.DataFrame(cars_list)
                df_temp.to_csv(content_csv, mode='a', header=False, index=False)
                cars_list = []



            response = r.get(item, headers=HEADERS, cookies=cookies)
            cookies['from_lifetime'] = response.cookies['from_lifetime']
            response.encoding = 'utf8'
            page = BeautifulSoup(response.text, 'html.parser')

            if page.find('div', class_='CardSold') == None:
                try:
                    json_data = json.loads(
                        page.find('script', type="application/ld+json").string)

                except:
                    print(item)
                    pass
                cls_str = 'CardInfoRow_'
                span_str = 'CardInfoRow__cell'

                try:
                    cars_list.append({
                    'brand': json_data['brand'],

                    'model_name': json_data['name'],
                    'bodyType': json_data['bodyType'],
                    'numberOfDoors': json_data['numberOfDoors'],
                    'modelDate': json_data['modelDate'],
                    'productionDate': json_data['productionDate'],
                    'color': json_data['color'],
                    #'description': json_data['description'],
                    'engineDisplacement': json_data['vehicleEngine']['engineDisplacement'],
                    'enginePower': json_data['vehicleEngine']['enginePower'],
                    'fuelType': json_data['fuelType'],
                    'image': json_data['image'],
                    'mileage': page.find(
                        'li', class_=cls_str+'kmAge').find_all('span')[1].text,

                    'name': json_data['vehicleEngine']['name'],

                    # 'priceCurrency': json_data['offers']['priceCurrency'],

                    'vehicleConfiguration': json_data['vehicleConfiguration'],
                    'vehicleTransmission': json_data['vehicleTransmission'],
                    'Владельцы': page.find(
                        'li', class_=cls_str+'ownersCount').find_all('span')[1].text,
                    'ПТС': page.find(
                        'li', class_=cls_str+'pts').find_all('span')[1].text,
                    'Привод': page.find(
                        'li', class_=cls_str+'drive').find_all('span')[1].text,
                    'Руль': page.find(
                        'li', class_=cls_str+'wheel').find_all('span')[1].text,
                    'Состояние': page.find(
                        'li', class_=cls_str+'state').find_all('span')[1].text,
                    'Таможня': page.find(
                        'li', class_=cls_str+'customs').find_all('span')[1].text,

                    'car_url': json_data['offers']['url'],
                    'price': page.find(
                        'span', class_='OfferPriceCaption__price').text
                    })
                    print(str(link_counter), cars_list[-1]["car_url"], cars_list[-1]["price"])
                except:
                    print("exception: ")
                    print(item)
                    pass

            rand_time = random.randint(2,4)
            time.sleep(rand_time)
        # if there is a tail, add it to file:
    if len(cars_list) > 0:
        print(str(link_counter) + " links parsed") 
        print("adding to " + content_csv)
        df_temp = pd.DataFrame(cars_list)
        df_temp.to_csv(content_csv, mode='a', header=False, index=False)
        cars_list = []
        
   

##  Get content  

In this part we will try to get real content fron site, using collected links

Get needed brands:

In [5]:
brands = list(pd.read_csv("test.csv")["brand"].unique())
brands

['SKODA',
 'AUDI',
 'HONDA',
 'VOLVO',
 'BMW',
 'NISSAN',
 'INFINITI',
 'MERCEDES',
 'TOYOTA',
 'LEXUS',
 'VOLKSWAGEN',
 'MITSUBISHI']

just parsing...

In [6]:
get_content("Skoda")

SKODA: 3483 urls to parse
starting from the beginning...
creating CONTENT/SKODA_content.csv
1 https://auto.ru/cars/used/sale/skoda/fabia/1105665977-6a34e412/ 370 000 ₽
2 https://auto.ru/cars/used/sale/skoda/fabia/1105414799-ea03f986/ 155 000 ₽
3 https://auto.ru/cars/used/sale/skoda/octavia/1105606055-e2b1f813/ 840 000 ₽
4 https://auto.ru/cars/used/sale/skoda/fabia_rs/1105659664-defce4d3/ 470 000 ₽
5 https://auto.ru/cars/used/sale/skoda/fabia/1105556436-47eb7bd8/ 290 000 ₽
6 https://auto.ru/cars/used/sale/skoda/octavia/1105000942-cb36e05b/ 225 000 ₽
7 https://auto.ru/cars/used/sale/skoda/superb/1105585378-ac463897/ 275 000 ₽
8 https://auto.ru/cars/used/sale/skoda/octavia/1105665995-069b81c5/ 625 000 ₽
9 https://auto.ru/cars/used/sale/skoda/octavia/1105467011-caaecb68/ 659 000 ₽
10 https://auto.ru/cars/used/sale/skoda/superb/1105657781-23e9f83a/ 720 000 ₽
11 https://auto.ru/cars/used/sale/skoda/octavia/1103121541-533014ba/ 430 000 ₽
12 https://auto.ru/cars/used/sale/skoda/fabia/110513175

109 https://auto.ru/cars/used/sale/skoda/superb/1105665992-5d8dfcf5/ 1 225 000 ₽
110 https://auto.ru/cars/used/sale/skoda/octavia/1105639803-a4a788bf/ 659 000 ₽
111 https://auto.ru/cars/used/sale/skoda/rapid/1105663229-8b9ffb07/ 899 000 ₽
112 https://auto.ru/cars/used/sale/skoda/octavia/1105665042-3ec9ef13/ 390 000 ₽
113 https://auto.ru/cars/used/sale/skoda/octavia/1105473016-6fcd98d1/ 799 000 ₽
114 https://auto.ru/cars/used/sale/skoda/fabia/1105665813-13edadce/ 490 000 ₽
115 https://auto.ru/cars/used/sale/skoda/rapid/1105547930-2d9cd985/ 1 060 000 ₽
116 https://auto.ru/cars/used/sale/skoda/octavia/1105580228-da09b0cf/ 2 800 000 ₽
117 https://auto.ru/cars/used/sale/skoda/octavia/1105618420-a5432c7c/ 360 000 ₽
118 https://auto.ru/cars/used/sale/skoda/fabia/1105555276-163566fa/ 465 000 ₽
119 https://auto.ru/cars/used/sale/skoda/rapid/1105660849-d8d6032b/ 1 081 000 ₽
120 https://auto.ru/cars/used/sale/skoda/octavia/1105641518-bf31dc03/ 1 299 000 ₽
121 https://auto.ru/cars/used/sale/skoda/

215 https://auto.ru/cars/used/sale/skoda/yeti/1105611934-e9290190/ 825 000 ₽
216 https://auto.ru/cars/used/sale/skoda/rapid/1105656923-079a630a/ 745 000 ₽
217 https://auto.ru/cars/used/sale/skoda/superb/1105324036-d9c9216b/ 1 785 000 ₽
218 https://auto.ru/cars/used/sale/skoda/rapid/1105435939-4a3b8112/ 913 000 ₽
219 https://auto.ru/cars/used/sale/skoda/rapid/1105317051-6bbbcb4a/ 835 000 ₽
220 https://auto.ru/cars/used/sale/skoda/yeti/1105476722-d51b9d30/ 848 000 ₽
221 https://auto.ru/cars/used/sale/skoda/fabia/1105280384-dec2e4f8/ 400 000 ₽
222 https://auto.ru/cars/used/sale/skoda/rapid/1105636062-0fc3659b/ 1 050 000 ₽
223 https://auto.ru/cars/used/sale/skoda/octavia/1105552820-f57384ad/ 1 630 000 ₽
224 https://auto.ru/cars/used/sale/skoda/fabia/1105233967-b73127b8/ 499 000 ₽
226 https://auto.ru/cars/used/sale/skoda/yeti/1105661065-e98b549a/ 850 000 ₽
227 https://auto.ru/cars/used/sale/skoda/rapid/1105562120-104a9c05/ 1 229 900 ₽
228 https://auto.ru/cars/used/sale/skoda/rapid/110552678

322 https://auto.ru/cars/used/sale/skoda/octavia/1105654251-d4c5507d/ 1 584 000 ₽
323 https://auto.ru/cars/used/sale/skoda/fabia/1105654548-9cb36b8b/ 499 000 ₽
324 https://auto.ru/cars/used/sale/skoda/octavia/1105654036-5cb8c999/ 999 000 ₽
325 https://auto.ru/cars/used/sale/skoda/yeti/1105545053-bfa7a953/ 1 050 000 ₽
326 https://auto.ru/cars/used/sale/skoda/rapid/1104931248-79d4677d/ 1 178 767 ₽
327 https://auto.ru/cars/used/sale/skoda/rapid/1105653713-147e922d/ 990 000 ₽
328 https://auto.ru/cars/used/sale/skoda/octavia/1105160353-f7a0c9f5/ 899 000 ₽
329 https://auto.ru/cars/used/sale/skoda/fabia/1105652152-c08f2a8e/ 198 000 ₽
330 https://auto.ru/cars/used/sale/skoda/rapid/1105612007-cc0b0882/ 713 000 ₽
331 https://auto.ru/cars/used/sale/skoda/rapid/1104973765-00d6b02b/ 912 000 ₽
332 https://auto.ru/cars/used/sale/skoda/superb/1105252702-4ba57497/ 730 000 ₽
333 https://auto.ru/cars/used/sale/skoda/rapid/1105654038-66f3ab8a/ 1 619 000 ₽
334 https://auto.ru/cars/used/sale/skoda/octavia/1

428 https://auto.ru/cars/used/sale/skoda/rapid/1105665552-5770496a/ 950 000 ₽
429 https://auto.ru/cars/used/sale/skoda/octavia/1105649046-bfbdb8be/ 1 865 500 ₽
430 https://auto.ru/cars/used/sale/skoda/octavia/1105647990-53f4467c/ 780 000 ₽
431 https://auto.ru/cars/used/sale/skoda/kodiaq/1105034521-a244e1b3/ 1 679 000 ₽
432 https://auto.ru/cars/used/sale/skoda/octavia/1105485110-bb132c85/ 450 000 ₽
433 https://auto.ru/cars/used/sale/skoda/superb/1105651627-675a36fb/ 799 000 ₽
435 https://auto.ru/cars/used/sale/skoda/octavia/1105451216-8929d667/ 715 000 ₽
436 https://auto.ru/cars/used/sale/skoda/octavia/1105084486-0efd75e4/ 2 200 000 ₽
437 https://auto.ru/cars/used/sale/skoda/octavia/1104164964-b1531fc6/ 350 000 ₽
438 https://auto.ru/cars/used/sale/skoda/octavia/1105571264-7572bd6d/ 350 000 ₽
439 https://auto.ru/cars/used/sale/skoda/kodiaq/1104971634-4e6f5234/ 2 950 000 ₽
440 https://auto.ru/cars/used/sale/skoda/octavia/1105650692-50ac20ed/ 799 999 ₽
441 https://auto.ru/cars/used/sale/sk

534 https://auto.ru/cars/used/sale/skoda/octavia/1105211349-b47ebf74/ 960 000 ₽
535 https://auto.ru/cars/used/sale/skoda/rapid/1105645099-6bcc5f93/ 978 000 ₽
536 https://auto.ru/cars/used/sale/skoda/octavia/1105664996-6e08d511/ 770 000 ₽
537 https://auto.ru/cars/used/sale/skoda/fabia/1105665016-8402e0a2/ 170 000 ₽
538 https://auto.ru/cars/used/sale/skoda/rapid/1105644869-11fd96a3/ 814 000 ₽
539 https://auto.ru/cars/used/sale/skoda/octavia/1105645096-d1db4a2b/ 989 000 ₽
540 https://auto.ru/cars/used/sale/skoda/octavia/1105645423-b578f233/ 1 379 000 ₽
541 https://auto.ru/cars/used/sale/skoda/octavia/1105645103-3848bea5/ 1 520 000 ₽
542 https://auto.ru/cars/used/sale/skoda/octavia/1105531593-38e509cd/ 175 000 ₽
543 https://auto.ru/cars/used/sale/skoda/octavia/1105211353-906c3137/ 330 000 ₽
544 https://auto.ru/cars/used/sale/skoda/felicia/1105644703-6f65711c/ 50 000 ₽
545 https://auto.ru/cars/used/sale/skoda/fabia/1105253548-38ba7c18/ 369 000 ₽
546 https://auto.ru/cars/used/sale/skoda/yeti

ConnectionError: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))

In [8]:
get_content("Skoda", 500)

SKODA: 3483 urls to parse
starting from ...500
creating CONTENT/SKODA_500_content.csv
501 https://auto.ru/cars/used/sale/skoda/felicia/1105129206-f3cda450/ 60 000 ₽
502 https://auto.ru/cars/used/sale/skoda/rapid/1105646107-15f67b49/ 985 000 ₽
503 https://auto.ru/cars/used/sale/skoda/octavia/1105645708-f8e025f6/ 897 000 ₽
504 https://auto.ru/cars/used/sale/skoda/fabia/1105647238-783bbaca/ 595 000 ₽
505 https://auto.ru/cars/used/sale/skoda/rapid/1105645524-3a2e15eb/ 765 000 ₽
506 https://auto.ru/cars/used/sale/skoda/octavia/1105645700-b87ba5ad/ 1 819 000 ₽
507 https://auto.ru/cars/used/sale/skoda/octavia/1104658742-fa2561fb/ 980 000 ₽
508 https://auto.ru/cars/used/sale/skoda/fabia/1105647273-abf6d646/ 599 000 ₽
509 https://auto.ru/cars/used/sale/skoda/rapid/1105665229-0e164238/ 890 000 ₽
510 https://auto.ru/cars/used/sale/skoda/octavia/1105645699-5f01be2b/ 1 239 000 ₽
511 https://auto.ru/cars/used/sale/skoda/kodiaq/1105646940-b8a0d2a9/ 2 498 657 ₽
512 https://auto.ru/cars/used/sale/skoda

602 https://auto.ru/cars/used/sale/skoda/rapid/1105563103-dc545846/ 1 390 000 ₽
603 https://auto.ru/cars/used/sale/skoda/octavia/1105642448-4bd9ba4b/ 1 024 315 ₽
604 https://auto.ru/cars/used/sale/skoda/superb/1105440205-123062bd/ 595 000 ₽
605 https://auto.ru/cars/used/sale/skoda/octavia/1105443220-8b606df7/ 530 000 ₽
606 https://auto.ru/cars/used/sale/skoda/rapid/1105641570-b5ee564b/ 1 645 000 ₽
607 https://auto.ru/cars/used/sale/skoda/octavia/1105643541-8c8f5b32/ 2 070 000 ₽
608 https://auto.ru/cars/used/sale/skoda/rapid/1105556710-f023be85/ 727 000 ₽
609 https://auto.ru/cars/used/sale/skoda/rapid/1105567299-9b6ac6df/ 1 238 000 ₽
610 https://auto.ru/cars/used/sale/skoda/fabia/1105664664-962384ff/ 310 000 ₽
611 https://auto.ru/cars/used/sale/skoda/yeti/1105641280-615004b4/ 697 100 ₽
612 https://auto.ru/cars/used/sale/skoda/superb/1105556708-7a201aa8/ 1 915 000 ₽
613 https://auto.ru/cars/used/sale/skoda/octavia/1105003397-0ae53add/ 349 000 ₽
614 https://auto.ru/cars/used/sale/skoda/ka

704 https://auto.ru/cars/used/sale/skoda/octavia/1105613896-b230cacb/ 1 350 000 ₽
705 https://auto.ru/cars/used/sale/skoda/octavia/1105581664-40b943f0/ 685 000 ₽
706 https://auto.ru/cars/used/sale/skoda/octavia/1105404173-a7c22906/ 660 000 ₽
707 https://auto.ru/cars/used/sale/skoda/rapid/1105463159-657c7114/ 1 120 000 ₽
708 https://auto.ru/cars/used/sale/skoda/felicia/1105176575-67109732/ 95 000 ₽
709 https://auto.ru/cars/used/sale/skoda/octavia/1105197307-afaed39e/ 2 248 978 ₽
710 https://auto.ru/cars/used/sale/skoda/yeti/1105245073-39959378/ 665 000 ₽
711 https://auto.ru/cars/used/sale/skoda/superb/1105255512-77f8ad02/ 1 900 000 ₽
713 https://auto.ru/cars/used/sale/skoda/octavia/1105514664-9119ad62/ 615 000 ₽
714 https://auto.ru/cars/used/sale/skoda/octavia/1105508799-3e39c8f5/ 700 000 ₽
715 https://auto.ru/cars/used/sale/skoda/rapid/1105629945-3ff8893b/ 567 300 ₽
716 https://auto.ru/cars/used/sale/skoda/rapid/1105631791-58680b1a/ 1 122 100 ₽
717 https://auto.ru/cars/used/sale/skoda/

810 https://auto.ru/cars/used/sale/skoda/octavia/1105514350-0b342824/ 1 175 000 ₽
811 https://auto.ru/cars/used/sale/skoda/octavia/1105072238-d985d257/ 1 089 000 ₽
812 https://auto.ru/cars/used/sale/skoda/octavia/1105663337-0397dcff/ 550 000 ₽
813 https://auto.ru/cars/used/sale/skoda/rapid/1105632333-88ab8ed8/ 950 000 ₽
814 https://auto.ru/cars/used/sale/skoda/fabia/1105563582-906e3c37/ 525 000 ₽
815 https://auto.ru/cars/used/sale/skoda/superb/1105632293-ff36b08a/ 720 000 ₽
816 https://auto.ru/cars/used/sale/skoda/yeti/1105059492-2900ddfd/ 989 000 ₽
817 https://auto.ru/cars/used/sale/skoda/yeti/1105634968-11b117f2/ 1 400 000 ₽
818 https://auto.ru/cars/used/sale/skoda/octavia/1105611411-5fd83d6e/ 785 000 ₽
819 https://auto.ru/cars/used/sale/skoda/octavia/1105291398-41259561/ 890 000 ₽
820 https://auto.ru/cars/used/sale/skoda/rapid/1105633366-255b08e0/ 1 424 000 ₽
821 https://auto.ru/cars/used/sale/skoda/fabia/1105625873-f1101aa1/ 170 000 ₽
822 https://auto.ru/cars/used/sale/skoda/rapid/

913 https://auto.ru/cars/used/sale/skoda/rapid/1105633389-8328713a/ 834 000 ₽
914 https://auto.ru/cars/used/sale/skoda/yeti/1105421428-06690897/ 975 000 ₽
915 https://auto.ru/cars/used/sale/skoda/octavia/1105546930-8ae51981/ 570 000 ₽
916 https://auto.ru/cars/used/sale/skoda/octavia/1104904628-a314707e/ 544 000 ₽
917 https://auto.ru/cars/used/sale/skoda/rapid/1105632331-5867c983/ 950 000 ₽
918 https://auto.ru/cars/used/sale/skoda/octavia/1105065637-fb249dd3/ 711 000 ₽
919 https://auto.ru/cars/used/sale/skoda/fabia/1105439729-ca12b674/ 365 000 ₽
920 https://auto.ru/cars/used/sale/skoda/octavia/1105220057-dff2a22c/ 1 239 900 ₽
921 https://auto.ru/cars/used/sale/skoda/fabia/1105453612-1472f16d/ 375 000 ₽
922 https://auto.ru/cars/used/sale/skoda/octavia/1105591867-00e47433/ 849 900 ₽
923 https://auto.ru/cars/used/sale/skoda/rapid/1105632332-b8208b66/ 950 000 ₽
924 https://auto.ru/cars/used/sale/skoda/karoq/1105631694-9e842701/ 1 640 000 ₽
925 https://auto.ru/cars/used/sale/skoda/fabia/1105

...

3012 https://auto.ru/cars/used/sale/skoda/octavia/1105629961-d5a497c5/ 360 000 ₽
3013 https://auto.ru/cars/used/sale/skoda/rapid/1104846564-4ec41071/ 888 000 ₽
3014 https://auto.ru/cars/used/sale/skoda/yeti/1105095819-79766cb5/ 525 000 ₽
3015 https://auto.ru/cars/used/sale/skoda/octavia/1105250789-bcaf3b4c/ 750 000 ₽
3016 https://auto.ru/cars/used/sale/skoda/octavia/1105306928-91bff8ed/ 1 000 000 ₽
3017 https://auto.ru/cars/used/sale/skoda/octavia/1102011527-b98b3ea1/ 1 076 599 ₽
3018 https://auto.ru/cars/used/sale/skoda/octavia/1099523584-722ac296/ 315 000 ₽
3019 https://auto.ru/cars/used/sale/skoda/fabia/1103145272-8bbebc77/ 500 000 ₽
3020 https://auto.ru/cars/used/sale/skoda/octavia/1103269756-9b9ecda4/ 890 000 ₽
3021 https://auto.ru/cars/used/sale/skoda/octavia/1105569696-868c367b/ 900 000 ₽
3022 https://auto.ru/cars/used/sale/skoda/rapid/1105573312-7d66616b/ 1 150 000 ₽
3023 https://auto.ru/cars/used/sale/skoda/superb/1105215080-9be15ef6/ 1 499 999 ₽
3024 https://auto.ru/cars/used

3113 https://auto.ru/cars/used/sale/skoda/yeti/1105019298-37ad6732/ 645 000 ₽
3114 https://auto.ru/cars/used/sale/skoda/fabia/1105599162-fc431eb7/ 365 000 ₽
3115 https://auto.ru/cars/used/sale/skoda/rapid/1105606216-66639fec/ 469 000 ₽
3116 https://auto.ru/cars/used/sale/skoda/kodiaq/1104964842-9af269f6/ 2 800 000 ₽
3117 https://auto.ru/cars/used/sale/skoda/octavia/1105410137-9c5001eb/ 650 000 ₽
3118 https://auto.ru/cars/used/sale/skoda/yeti/1105305460-6bd64786/ 725 000 ₽
3119 https://auto.ru/cars/used/sale/skoda/octavia/1094912380-8f56b05c/ 851 000 ₽
3120 https://auto.ru/cars/used/sale/skoda/fabia/1105271823-3b7a0ba0/ 420 000 ₽
3121 https://auto.ru/cars/used/sale/skoda/octavia/1105497705-a7ec249b/ 1 690 000 ₽
3122 https://auto.ru/cars/used/sale/skoda/fabia/1105112340-576b9aa4/ 530 000 ₽
3123 https://auto.ru/cars/used/sale/skoda/octavia/1103440872-1901ed01/ 675 000 ₽
3124 https://auto.ru/cars/used/sale/skoda/kodiaq/1105118712-9c5c39cb/ 1 882 399 ₽
3125 https://auto.ru/cars/used/sale/sk

3215 https://auto.ru/cars/used/sale/skoda/rapid/1105091093-37276bb4/ 595 000 ₽
3216 https://auto.ru/cars/used/sale/skoda/octavia/1099959148-e4a0b04d/ 600 000 ₽
3217 https://auto.ru/cars/used/sale/skoda/octavia/1101511435-69725ea8/ 600 000 ₽
3218 https://auto.ru/cars/used/sale/skoda/octavia/1102951405-5070a54d/ 1 400 000 ₽
3219 https://auto.ru/cars/used/sale/skoda/superb/1103486410-e92be740/ 789 000 ₽
3220 https://auto.ru/cars/used/sale/skoda/octavia/1105514627-abc9e49f/ 550 000 ₽
3221 https://auto.ru/cars/used/sale/skoda/yeti/1104700333-85360525/ 850 000 ₽
3222 https://auto.ru/cars/used/sale/skoda/fabia/1104279974-dcfedd54/ 500 000 ₽
3223 https://auto.ru/cars/used/sale/skoda/fabia/1104957351-3bcc8150/ 330 000 ₽
3224 https://auto.ru/cars/used/sale/skoda/octavia/1103072213-d6a4036d/ 570 000 ₽
3225 https://auto.ru/cars/used/sale/skoda/octavia_rs/1105518167-0b96eb42/ 2 700 000 ₽
3226 https://auto.ru/cars/used/sale/skoda/fabia/1105100850-7b93f59f/ 435 000 ₽
3227 https://auto.ru/cars/used/sa

3317 https://auto.ru/cars/used/sale/skoda/octavia/1103694317-96995751/ 585 000 ₽
3318 https://auto.ru/cars/used/sale/skoda/rapid/1104968874-12abb420/ 780 000 ₽
3319 https://auto.ru/cars/used/sale/skoda/rapid/1104631528-0e50c569/ 1 500 000 ₽
3320 https://auto.ru/cars/used/sale/skoda/rapid/1105173792-51295031/ 970 000 ₽
3321 https://auto.ru/cars/used/sale/skoda/octavia/1103737267-ea8aa7aa/ 750 000 ₽
3322 https://auto.ru/cars/used/sale/skoda/roomster/1105531179-93f507a0/ 595 299 ₽
3323 https://auto.ru/cars/used/sale/skoda/yeti/1103310213-2cd9e2c8/ 860 000 ₽
3324 https://auto.ru/cars/used/sale/skoda/rapid/1102958894-8e5cb6d3/ 800 000 ₽
3325 https://auto.ru/cars/used/sale/skoda/fabia/1105375801-46b02122/ 530 000 ₽
3326 https://auto.ru/cars/used/sale/skoda/octavia/1105144833-e512641e/ 1 030 000 ₽
3327 https://auto.ru/cars/used/sale/skoda/octavia/1102612684-57dba725/ 919 900 ₽
3328 https://auto.ru/cars/used/sale/skoda/fabia/1104247970-1ae8f870/ 440 000 ₽
3329 https://auto.ru/cars/used/sale/sk

3419 https://auto.ru/cars/used/sale/skoda/fabia/1104972879-b967dd80/ 460 000 ₽
3420 https://auto.ru/cars/used/sale/skoda/octavia/1095672824-3ddd8738/ 850 000 ₽
3421 https://auto.ru/cars/used/sale/skoda/octavia/1105259999-b300b0b2/ 1 439 299 ₽
3422 https://auto.ru/cars/used/sale/skoda/octavia/1105305861-bf776484/ 260 000 ₽
3423 https://auto.ru/cars/used/sale/skoda/yeti/1105332023-a4431240/ 650 000 ₽
3424 https://auto.ru/cars/used/sale/skoda/fabia/1104857831-eb1fecf7/ 515 000 ₽
3425 https://auto.ru/cars/used/sale/skoda/fabia/1105125991-4299a5ee/ 650 000 ₽
3426 https://auto.ru/cars/used/sale/skoda/fabia/1104034909-74acde38/ 290 000 ₽
3427 https://auto.ru/cars/used/sale/skoda/octavia/1105161837-f531f822/ 345 000 ₽
3428 https://auto.ru/cars/used/sale/skoda/rapid/1105540767-c42bd5f9/ 800 000 ₽
3429 https://auto.ru/cars/used/sale/skoda/octavia/1098666272-53f0f3cd/ 1 800 000 ₽
3430 https://auto.ru/cars/used/sale/skoda/octavia/1105094089-d9c0358a/ 850 000 ₽
3431 https://auto.ru/cars/used/sale/s

In [9]:
get_content("Audi")

AUDI: 3503 urls to parse
starting from the beginning...
creating CONTENT/AUDI_content.csv
1 https://auto.ru/cars/used/sale/audi/a8/1105410107-e78dd73a/ 2 850 000 ₽
2 https://auto.ru/cars/used/sale/audi/a4/1105581075-129a636a/ 610 000 ₽
3 https://auto.ru/cars/used/sale/audi/a8/1103694871-80c238bd/ 560 000 ₽
5 https://auto.ru/cars/used/sale/audi/sq8/1105148647-671bb0d0/ 10 500 000 ₽
6 https://auto.ru/cars/used/sale/audi/a4/1104812142-3da392a4/ 602 000 ₽
7 https://auto.ru/cars/used/sale/audi/a6/1105548890-a65bd5c6/ 715 000 ₽
8 https://auto.ru/cars/used/sale/audi/a3/1104231862-1f738ebc/ 415 000 ₽
9 https://auto.ru/cars/used/sale/audi/a7/1105532516-2b7081b3/ 1 870 000 ₽
10 https://auto.ru/cars/used/sale/audi/allroad/1105661678-3bb6fa0c/ 300 000 ₽
11 https://auto.ru/cars/used/sale/audi/a8/1105299771-3b42ae9e/ 1 290 000 ₽
12 https://auto.ru/cars/used/sale/audi/a6/1103883087-195d7c22/ 3 489 000 ₽
13 https://auto.ru/cars/used/sale/audi/a4/1105484139-c5e8a1be/ 989 000 ₽
14 https://auto.ru/cars/u

115 https://auto.ru/cars/used/sale/audi/q7/1105396558-a3ddaba0/ 4 977 777 ₽
116 https://auto.ru/cars/used/sale/audi/a5/1105664994-98c8f74e/ 3 400 000 ₽
117 https://auto.ru/cars/used/sale/audi/a4/1105273761-8994260a/ 2 650 000 ₽
118 https://auto.ru/cars/used/sale/audi/q7/1105632225-e6f6b0fa/ 4 790 000 ₽
119 https://auto.ru/cars/used/sale/audi/a6/1104810068-911371b7/ 1 250 000 ₽
120 https://auto.ru/cars/used/sale/audi/a5/1105195265-a9e6e5fe/ 3 078 000 ₽
121 https://auto.ru/cars/used/sale/audi/q3/1105664211-202a1e59/ 1 688 000 ₽
122 https://auto.ru/cars/used/sale/audi/q7/1105664212-90512ea2/ 3 447 000 ₽
123 https://auto.ru/cars/used/sale/audi/q5/1105241491-723f9a07/ 1 149 000 ₽
124 https://auto.ru/cars/used/sale/audi/a6/1105253159-f5693ec9/ 2 448 000 ₽
125 https://auto.ru/cars/used/sale/audi/a5/1104267990-7a05bae5/ 3 800 000 ₽
126 https://auto.ru/cars/used/sale/audi/a4/1105505356-12afc9a4/ 530 000 ₽
127 https://auto.ru/cars/used/sale/audi/a5/1105119829-3797444c/ 1 598 000 ₽
128 https://au

229 https://auto.ru/cars/used/sale/audi/q7/1105487228-331aabe2/ 1 200 000 ₽
230 https://auto.ru/cars/used/sale/audi/a4/1105545003-58cc8a77/ 1 065 000 ₽
231 https://auto.ru/cars/used/sale/audi/a8/1105420316-03f6a9a4/ 1 395 000 ₽
232 https://auto.ru/cars/used/sale/audi/q7/1105660297-0cd80b6e/ 3 629 000 ₽
233 https://auto.ru/cars/used/sale/audi/a8/1105415225-e6349dea/ 5 550 000 ₽
234 https://auto.ru/cars/used/sale/audi/a4/1105661866-4816d243/ 1 199 000 ₽
235 https://auto.ru/cars/used/sale/audi/a3/1105659696-b7354a50/ 1 020 000 ₽
236 https://auto.ru/cars/used/sale/audi/a4/1104724000-80145ee8/ 190 000 ₽
238 https://auto.ru/cars/used/sale/audi/a8/1104090991-4509083b/ 9 300 000 ₽
239 https://auto.ru/cars/used/sale/audi/q5/1105656656-bbc3866f/ 3 850 000 ₽
240 https://auto.ru/cars/used/sale/audi/a8/1105657040-ed8e46d6/ 1 450 000 ₽
241 https://auto.ru/cars/used/sale/audi/a8/1103180193-97ad1581/ 770 000 ₽
242 https://auto.ru/cars/used/sale/audi/a4/1105659924-9e4b9f49/ 4 437 900 ₽
243 https://auto

347 https://auto.ru/cars/used/sale/audi/tt/1105665909-49a9ec83/ 2 700 000 ₽
348 https://auto.ru/cars/used/sale/audi/e_tron/1105576711-524a4b73/ 6 150 000 ₽
350 https://auto.ru/cars/used/sale/audi/q5/1105229105-5535a433/ 3 100 000 ₽
351 https://auto.ru/cars/used/sale/audi/q3/1105665918-5fba70f2/ 1 650 000 ₽
352 https://auto.ru/cars/used/sale/audi/q5/1105654341-c889f54f/ 1 050 000 ₽
353 https://auto.ru/cars/used/sale/audi/a4/1105625967-33233429/ 385 000 ₽
354 https://auto.ru/cars/used/sale/audi/allroad/1105654308-a67066c0/ 650 000 ₽
355 https://auto.ru/cars/used/sale/audi/q7/1105403481-086191b5/ 3 669 000 ₽
356 https://auto.ru/cars/used/sale/audi/80/1103824567-4678b30d/ 100 000 ₽
357 https://auto.ru/cars/used/sale/audi/a6/1105172186-96497d3c/ 630 000 ₽
358 https://auto.ru/cars/used/sale/audi/q7/1105577367-6477a76e/ 4 590 000 ₽
359 https://auto.ru/cars/used/sale/audi/a8/1105649592-37fd0d66/ 5 700 000 ₽
360 https://auto.ru/cars/used/sale/audi/a3/1105653898-1d32e40e/ 155 000 ₽
361 https://a

462 https://auto.ru/cars/used/sale/audi/tt/1105222462-80eef45f/ 2 599 000 ₽
463 https://auto.ru/cars/used/sale/audi/tt/1105193516-b0ab4ddb/ 2 650 000 ₽
465 https://auto.ru/cars/used/sale/audi/80/1105407480-d211b3a3/ 149 000 ₽
466 https://auto.ru/cars/used/sale/audi/a6/1105332431-e7736f11/ 350 000 ₽
469 https://auto.ru/cars/used/sale/audi/a4/1105514808-96b80841/ 1 200 000 ₽
470 https://auto.ru/cars/used/sale/audi/a5/1105475508-05fbe6dd/ 2 199 000 ₽
471 https://auto.ru/cars/used/sale/audi/a4/1105545781-d6e4b146/ 3 049 000 ₽
472 https://auto.ru/cars/used/sale/audi/a3/1105596584-0faa6e55/ 1 555 000 ₽
473 https://auto.ru/cars/used/sale/audi/a4/1105297040-65859ec5/ 335 000 ₽
474 https://auto.ru/cars/used/sale/audi/a4/1105129011-38a30e50/ 2 139 000 ₽
475 https://auto.ru/cars/used/sale/audi/q7/1105385828-eaeddd4b/ 1 649 000 ₽
476 https://auto.ru/cars/used/sale/audi/q7/1105533089-38d9e74b/ 3 387 565 ₽
477 https://auto.ru/cars/used/sale/audi/a1/1105556413-fbfda86b/ 1 299 000 ₽
478 https://auto.r

...

2443 https://auto.ru/cars/used/sale/audi/a6/1105383911-667a37a5/ 280 000 ₽
2444 https://auto.ru/cars/used/sale/audi/q3/1105321748-8bf5bca7/ 2 949 000 ₽
2445 https://auto.ru/cars/used/sale/audi/q7/1105520712-48e7e6df/ 4 300 000 ₽
2446 https://auto.ru/cars/used/sale/audi/a8/1104107989-f0e217fa/ 3 000 000 ₽
2447 https://auto.ru/cars/used/sale/audi/a4/1105604370-f77aa0b1/ 600 000 ₽
2448 https://auto.ru/cars/used/sale/audi/q7/1103911996-3cd517ea/ 869 000 ₽
2449 https://auto.ru/cars/used/sale/audi/a3/1105390288-fb2a73ff/ 515 000 ₽
2450 https://auto.ru/cars/used/sale/audi/a6/1103922621-269870cc/ 5 200 000 ₽
2451 https://auto.ru/cars/used/sale/audi/q8/1105239445-f21612ef/ 7 190 000 ₽
2452 https://auto.ru/cars/used/sale/audi/a6/1105040111-60cc4c54/ 370 000 ₽
2453 https://auto.ru/cars/used/sale/audi/a6/1105324356-8a53da51/ 550 000 ₽
2454 https://auto.ru/cars/used/sale/audi/rs5/1105643096-71fe8d00/ 1 550 000 ₽
2455 https://auto.ru/cars/used/sale/audi/q5/1105007498-2101e084/ 1 399 000 ₽
2456 https

2554 https://auto.ru/cars/used/sale/audi/a3/1105642111-af924358/ 1 100 000 ₽
2555 https://auto.ru/cars/used/sale/audi/a8/1103924762-1625a4da/ 5 445 000 ₽
2556 https://auto.ru/cars/used/sale/audi/r8/1102702023-8ccd6e55/ 4 900 000 ₽
2557 https://auto.ru/cars/used/sale/audi/a4/1105642571-7be17b65/ 1 399 999 ₽
2558 https://auto.ru/cars/used/sale/audi/q7/1105321281-8e4f0145/ 2 189 000 ₽
2559 https://auto.ru/cars/used/sale/audi/q3/1105642516-715f3df4/ 1 200 000 ₽
2560 https://auto.ru/cars/used/sale/audi/a6/1104042906-369c5d0e/ 1 200 000 ₽
2561 https://auto.ru/cars/used/sale/audi/rs6/1105411414-8266e762/ 3 200 000 ₽
2562 https://auto.ru/cars/used/sale/audi/q5/1105641264-81e26596/ 3 500 000 ₽
2564 https://auto.ru/cars/used/sale/audi/a5/1103230311-b2226bc5/ 800 000 ₽
2565 https://auto.ru/cars/used/sale/audi/a6/1099528854-70e31aba/ 2 100 000 ₽
2566 https://auto.ru/cars/used/sale/audi/a6/1103494669-1782f382/ 2 500 000 ₽
2567 https://auto.ru/cars/used/sale/audi/200/1105641931-22b6c9f5/ 450 000 ₽
2

2666 https://auto.ru/cars/used/sale/audi/rs5/1105607606-9a5e97af/ 6 900 000 ₽
2667 https://auto.ru/cars/used/sale/audi/rs6/1104955891-5b01d76b/ 6 500 000 ₽
2668 https://auto.ru/cars/used/sale/audi/q3/1104992611-dcd3e893/ 3 700 000 ₽
2669 https://auto.ru/cars/used/sale/audi/s4/1103306919-5bd3b80f/ 700 000 ₽
2670 https://auto.ru/cars/used/sale/audi/a4/1090116196-7f46cd46/ 580 000 ₽
2671 https://auto.ru/cars/used/sale/audi/q5/1104828802-bab18684/ 930 000 ₽
2672 https://auto.ru/cars/used/sale/audi/tt_rs/1104163562-f69310b8/ 4 550 000 ₽
2673 https://auto.ru/cars/used/sale/audi/a6/1105366060-37b038af/ 650 000 ₽
2674 https://auto.ru/cars/used/sale/audi/a6/1104972113-fd60b4f0/ 815 000 ₽
2675 https://auto.ru/cars/used/sale/audi/q3/1103005264-35a0fc57/ 1 570 000 ₽
2676 https://auto.ru/cars/used/sale/audi/q5/1103699953-d0ec0f80/ 950 000 ₽
2677 https://auto.ru/cars/used/sale/audi/a3/1105635948-8baa9ee5/ 550 000 ₽
2678 https://auto.ru/cars/used/sale/audi/80/1105604267-ead96af2/ 170 000 ₽
2679 https

2774 https://auto.ru/cars/used/sale/audi/80/1105067310-83a2a53d/ 50 000 ₽
2775 https://auto.ru/cars/used/sale/audi/80/1105137305-6efbc4ed/ 99 000 ₽
2776 https://auto.ru/cars/used/sale/audi/q5/1105155394-a03b1c9f/ 1 400 000 ₽
2777 https://auto.ru/cars/used/sale/audi/a4/1104872235-0a65fd9b/ 1 100 000 ₽
2778 https://auto.ru/cars/used/sale/audi/q3/1104607124-c31b5aac/ 1 290 000 ₽
2779 https://auto.ru/cars/used/sale/audi/a8/1105301317-9c22766c/ 1 700 000 ₽
2780 https://auto.ru/cars/used/sale/audi/a4/1105640502-898af23f/ 175 000 ₽
2782 https://auto.ru/cars/used/sale/audi/q5/1104630464-49dc2bb2/ 1 100 000 ₽
2783 https://auto.ru/cars/used/sale/audi/100/1104968737-86aac746/ 159 000 ₽
2784 https://auto.ru/cars/used/sale/audi/a8/1104902971-45ea0cd1/ 1 290 000 ₽
2785 https://auto.ru/cars/used/sale/audi/q5/1105642113-522a350f/ 1 300 000 ₽
2786 https://auto.ru/cars/used/sale/audi/a3/1105226385-a9669a32/ 535 000 ₽
2787 https://auto.ru/cars/used/sale/audi/a5/1104991739-9f515f49/ 1 550 000 ₽
2788 https

KeyError: "name='from_lifetime', domain=None, path=None"

In [12]:
get_content("Audi", 2800)

AUDI: 3503 urls to parse
starting from ...2800
creating CONTENT/AUDI_2800_content.csv
2802 https://auto.ru/cars/used/sale/audi/q5/1105423905-42799f83/ 1 180 000 ₽
2803 https://auto.ru/cars/used/sale/audi/a6/1105484111-22eb31ec/ 1 300 000 ₽
2804 https://auto.ru/cars/used/sale/audi/q5/1105344311-876092cb/ 3 820 000 ₽
2805 https://auto.ru/cars/used/sale/audi/a8/1105570418-f6ece493/ 1 299 000 ₽
2806 https://auto.ru/cars/used/sale/audi/a6/1105194045-d321447b/ 650 000 ₽
2807 https://auto.ru/cars/used/sale/audi/a6/1105360909-e6f98cea/ 325 000 ₽
2808 https://auto.ru/cars/used/sale/audi/a7/1101883669-08cf061f/ 4 600 000 ₽
2809 https://auto.ru/cars/used/sale/audi/q3/1105071591-61c89acf/ 1 750 000 ₽
2810 https://auto.ru/cars/used/sale/audi/q5/1102667070-6dc956c9/ 2 050 000 ₽
2811 https://auto.ru/cars/used/sale/audi/q7/1103514481-5f046fde/ 850 000 ₽
2812 https://auto.ru/cars/used/sale/audi/q3/1103912540-8ece3472/ 1 250 000 ₽
2813 https://auto.ru/cars/used/sale/audi/a6/1102628685-05053b9d/ 620 000 

2909 https://auto.ru/cars/used/sale/audi/a6/1105506070-f50cbbb5/ 1 200 000 ₽
2910 https://auto.ru/cars/used/sale/audi/sq5/1103802738-931a3d23/ 2 500 000 ₽
2911 https://auto.ru/cars/used/sale/audi/a4/1105103751-a0323c68/ 210 000 ₽
2912 https://auto.ru/cars/used/sale/audi/80/1080880246-b149dd/ 125 000 ₽
2914 https://auto.ru/cars/used/sale/audi/a3/1103038300-86bd6d98/ 620 000 ₽
2915 https://auto.ru/cars/used/sale/audi/a8/1082582704-f3d92a3a/ 695 000 ₽
2916 https://auto.ru/cars/used/sale/audi/a7/1072468766-bbc436c7/ 1 370 000 ₽
2917 https://auto.ru/cars/used/sale/audi/a4/1103178486-d195a437/ 899 000 ₽
2918 https://auto.ru/cars/used/sale/audi/a4/1101900128-53b72c99/ 230 000 ₽
2919 https://auto.ru/cars/used/sale/audi/80/1105573881-c7366d3f/ 80 000 ₽
2920 https://auto.ru/cars/used/sale/audi/80/1105027115-03d55488/ 80 000 ₽
2921 https://auto.ru/cars/used/sale/audi/a6/1105189302-5a930a23/ 750 000 ₽
2922 https://auto.ru/cars/used/sale/audi/100/1098656846-53d90611/ 160 000 ₽
2923 https://auto.ru/

3021 https://auto.ru/cars/used/sale/audi/a6/1105250432-9048cb92/ 550 000 ₽
3022 https://auto.ru/cars/used/sale/audi/q7/1104271163-103f5edd/ 4 200 000 ₽
3023 https://auto.ru/cars/used/sale/audi/a6/1102710768-aa06f641/ 1 550 000 ₽
3024 https://auto.ru/cars/used/sale/audi/80/1105576716-31505686/ 100 000 ₽
3025 https://auto.ru/cars/used/sale/audi/q7/1105470798-f23835ec/ 1 150 000 ₽
3026 https://auto.ru/cars/used/sale/audi/q7/1105499016-4887bfad/ 990 000 ₽
3027 https://auto.ru/cars/used/sale/audi/100/1105458757-08e49b51/ 200 000 ₽
3028 https://auto.ru/cars/used/sale/audi/a3/1104638163-5ff3a1d2/ 600 000 ₽
3029 https://auto.ru/cars/used/sale/audi/80/1098558348-9e75b5cc/ 100 000 ₽
3030 https://auto.ru/cars/used/sale/audi/a6/1101125367-2f91aa4f/ 830 000 ₽
3031 https://auto.ru/cars/used/sale/audi/s7/1104961706-1d5df4c7/ 3 400 000 ₽
3032 https://auto.ru/cars/used/sale/audi/q3/1105572668-3b867ce5/ 1 290 000 ₽
3033 https://auto.ru/cars/used/sale/audi/80/1104823368-9de2f55f/ 120 000 ₽
3034 https://a

3131 https://auto.ru/cars/used/sale/audi/100/1105183675-3c9181cb/ 135 000 ₽
3132 https://auto.ru/cars/used/sale/audi/s4/1105227457-1e0aa887/ 2 000 000 ₽
3133 https://auto.ru/cars/used/sale/audi/q3/1105567772-2483b535/ 1 630 000 ₽
3134 https://auto.ru/cars/used/sale/audi/a3/1102711600-74a031f1/ 1 140 000 ₽
3135 https://auto.ru/cars/used/sale/audi/q5/1104967553-42d782df/ 1 690 000 ₽
3136 https://auto.ru/cars/used/sale/audi/100/1104794448-86addccf/ 200 000 ₽
3137 https://auto.ru/cars/used/sale/audi/a6/1105099164-539b9f83/ 305 000 ₽
3138 https://auto.ru/cars/used/sale/audi/q5/1091630916-73c5f5c9/ 1 650 000 ₽
3139 https://auto.ru/cars/used/sale/audi/a4/1105510589-2bac0b60/ 200 000 ₽
3140 https://auto.ru/cars/used/sale/audi/tt/1101947195-a1c98f1f/ 2 900 000 ₽
3141 https://auto.ru/cars/used/sale/audi/a6/1105172757-4c37897f/ 970 000 ₽
3142 https://auto.ru/cars/used/sale/audi/q5/1076367380-5542165e/ 933 000 ₽
3143 https://auto.ru/cars/used/sale/audi/q7/1105396854-84e8ed97/ 1 750 000 ₽
3144 http

3241 https://auto.ru/cars/used/sale/audi/a6/1105061616-4c150456/ 360 000 ₽
3242 https://auto.ru/cars/used/sale/audi/rsq3/1105353180-7811c9f0/ 2 590 000 ₽
3243 https://auto.ru/cars/used/sale/audi/a4/1105174364-35fa30f9/ 745 000 ₽
3244 https://auto.ru/cars/used/sale/audi/a6/1105266557-03b0af01/ 1 000 000 ₽
3245 https://auto.ru/cars/used/sale/audi/a1/1105476784-2c07174e/ 1 000 000 ₽
3246 https://auto.ru/cars/used/sale/audi/a7/1105337737-bd787b78/ 1 420 000 ₽
3247 https://auto.ru/cars/used/sale/audi/a4/1105525971-8fd22679/ 390 000 ₽
3248 https://auto.ru/cars/used/sale/audi/a4_allroad/1082573116-3684f/ 720 000 ₽
3249 https://auto.ru/cars/used/sale/audi/a6/1101806691-85c8724a/ 730 000 ₽
3250 https://auto.ru/cars/used/sale/audi/q7/1105158913-3e36cbb7/ 1 046 599 ₽
3251 https://auto.ru/cars/used/sale/audi/rs5/1104186799-e3760c1d/ 1 715 000 ₽
3252 https://auto.ru/cars/used/sale/audi/s8/1091224992-78ea2765/ 870 000 ₽
3253 https://auto.ru/cars/used/sale/audi/q7/1099159718-105b7598/ 1 549 000 ₽
325

3349 https://auto.ru/cars/used/sale/audi/a6/1104153900-34711207/ 1 299 000 ₽
3350 https://auto.ru/cars/used/sale/audi/q7/1007384869-338682/ 1 650 000 ₽
3351 https://auto.ru/cars/used/sale/audi/a4/1098141312-300a5e0a/ 450 000 ₽
3352 https://auto.ru/cars/used/sale/audi/a6/1103882180-3a2cad52/ 1 290 000 ₽
3353 https://auto.ru/cars/used/sale/audi/a7/1104706405-88064374/ 1 400 000 ₽
3354 https://auto.ru/cars/used/sale/audi/a8/1100097724-b2ea9262/ 1 300 000 ₽
3355 https://auto.ru/cars/used/sale/audi/a8/1105107678-886cc75b/ 400 000 ₽
3356 https://auto.ru/cars/used/sale/audi/80/1105340466-f4a04464/ 170 000 ₽


KeyError: "name='from_lifetime', domain=None, path=None"

In [13]:
get_content("Audi", 3300)

AUDI: 3503 urls to parse
starting from ...3300
creating CONTENT/AUDI_3300_content.csv
3301 https://auto.ru/cars/used/sale/audi/q5/1105617070-0ede1283/ 1 790 000 ₽
3302 https://auto.ru/cars/used/sale/audi/q3/1104708054-7f15d09f/ 1 450 000 ₽
3303 https://auto.ru/cars/used/sale/audi/a5/1104221858-7a7e1e44/ 3 100 000 ₽
3304 https://auto.ru/cars/used/sale/audi/rs5/1102225040-7bae6b88/ 2 000 000 ₽
3305 https://auto.ru/cars/used/sale/audi/a3/1103589017-5ed2ba4a/ 2 275 000 ₽
3306 https://auto.ru/cars/used/sale/audi/q8/1102969285-9e7c3365/ 8 250 000 ₽
3307 https://auto.ru/cars/used/sale/audi/rsq3/1104998135-e92b9a67/ 2 950 000 ₽
3308 https://auto.ru/cars/used/sale/audi/q7/1103627111-73e0a20e/ 1 390 000 ₽
3309 https://auto.ru/cars/used/sale/audi/100/1100195468-719c2054/ 160 000 ₽
3310 https://auto.ru/cars/used/sale/audi/a6/1103433111-a64b77a5/ 405 000 ₽
3311 https://auto.ru/cars/used/sale/audi/a6/1105233638-0654abcf/ 215 000 ₽
3312 https://auto.ru/cars/used/sale/audi/a5/1105397773-bd312e42/ 900 

3408 https://auto.ru/cars/used/sale/audi/q7/1100885636-eb0044ca/ 2 050 000 ₽
3409 https://auto.ru/cars/used/sale/audi/rs6/1105494261-d7980705/ 7 000 000 ₽
3410 https://auto.ru/cars/used/sale/audi/a6/1104920486-b0f7f081/ 1 845 000 ₽
3411 https://auto.ru/cars/used/sale/audi/80/1103044986-7e7e861c/ 130 000 ₽
3412 https://auto.ru/cars/used/sale/audi/a5/1101608928-b7990400/ 2 190 000 ₽
3413 https://auto.ru/cars/used/sale/audi/q8/1102539359-247a7e92/ 9 000 000 ₽
3414 https://auto.ru/cars/used/sale/audi/q3/1105186215-0e8bca5d/ 1 099 000 ₽
3415 https://auto.ru/cars/used/sale/audi/q7/1104170305-9a501a88/ 1 550 000 ₽
3416 https://auto.ru/cars/used/sale/audi/a6/1102033463-c8428e7e/ 550 000 ₽
3417 https://auto.ru/cars/used/sale/audi/a6/1105555870-d14a6d5e/ 360 000 ₽
3418 https://auto.ru/cars/used/sale/audi/a6/1105044512-0f125c49/ 2 650 000 ₽
3419 https://auto.ru/cars/used/sale/audi/a6/1104895749-ea7488be/ 1 285 000 ₽
3420 https://auto.ru/cars/used/sale/audi/q5/1094402470-5e9af95d/ 2 300 000 ₽
3421

In [14]:
get_content("Honda")

HONDA: 3511 urls to parse
starting from the beginning...
creating CONTENT/HONDA_content.csv
1 https://auto.ru/cars/used/sale/honda/freed/1105632666-b724a65d/ 735 000 ₽
2 https://auto.ru/cars/used/sale/honda/cr_v/1105511067-9d1490ce/ 285 000 ₽
3 https://auto.ru/cars/used/sale/honda/cr_v/1105543170-d1e704f3/ 1 150 000 ₽
4 https://auto.ru/cars/used/sale/honda/accord/1105070161-c7242d14/ 670 000 ₽
5 https://auto.ru/cars/used/sale/honda/accord/1105305983-deae3a64/ 220 000 ₽
6 https://auto.ru/cars/used/sale/honda/accord/1105659693-08240cb5/ 830 000 ₽
7 https://auto.ru/cars/used/sale/honda/accord/1105551093-cfd53008/ 780 000 ₽
8 https://auto.ru/cars/used/sale/honda/crosstour/1105411259-dc8b85bd/ 1 550 000 ₽
9 https://auto.ru/cars/used/sale/honda/vezel/1104934004-197b85c7/ 980 000 ₽
10 https://auto.ru/cars/used/sale/honda/civic/1105656616-dcd9d804/ 670 000 ₽
11 https://auto.ru/cars/used/sale/honda/accord/1105483271-7e538677/ 1 750 000 ₽
12 https://auto.ru/cars/used/sale/honda/civic/1105554496-

116 https://auto.ru/cars/used/sale/honda/accord/1105659829-2ff2c8e2/ 1 279 000 ₽
117 https://auto.ru/cars/used/sale/honda/freed/1105654593-3459ed48/ 690 000 ₽
118 https://auto.ru/cars/used/sale/honda/civic/1105661336-2a3f4274/ 648 000 ₽
119 https://auto.ru/cars/used/sale/honda/civic/1105470890-a85baa49/ 1 550 000 ₽
120 https://auto.ru/cars/used/sale/honda/accord/1105478515-ebddec6d/ 675 000 ₽
121 https://auto.ru/cars/used/sale/honda/logo/1105090994-28ebdc72/ 180 000 ₽
122 https://auto.ru/cars/used/sale/honda/accord/1105665794-63d6e23b/ 1 128 000 ₽
125 https://auto.ru/cars/used/sale/honda/jazz/1105386043-2dbca256/ 550 000 ₽
126 https://auto.ru/cars/used/sale/honda/vezel/1105577789-47879dca/ 1 260 000 ₽
127 https://auto.ru/cars/used/sale/honda/shuttle/1105160390-3e4045a1/ 1 210 000 ₽
128 https://auto.ru/cars/used/sale/honda/fit/1089753824-21503791/ 1 390 000 ₽
129 https://auto.ru/cars/used/sale/honda/hr_v/1105037686-bf6fb324/ 590 000 ₽
130 https://auto.ru/cars/used/sale/honda/cr_v/110547

235 https://auto.ru/cars/used/sale/honda/odyssey/1105356353-5e05ba50/ 290 000 ₽
236 https://auto.ru/cars/used/sale/honda/n_box_slash/1105457102-0c9a3b19/ 550 000 ₽
237 https://auto.ru/cars/used/sale/honda/cr_v/1105489484-027d14bc/ 315 000 ₽
238 https://auto.ru/cars/used/sale/honda/hr_v/1105649075-990af36d/ 495 000 ₽
239 https://auto.ru/cars/used/sale/honda/fit_shuttle/1105652412-d223417a/ 887 000 ₽
240 https://auto.ru/cars/used/sale/honda/accord/1105644434-b632d51c/ 835 000 ₽
241 https://auto.ru/cars/used/sale/honda/cr_v/1105649207-ae91cfb8/ 1 260 000 ₽
242 https://auto.ru/cars/used/sale/honda/cr_v/1105543367-62f5430d/ 669 000 ₽
243 https://auto.ru/cars/used/sale/honda/civic/1105571024-bc1d27af/ 180 000 ₽
244 https://auto.ru/cars/used/sale/honda/cr_v/1105649125-7181277e/ 1 399 000 ₽
245 https://auto.ru/cars/used/sale/honda/civic/1105399256-0067e636/ 535 000 ₽
246 https://auto.ru/cars/used/sale/honda/logo/1105648760-2c425699/ 180 000 ₽
247 https://auto.ru/cars/used/sale/honda/jazz/11054

347 https://auto.ru/cars/used/sale/honda/civic/1104200389-dce8d3a3/ 549 000 ₽
348 https://auto.ru/cars/used/sale/honda/accord/1105639311-43da2ecb/ 640 000 ₽
349 https://auto.ru/cars/used/sale/honda/accord/1104755635-55c6ba3c/ 1 195 000 ₽
350 https://auto.ru/cars/used/sale/honda/cr_v/1105148637-6a568123/ 1 849 000 ₽
352 https://auto.ru/cars/used/sale/honda/cr_v/1105639571-1f44871f/ 475 000 ₽
353 https://auto.ru/cars/used/sale/honda/stream/1102297351-ee3cc32a/ 347 000 ₽
354 https://auto.ru/cars/used/sale/honda/cr_v/1105441248-d58d672a/ 1 495 000 ₽
355 https://auto.ru/cars/used/sale/honda/accord/1105638966-7b2d210f/ 710 000 ₽
356 https://auto.ru/cars/used/sale/honda/cr_v/1105643383-14b45b3e/ 1 619 000 ₽
357 https://auto.ru/cars/used/sale/honda/n_box/1105414735-20ad5aa5/ 770 000 ₽
358 https://auto.ru/cars/used/sale/honda/accord/1105637612-dc16b83f/ 1 275 000 ₽
359 https://auto.ru/cars/used/sale/honda/stream/1105638744-0f5261b7/ 369 000 ₽
360 https://auto.ru/cars/used/sale/honda/shuttle/110

458 https://auto.ru/cars/used/sale/honda/civic/1103795865-5e3c6f16/ 570 000 ₽
459 https://auto.ru/cars/used/sale/honda/orthia/1105629537-08f3d53c/ 179 000 ₽
460 https://auto.ru/cars/used/sale/honda/civic/1105492267-417d725e/ 490 000 ₽
461 https://auto.ru/cars/used/sale/honda/fit/1105627822-b6c5823d/ 810 000 ₽
462 https://auto.ru/cars/used/sale/honda/cr_v/1105274979-47523f56/ 639 000 ₽
463 https://auto.ru/cars/used/sale/honda/freed/1105522918-b920b56b/ 645 000 ₽
464 https://auto.ru/cars/used/sale/honda/accord/1104922103-cb1b05bd/ 1 287 000 ₽
465 https://auto.ru/cars/used/sale/honda/civic/1105453976-e858145e/ 475 000 ₽
466 https://auto.ru/cars/used/sale/honda/cr_v/1105189074-de257574/ 1 595 000 ₽
467 https://auto.ru/cars/used/sale/honda/shuttle/1105627769-40afa321/ 780 000 ₽
468 https://auto.ru/cars/used/sale/honda/cr_v/1105627144-ee9314d0/ 3 350 000 ₽
469 https://auto.ru/cars/used/sale/honda/n_wgn/1105062830-9c97c7a4/ 627 000 ₽
470 https://auto.ru/cars/used/sale/honda/accord/1105017745-

...

2984 https://auto.ru/cars/used/sale/honda/jazz/1105004903-47a46385/ 399 999 ₽
2985 https://auto.ru/cars/used/sale/honda/prelude/1105545528-c786de00/ 250 000 ₽
2986 https://auto.ru/cars/used/sale/honda/cr_v/1103970672-37ae12aa/ 1 000 000 ₽
2987 https://auto.ru/cars/used/sale/honda/cr_v/1104925495-c70336b8/ 750 000 ₽
2988 https://auto.ru/cars/used/sale/honda/fit/1105610434-2c2f6672/ 420 000 ₽
2989 https://auto.ru/cars/used/sale/honda/cr_v/1105001270-49f383da/ 150 000 ₽
2990 https://auto.ru/cars/used/sale/honda/accord/1105490209-ec101711/ 450 000 ₽
2991 https://auto.ru/cars/used/sale/honda/accord/1105010795-77dd8da8/ 570 000 ₽
2992 https://auto.ru/cars/used/sale/honda/civic/1104092778-5c7e868c/ 440 000 ₽
2993 https://auto.ru/cars/used/sale/honda/jazz/1105156023-3fd4b974/ 490 000 ₽
2994 https://auto.ru/cars/used/sale/honda/integra/1105532183-5850e2ba/ 750 000 ₽
2995 https://auto.ru/cars/used/sale/honda/civic/1094704638-dbd61afc/ 730 000 ₽
2996 https://auto.ru/cars/used/sale/honda/accord/11

3089 https://auto.ru/cars/used/sale/honda/accord/1093834716-90d7a55e/ 450 000 ₽
3090 https://auto.ru/cars/used/sale/honda/hr_v/1104701856-6fd353fc/ 375 000 ₽
3091 https://auto.ru/cars/used/sale/honda/civic/1103150008-360fbddf/ 860 000 ₽
3092 https://auto.ru/cars/used/sale/honda/civic/1104192794-ad7d5063/ 490 000 ₽
3093 https://auto.ru/cars/used/sale/honda/accord/1102449162-5ee38b69/ 690 000 ₽
3094 https://auto.ru/cars/used/sale/honda/civic/1105039647-c14b2e31/ 450 000 ₽
3095 https://auto.ru/cars/used/sale/honda/civic/1105539977-fef29119/ 240 000 ₽
3096 https://auto.ru/cars/used/sale/honda/cr_v/1103854759-7ff6e7c7/ 910 000 ₽
3097 https://auto.ru/cars/used/sale/honda/civic/1102679067-36ac03c0/ 600 000 ₽
3098 https://auto.ru/cars/used/sale/honda/civic/1104819687-a357299e/ 530 000 ₽
3099 https://auto.ru/cars/used/sale/honda/cr_v/1104904768-9d3d65be/ 2 450 000 ₽
3100 links parsed
adding to CONTENT/HONDA_content.csv
3100 https://auto.ru/cars/used/sale/honda/fit/1105437693-3b6f2b4d/ 485 000 ₽

3193 https://auto.ru/cars/used/sale/honda/accord/1095332260-b37d953c/ 1 450 000 ₽
3194 https://auto.ru/cars/used/sale/honda/freed/1105295911-f0e3204e/ 790 000 ₽
3195 https://auto.ru/cars/used/sale/honda/cr_v/1104713471-f83182cc/ 200 000 ₽
3196 https://auto.ru/cars/used/sale/honda/civic/1105485383-f839277e/ 100 000 ₽
3197 https://auto.ru/cars/used/sale/honda/accord/1104960319-abb72e8d/ 830 000 ₽
3198 https://auto.ru/cars/used/sale/honda/cr_v/1105250697-52622082/ 500 000 ₽
3199 https://auto.ru/cars/used/sale/honda/fit/1104578884-f1846811/ 850 000 ₽
3200 links parsed
adding to CONTENT/HONDA_content.csv
3200 https://auto.ru/cars/used/sale/honda/civic/1105488553-02617cbd/ 950 000 ₽
3201 https://auto.ru/cars/used/sale/honda/accord/1102352594-20f7182c/ 415 000 ₽
3202 https://auto.ru/cars/used/sale/honda/accord/1105568989-73bf5a8b/ 1 050 000 ₽
3203 https://auto.ru/cars/used/sale/honda/ascot/1103996062-5710f54f/ 140 000 ₽
3204 https://auto.ru/cars/used/sale/honda/civic/1101964189-d2a85387/ 710 

3300 https://auto.ru/cars/used/sale/honda/pilot/1105302825-0505d377/ 870 000 ₽
3301 https://auto.ru/cars/used/sale/honda/accord/1104032508-a278810b/ 620 000 ₽
3302 https://auto.ru/cars/used/sale/honda/civic_type_r/1105061813-faaa4f4c/ 875 000 ₽
3303 https://auto.ru/cars/used/sale/honda/hr_v/1105332416-25367854/ 338 000 ₽
3304 https://auto.ru/cars/used/sale/honda/cr_v/1095404352-ca92fe7d/ 220 000 ₽
3305 https://auto.ru/cars/used/sale/honda/cr_v/1105351544-77e05945/ 2 150 000 ₽
3308 https://auto.ru/cars/used/sale/honda/civic/1104712607-395597ce/ 350 000 ₽
3309 https://auto.ru/cars/used/sale/honda/vamos/1105473047-c052be93/ 500 000 ₽
3310 https://auto.ru/cars/used/sale/honda/civic/1105587872-d98b6045/ 429 000 ₽
3311 https://auto.ru/cars/used/sale/honda/cr_v/1102357319-94d3ec35/ 560 000 ₽
3312 https://auto.ru/cars/used/sale/honda/accord/1105222532-c82b71b4/ 425 000 ₽
3313 https://auto.ru/cars/used/sale/honda/accord/1105300748-bdec27bb/ 66 000 ₽
3314 https://auto.ru/cars/used/sale/honda/civ

3407 https://auto.ru/cars/used/sale/honda/civic/1097333926-50a40ef0/ 500 000 ₽
3408 https://auto.ru/cars/used/sale/honda/accord/1104171280-b4bcb64e/ 650 000 ₽
3409 https://auto.ru/cars/used/sale/honda/fit/1102836588-383d4d6e/ 780 000 ₽
3411 https://auto.ru/cars/used/sale/honda/cr_v/1103951295-4943399c/ 250 000 ₽
3412 https://auto.ru/cars/used/sale/honda/civic/1097920088-310f5891/ 715 000 ₽
3413 https://auto.ru/cars/used/sale/honda/cr_v/1101492473-2d0cc528/ 1 650 000 ₽
3414 https://auto.ru/cars/used/sale/honda/accord/1105133045-cb0af243/ 890 000 ₽
3415 https://auto.ru/cars/used/sale/honda/civic/1105352475-7ec82e5a/ 700 000 ₽
3416 https://auto.ru/cars/used/sale/honda/civic/1105602438-597177fd/ 240 000 ₽
3417 https://auto.ru/cars/used/sale/honda/civic/1105202181-9b6f6b7e/ 168 000 ₽
3418 https://auto.ru/cars/used/sale/honda/cr_v/1104871814-4736eb7e/ 1 750 000 ₽
3419 https://auto.ru/cars/used/sale/honda/cr_v/1105484374-1cf8412d/ 2 200 000 ₽
3420 https://auto.ru/cars/used/sale/honda/cr_v/110

In [15]:
get_content("Volvo")

VOLVO: 2524 urls to parse
starting from the beginning...
creating CONTENT/VOLVO_content.csv
1 https://auto.ru/cars/used/sale/volvo/xc70/1105157565-f01bd666/ 535 000 ₽
2 https://auto.ru/cars/used/sale/volvo/850/1105463837-d84b7476/ 120 000 ₽
3 https://auto.ru/cars/used/sale/volvo/960/1103206094-139c9a9d/ 360 000 ₽
4 https://auto.ru/cars/used/sale/volvo/s80/1105498976-334bfb28/ 620 000 ₽
7 https://auto.ru/cars/used/sale/volvo/xc90/1105518719-815de96e/ 799 000 ₽
8 https://auto.ru/cars/used/sale/volvo/s40/1103182548-c945c1e3/ 260 000 ₽
9 https://auto.ru/cars/used/sale/volvo/s70/1103181789-62f52be6/ 290 000 ₽
10 https://auto.ru/cars/used/sale/volvo/s80/1103184952-ab7e265d/ 340 000 ₽
12 https://auto.ru/cars/used/sale/volvo/s80/1103178983-24ddb8ef/ 540 000 ₽
13 https://auto.ru/cars/used/sale/volvo/s40/1104618945-4e058add/ 455 000 ₽
14 https://auto.ru/cars/used/sale/volvo/s40/1103185807-a3d7865c/ 260 000 ₽
15 https://auto.ru/cars/used/sale/volvo/xc90/1105648005-c2904c45/ 615 000 ₽
16 https://a

117 https://auto.ru/cars/used/sale/volvo/xc70/1105656149-dd9fac21/ 1 449 000 ₽
118 https://auto.ru/cars/used/sale/volvo/xc90/1104136039-feecbd01/ 2 950 000 ₽
119 https://auto.ru/cars/used/sale/volvo/v90/1105663907-a13db7d3/ 2 359 000 ₽
120 https://auto.ru/cars/used/sale/volvo/s60/1105300738-57ac14b3/ 1 575 000 ₽
121 https://auto.ru/cars/used/sale/volvo/xc60/1105663240-dd45eb19/ 1 550 000 ₽
122 https://auto.ru/cars/used/sale/volvo/s60/1105658444-1e37cc4d/ 1 250 000 ₽
124 https://auto.ru/cars/used/sale/volvo/v40_cc/1105665038-6c219410/ 2 249 000 ₽
125 https://auto.ru/cars/used/sale/volvo/xc90/1105400870-cab90355/ 3 950 000 ₽
126 https://auto.ru/cars/used/sale/volvo/xc60/1105577405-1ae91d91/ 2 290 000 ₽
127 https://auto.ru/cars/used/sale/volvo/850/1105598351-ee19ee7a/ 169 000 ₽
128 https://auto.ru/cars/used/sale/volvo/xc60/1105422236-277d6a3d/ 3 399 000 ₽
130 https://auto.ru/cars/used/sale/volvo/xc90/1103962675-de18a7fb/ 3 130 000 ₽
131 https://auto.ru/cars/used/sale/volvo/xc70/1104919486

229 https://auto.ru/cars/used/sale/volvo/xc40/1105234758-b66c27b1/ 2 699 000 ₽
230 https://auto.ru/cars/used/sale/volvo/xc70/1105644128-42dbeb19/ 1 267 000 ₽
231 https://auto.ru/cars/used/sale/volvo/xc90/1105565804-77bae194/ 1 050 000 ₽
232 https://auto.ru/cars/used/sale/volvo/s80/1105321849-8d497671/ 545 000 ₽
233 https://auto.ru/cars/used/sale/volvo/v60/1105393162-59ea14eb/ 2 199 000 ₽
234 https://auto.ru/cars/used/sale/volvo/s60/1105664907-fd065a58/ 950 000 ₽
235 https://auto.ru/cars/used/sale/volvo/xc90/1105564326-5454fc02/ 695 000 ₽
236 https://auto.ru/cars/used/sale/volvo/s40/1105529746-327caa3f/ 350 000 ₽
237 https://auto.ru/cars/used/sale/volvo/v90_cross_country/1105645097-d43dd755/ 3 185 000 ₽
238 https://auto.ru/cars/used/sale/volvo/xc90/1105529866-ed2c39ac/ 739 000 ₽
240 https://auto.ru/cars/used/sale/volvo/s40/1105535396-07e31b39/ 700 000 ₽
241 https://auto.ru/cars/used/sale/volvo/xc90/1105644959-cc00bb65/ 1 349 000 ₽
242 https://auto.ru/cars/used/sale/volvo/xc60/1105009962

343 https://auto.ru/cars/used/sale/volvo/s80/1105404143-600aafe8/ 945 000 ₽
344 https://auto.ru/cars/used/sale/volvo/xc70/1104823529-c5553df6/ 1 180 000 ₽
345 https://auto.ru/cars/used/sale/volvo/xc60/1105107340-c1ff5869/ 1 099 000 ₽
346 https://auto.ru/cars/used/sale/volvo/v40_cc/1105007425-2d873fcf/ 1 565 000 ₽
347 https://auto.ru/cars/used/sale/volvo/xc60/1105631472-6d41b2de/ 2 650 000 ₽
349 https://auto.ru/cars/used/sale/volvo/s60/1105629413-55dc4e53/ 510 000 ₽
350 https://auto.ru/cars/used/sale/volvo/xc90/1105243599-b36ff2d4/ 1 360 000 ₽
351 https://auto.ru/cars/used/sale/volvo/xc90/1105633078-2ec57c88/ 5 839 900 ₽
352 https://auto.ru/cars/used/sale/volvo/xc90/1105628809-05696261/ 1 170 000 ₽
353 https://auto.ru/cars/used/sale/volvo/xc40/1105632916-993fad14/ 3 398 000 ₽
354 https://auto.ru/cars/used/sale/volvo/xc90/1105218327-09e5d7dc/ 3 300 000 ₽
355 https://auto.ru/cars/used/sale/volvo/xc90/1105290895-eb9b1fc2/ 1 085 000 ₽
356 https://auto.ru/cars/used/sale/volvo/xc70/1104085997

455 https://auto.ru/cars/used/sale/volvo/xc90/1105336632-e640368e/ 5 650 000 ₽
456 https://auto.ru/cars/used/sale/volvo/240_series/1103188967-4ef84ad3/ 500 000 ₽
457 https://auto.ru/cars/used/sale/volvo/xc60/1105613428-74cbc586/ 3 730 000 ₽
458 https://auto.ru/cars/used/sale/volvo/xc70/1105612121-02e04e82/ 845 000 ₽
459 https://auto.ru/cars/used/sale/volvo/v40_cc/1105611181-4884a353/ 1 750 000 ₽
460 https://auto.ru/cars/used/sale/volvo/s60/1105613300-1e0f362b/ 1 033 500 ₽
461 https://auto.ru/cars/used/sale/volvo/xc90/1105612654-9f01f120/ 780 000 ₽
463 https://auto.ru/cars/used/sale/volvo/xc90/1105390838-4062a82c/ 1 150 000 ₽
464 https://auto.ru/cars/used/sale/volvo/s80/1105504083-05641780/ 500 000 ₽
465 https://auto.ru/cars/used/sale/volvo/v40_cc/1105129027-9124d121/ 1 065 000 ₽
466 https://auto.ru/cars/used/sale/volvo/xc60/1105266001-eb0be247/ 3 634 000 ₽
467 https://auto.ru/cars/used/sale/volvo/xc90/1105607949-a23ff1f0/ 850 000 ₽
468 https://auto.ru/cars/used/sale/volvo/v90_cross_cou

...

2084 https://auto.ru/cars/used/sale/volvo/240_series/1103819503-25afb28d/ 50 000 ₽
2085 https://auto.ru/cars/used/sale/volvo/xc70/1105366445-8e3056c0/ 1 400 000 ₽
2086 https://auto.ru/cars/used/sale/volvo/s40/1105180142-02dd050e/ 345 000 ₽
2087 https://auto.ru/cars/used/sale/volvo/960/1102536976-ea918604/ 150 000 ₽
2088 https://auto.ru/cars/used/sale/volvo/c30/1105494652-a67e6534/ 445 000 ₽
2089 https://auto.ru/cars/used/sale/volvo/xc90/1105185330-48562ea0/ 1 300 000 ₽
2090 https://auto.ru/cars/used/sale/volvo/xc60/1105312137-ff863423/ 1 050 000 ₽
2091 https://auto.ru/cars/used/sale/volvo/740/1105620043-d5b44109/ 270 000 ₽
2092 https://auto.ru/cars/used/sale/volvo/c30/1105323945-04ab3997/ 675 000 ₽
2093 https://auto.ru/cars/used/sale/volvo/xc90/1105099146-68158eb9/ 1 609 900 ₽
2094 https://auto.ru/cars/used/sale/volvo/s40/1105585351-cb0ab9c4/ 170 000 ₽
2095 https://auto.ru/cars/used/sale/volvo/xc90/1105333012-86e66123/ 700 000 ₽
2096 https://auto.ru/cars/used/sale/volvo/xc90/1105382213

2190 https://auto.ru/cars/used/sale/volvo/s60/1104908230-85daa8cb/ 1 060 000 ₽
2191 https://auto.ru/cars/used/sale/volvo/960/1105194587-6cc61e82/ 200 000 ₽
2192 https://auto.ru/cars/used/sale/volvo/xc60/1104903648-db14c1f3/ 2 099 000 ₽
2195 https://auto.ru/cars/used/sale/volvo/s40/1105456306-60566036/ 480 000 ₽
2196 https://auto.ru/cars/used/sale/volvo/s60/1104964667-b2dd95eb/ 935 000 ₽
2197 https://auto.ru/cars/used/sale/volvo/s40/1105580992-5388062f/ 350 000 ₽
2198 https://auto.ru/cars/used/sale/volvo/240_series/1097359706-c73411ab/ 500 000 ₽
2199 https://auto.ru/cars/used/sale/volvo/c30/1105560660-6416a56c/ 360 000 ₽
2200 links parsed
adding to CONTENT/VOLVO_content.csv
2200 https://auto.ru/cars/used/sale/volvo/s40/1104842052-2f6d8b4a/ 170 000 ₽
2201 https://auto.ru/cars/used/sale/volvo/s80/1103121029-bb648e98/ 880 000 ₽
2202 https://auto.ru/cars/used/sale/volvo/s40/1105479562-235f586e/ 639 000 ₽
2203 https://auto.ru/cars/used/sale/volvo/s80/1105349315-2a081e05/ 380 000 ₽
2204 https

2299 https://auto.ru/cars/used/sale/volvo/s60/1105233749-48210496/ 970 000 ₽
2300 links parsed
adding to CONTENT/VOLVO_content.csv
2300 https://auto.ru/cars/used/sale/volvo/s60/1105548284-51ebac1c/ 819 000 ₽
2301 https://auto.ru/cars/used/sale/volvo/s60/1105475210-5e04d284/ 300 000 ₽
2302 https://auto.ru/cars/used/sale/volvo/s60/1105177957-8f3c6d3d/ 600 000 ₽
2303 https://auto.ru/cars/used/sale/volvo/740/1105441359-3787ba45/ 95 000 ₽
2304 https://auto.ru/cars/used/sale/volvo/s80/1105438870-dd84df34/ 650 000 ₽
2305 https://auto.ru/cars/used/sale/volvo/xc90/1104039728-12bba561/ 975 000 ₽
2306 https://auto.ru/cars/used/sale/volvo/s80/1105532361-907c9ef4/ 810 000 ₽
2307 https://auto.ru/cars/used/sale/volvo/c30/1102665686-0f12b572/ 400 000 ₽
2308 https://auto.ru/cars/used/sale/volvo/s80/1105536154-452d95ed/ 390 000 ₽
exception: 
https://auto.ru/cars/used/sale/volvo/s40/1105574081-4df26eed/
2310 https://auto.ru/cars/used/sale/volvo/s60/1105425999-147fd65a/ 1 390 000 ₽
2311 https://auto.ru/ca

2406 https://auto.ru/cars/used/sale/volvo/240_series/1105524363-18a62863/ 169 000 ₽
2407 https://auto.ru/cars/used/sale/volvo/850/1104870942-d1d2dec6/ 125 000 ₽
2408 https://auto.ru/cars/used/sale/volvo/s60/1105563652-6f152189/ 995 000 ₽
2409 https://auto.ru/cars/used/sale/volvo/xc70/1105603866-317c9e2c/ 2 300 000 ₽
2410 https://auto.ru/cars/used/sale/volvo/xc90/1101808010-1de3becd/ 970 000 ₽
2411 https://auto.ru/cars/used/sale/volvo/s40/1105632089-306658c4/ 620 000 ₽
2412 https://auto.ru/cars/used/sale/volvo/xc90/1105605340-4818ab3a/ 1 100 000 ₽
2413 https://auto.ru/cars/used/sale/volvo/xc90/1104221877-2a7abe66/ 450 000 ₽
2414 https://auto.ru/cars/used/sale/volvo/s40/1105613621-a15f9234/ 650 000 ₽
2415 https://auto.ru/cars/used/sale/volvo/xc90/1098129756-83923a53/ 1 100 000 ₽
2416 https://auto.ru/cars/used/sale/volvo/s60/1103966364-219cc12b/ 430 000 ₽
2417 https://auto.ru/cars/used/sale/volvo/940/1105264112-b5003fc0/ 180 000 ₽
2418 https://auto.ru/cars/used/sale/volvo/s80/1105041697-0

2514 https://auto.ru/cars/used/sale/volvo/300_series/1103685912-78a21ba7/ 100 000 ₽
2515 https://auto.ru/cars/used/sale/volvo/s80/1103823221-2498291c/ 450 000 ₽
2516 https://auto.ru/cars/used/sale/volvo/s80/1097396242-643eed66/ 695 000 ₽
2517 https://auto.ru/cars/used/sale/volvo/960/1093644614-db1999a6/ 130 000 ₽
2518 https://auto.ru/cars/used/sale/volvo/740/1086971901-2e93f40f/ 80 000 ₽
2519 https://auto.ru/cars/used/sale/volvo/s40/1094125918-36211fb1/ 360 000 ₽
2520 https://auto.ru/cars/used/sale/volvo/xc90/1105535986-35874ffc/ 915 000 ₽
2521 https://auto.ru/cars/used/sale/volvo/s40/1104949469-e0b7fd2b/ 294 000 ₽
2522 https://auto.ru/cars/used/sale/volvo/s80/1105079212-13447fe3/ 220 000 ₽
2523 https://auto.ru/cars/used/sale/volvo/s40/1105319418-450dbab7/ 125 990 ₽
2524 https://auto.ru/cars/used/sale/volvo/s40/1092565502-667e3c53/ 250 000 ₽
2524 links parsed
adding to CONTENT/VOLVO_content.csv


In [16]:
get_content("BMW")

BMW: 3403 urls to parse
starting from the beginning...
creating CONTENT/BMW_content.csv
1 https://auto.ru/cars/used/sale/bmw/x5/1104981355-2ac4b782/ 2 360 000 ₽
2 https://auto.ru/cars/used/sale/bmw/x5/1105649463-b8a4be2e/ 5 480 000 ₽
3 https://auto.ru/cars/used/sale/bmw/5er/1105332430-8ad9002f/ 1 650 000 ₽
4 https://auto.ru/cars/used/sale/bmw/x6_m/1105351312-cbcf241c/ 1 500 000 ₽
6 https://auto.ru/cars/used/sale/bmw/x5/1105663529-b0e3d539/ 879 000 ₽
7 https://auto.ru/cars/used/sale/bmw/x6_m/1047361698-5f3f/ 2 600 000 ₽
8 https://auto.ru/cars/used/sale/bmw/3er/1105304030-511ecee5/ 1 650 000 ₽
9 https://auto.ru/cars/used/sale/bmw/x3/1105556876-8cfbc45a/ 3 670 000 ₽
10 https://auto.ru/cars/used/sale/bmw/x5/1105664975-d845ab9e/ 785 000 ₽
11 https://auto.ru/cars/used/sale/bmw/6er/1105590056-10126aa7/ 1 150 000 ₽
12 https://auto.ru/cars/used/sale/bmw/3er/1105443445-a3a2c792/ 350 000 ₽
13 https://auto.ru/cars/used/sale/bmw/x3/1105146616-2910ea9e/ 3 229 000 ₽
14 https://auto.ru/cars/used/sale/

121 https://auto.ru/cars/used/sale/bmw/3er/1105066516-ca4beec7/ 2 825 000 ₽
122 https://auto.ru/cars/used/sale/bmw/5er/1105664968-2ddfd0ba/ 3 825 000 ₽
123 https://auto.ru/cars/used/sale/bmw/5er/1105586224-71039d90/ 1 450 000 ₽
124 https://auto.ru/cars/used/sale/bmw/2er/1105468246-338941a7/ 1 797 000 ₽
125 https://auto.ru/cars/used/sale/bmw/5er/1105253749-c046bf2d/ 2 995 000 ₽
126 https://auto.ru/cars/used/sale/bmw/3er/1104577106-b9e6d39e/ 3 594 000 ₽
127 https://auto.ru/cars/used/sale/bmw/5er/1105372077-23a14571/ 4 778 000 ₽
128 https://auto.ru/cars/used/sale/bmw/x6/1105412257-571a653c/ 8 495 000 ₽
129 https://auto.ru/cars/used/sale/bmw/x4/1105408528-0de0560d/ 5 075 000 ₽
130 https://auto.ru/cars/used/sale/bmw/x5/1105665953-9eb5254a/ 5 247 070 ₽
131 https://auto.ru/cars/used/sale/bmw/x5/1105025102-f0f395da/ 3 110 299 ₽
132 https://auto.ru/cars/used/sale/bmw/x5/1105664204-ef052384/ 2 187 000 ₽
133 https://auto.ru/cars/used/sale/bmw/3er/1105089563-d7f37019/ 1 899 000 ₽
134 https://auto.

257 https://auto.ru/cars/used/sale/bmw/x5/1103833063-64ae88ca/ 1 425 000 ₽
258 https://auto.ru/cars/used/sale/bmw/3er/1104954625-31fdd591/ 1 755 000 ₽
259 https://auto.ru/cars/used/sale/bmw/x4/1105239100-7fe2d224/ 2 026 000 ₽
260 https://auto.ru/cars/used/sale/bmw/x1/1103928936-e47eaf41/ 895 000 ₽
261 https://auto.ru/cars/used/sale/bmw/3er/1105653176-be25d361/ 3 555 000 ₽
262 https://auto.ru/cars/used/sale/bmw/x5/1105169066-16ba54c7/ 2 040 000 ₽
263 https://auto.ru/cars/used/sale/bmw/7er/1105651446-fb4c67bc/ 8 698 000 ₽
264 https://auto.ru/cars/used/sale/bmw/5er/1104859821-147fe2e7/ 1 749 000 ₽
265 https://auto.ru/cars/used/sale/bmw/3er/1105505401-8f718630/ 1 100 000 ₽
266 https://auto.ru/cars/used/sale/bmw/x3/1105653319-51c70f01/ 4 190 000 ₽
267 https://auto.ru/cars/used/sale/bmw/3er/1105304768-2add8b18/ 1 429 817 ₽
268 https://auto.ru/cars/used/sale/bmw/x5/1105664206-5fa505b2/ 3 048 000 ₽
271 https://auto.ru/cars/used/sale/bmw/x4/1105312364-3074f6a0/ 3 935 000 ₽
272 https://auto.ru/c

379 https://auto.ru/cars/used/sale/bmw/x6/1104792488-ec3707a6/ 4 450 000 ₽
380 https://auto.ru/cars/used/sale/bmw/x5/1105652943-8f083ed2/ 2 767 000 ₽
381 https://auto.ru/cars/used/sale/bmw/x6/1105654052-01f56cd5/ 1 648 000 ₽
382 https://auto.ru/cars/used/sale/bmw/4/1105653521-eb29bbbc/ 2 249 000 ₽
383 https://auto.ru/cars/used/sale/bmw/x5/1105652959-ced3a5d3/ 5 973 000 ₽
385 https://auto.ru/cars/used/sale/bmw/x5/1105654063-9e17fb37/ 1 754 000 ₽
387 https://auto.ru/cars/used/sale/bmw/x5/1105652696-dfbf5974/ 1 745 000 ₽
388 https://auto.ru/cars/used/sale/bmw/5er/1105615037-7f91c5b0/ 1 830 000 ₽
389 https://auto.ru/cars/used/sale/bmw/5er/1105652029-803c48eb/ 1 399 000 ₽
390 https://auto.ru/cars/used/sale/bmw/i3/1105653728-be6cbe8c/ 2 150 000 ₽
391 https://auto.ru/cars/used/sale/bmw/3er/1105651547-e5eb5dc3/ 1 315 000 ₽
393 https://auto.ru/cars/used/sale/bmw/3er/1105382308-90babf4c/ 817 000 ₽
394 https://auto.ru/cars/used/sale/bmw/x3/1105527674-a390759f/ 1 434 000 ₽
395 https://auto.ru/cars

503 https://auto.ru/cars/used/sale/bmw/x3/1105646647-b2e8e367/ 1 688 000 ₽
504 https://auto.ru/cars/used/sale/bmw/5er/1105646575-bc11171a/ 4 247 000 ₽
505 https://auto.ru/cars/used/sale/bmw/3er/1105645890-ffba9890/ 2 063 000 ₽
506 https://auto.ru/cars/used/sale/bmw/7er/1105647110-d0632768/ 7 619 000 ₽
507 https://auto.ru/cars/used/sale/bmw/5er/1105646777-7e8d1bfd/ 1 240 000 ₽
508 https://auto.ru/cars/used/sale/bmw/5er/1105645955-192b8f88/ 2 469 000 ₽
509 https://auto.ru/cars/used/sale/bmw/5er/1105665286-7234bf6b/ 220 000 ₽
510 https://auto.ru/cars/used/sale/bmw/4/1105645839-7807d9a8/ 1 699 900 ₽
511 https://auto.ru/cars/used/sale/bmw/3er/1105429983-3eff803e/ 3 820 000 ₽
513 https://auto.ru/cars/used/sale/bmw/3er/1105327814-d9103248/ 160 000 ₽
514 https://auto.ru/cars/used/sale/bmw/3er/1105665218-4abc8d0f/ 460 000 ₽
515 https://auto.ru/cars/used/sale/bmw/x3/1105646242-9f60da3a/ 1 550 000 ₽
516 https://auto.ru/cars/used/sale/bmw/x5/1105645537-417aab55/ 2 849 000 ₽
517 https://auto.ru/car

...

2552 https://auto.ru/cars/used/sale/bmw/x5/1105483066-75d405c6/ 3 170 000 ₽
2553 https://auto.ru/cars/used/sale/bmw/x5/1104858218-4ee18a9f/ 5 915 000 ₽
2554 https://auto.ru/cars/used/sale/bmw/3er/1105482789-1f96b8ee/ 1 265 000 ₽
2555 https://auto.ru/cars/used/sale/bmw/3er/1105652620-c97e6cf2/ 820 000 ₽
2556 https://auto.ru/cars/used/sale/bmw/x5/1105484448-3a3e802c/ 2 810 000 ₽
2557 https://auto.ru/cars/used/sale/bmw/1er/1105483649-f71680ac/ 1 549 000 ₽
2558 https://auto.ru/cars/used/sale/bmw/i3/1105302730-a153f8e0/ 1 400 000 ₽
2559 https://auto.ru/cars/used/sale/bmw/1er/1105484731-77ed31ef/ 723 000 ₽
2560 https://auto.ru/cars/used/sale/bmw/5er/1105652558-123b3248/ 690 000 ₽
2562 https://auto.ru/cars/used/sale/bmw/3er/1105481606-55a90eb6/ 568 000 ₽
2563 https://auto.ru/cars/used/sale/bmw/x5/1105652564-a56547ba/ 650 000 ₽
2564 https://auto.ru/cars/used/sale/bmw/7er/1105483605-cad8b1f9/ 4 250 000 ₽
2565 https://auto.ru/cars/used/sale/bmw/x4/1104131828-ac80a432/ 4 355 000 ₽
2566 https://au

2665 https://auto.ru/cars/used/sale/bmw/1er/1105444346-6f3fa857/ 400 000 ₽
2666 https://auto.ru/cars/used/sale/bmw/3er/1105651162-037bac0c/ 2 500 000 ₽
2667 https://auto.ru/cars/used/sale/bmw/3er/1105466143-8c403a67/ 1 719 000 ₽
2668 https://auto.ru/cars/used/sale/bmw/x5/1105470989-0253a6ad/ 3 200 000 ₽
2669 https://auto.ru/cars/used/sale/bmw/x6/1105466126-f860907d/ 4 850 000 ₽
2670 https://auto.ru/cars/used/sale/bmw/x5/1105462102-432672ce/ 2 850 000 ₽
2671 https://auto.ru/cars/used/sale/bmw/x6/1105463699-28d33f48/ 8 000 000 ₽
2672 https://auto.ru/cars/used/sale/bmw/x6/1105462369-08ae71c8/ 2 300 000 ₽
2673 https://auto.ru/cars/used/sale/bmw/x5/1105462482-3c82f296/ 933 000 ₽
2674 https://auto.ru/cars/used/sale/bmw/5er/1104777791-b21630d3/ 60 000 ₽
2675 https://auto.ru/cars/used/sale/bmw/x1/1105466070-fded4010/ 995 000 ₽
2676 https://auto.ru/cars/used/sale/bmw/1er/1105650960-4e48173a/ 860 000 ₽
2677 https://auto.ru/cars/used/sale/bmw/i3/1105170255-19868b3e/ 2 599 000 ₽
2678 https://auto.

2779 https://auto.ru/cars/used/sale/bmw/x5_m/1105422956-f4df26bd/ 4 650 000 ₽
2780 https://auto.ru/cars/used/sale/bmw/x3/1105448260-08c8b2b7/ 1 240 000 ₽
2781 https://auto.ru/cars/used/sale/bmw/x5/1105566646-9b18e655/ 670 000 ₽
2782 https://auto.ru/cars/used/sale/bmw/3er/1105649806-e491d618/ 370 000 ₽
2783 https://auto.ru/cars/used/sale/bmw/3er/1105540519-8c5e578d/ 470 000 ₽
2784 https://auto.ru/cars/used/sale/bmw/x5/1105649791-524c4714/ 3 199 999 ₽
2785 https://auto.ru/cars/used/sale/bmw/x6_m/1105168751-59626b5e/ 5 399 999 ₽
2786 https://auto.ru/cars/used/sale/bmw/x1/1105449060-0f849786/ 1 700 000 ₽
2787 https://auto.ru/cars/used/sale/bmw/1er/1105448732-62e7fb1a/ 1 799 847 ₽
2788 https://auto.ru/cars/used/sale/bmw/5er/1105649787-fce58e67/ 3 100 000 ₽
2789 https://auto.ru/cars/used/sale/bmw/7er/1105448763-7bf36f96/ 1 500 000 ₽
2791 https://auto.ru/cars/used/sale/bmw/1er/1105649722-c6437c50/ 1 600 000 ₽
2792 https://auto.ru/cars/used/sale/bmw/x3/1105649642-2c23c020/ 1 050 000 ₽
2793 htt

2899 https://auto.ru/cars/used/sale/bmw/3er/1105435740-71b13f94/ 1 670 000 ₽
2900 links parsed
adding to CONTENT/BMW_content.csv
2900 https://auto.ru/cars/used/sale/bmw/x3/1105433797-66be98ed/ 1 495 000 ₽
2901 https://auto.ru/cars/used/sale/bmw/x5/1105364222-fbc6275f/ 1 049 000 ₽
2902 https://auto.ru/cars/used/sale/bmw/6er/1105435564-03f6f069/ 2 697 000 ₽
2903 https://auto.ru/cars/used/sale/bmw/x1/1105243755-e67e8d65/ 1 150 000 ₽
2904 https://auto.ru/cars/used/sale/bmw/x7/1104854669-5c3a18cc/ 7 097 000 ₽
2905 https://auto.ru/cars/used/sale/bmw/1er/1105479113-dfd7b3d6/ 545 000 ₽
2906 https://auto.ru/cars/used/sale/bmw/5er/1105647933-a4c79100/ 2 750 000 ₽
2907 https://auto.ru/cars/used/sale/bmw/x1/1105418011-ae8e4ad0/ 1 225 000 ₽
2909 https://auto.ru/cars/used/sale/bmw/i3/1105571549-316d8b55/ 2 200 000 ₽
2910 https://auto.ru/cars/used/sale/bmw/x5/1105334317-ef04fb19/ 2 459 000 ₽
2911 https://auto.ru/cars/used/sale/bmw/x5/1105177028-5016a9a2/ 2 899 000 ₽
2912 https://auto.ru/cars/used/sal

KeyboardInterrupt: 

In [17]:
get_content("BMW", 2900)


BMW: 3403 urls to parse
starting from ...2900
creating CONTENT/BMW_2900_content.csv
2901 https://auto.ru/cars/used/sale/bmw/x5/1105364222-fbc6275f/ 1 049 000 ₽
2902 https://auto.ru/cars/used/sale/bmw/6er/1105435564-03f6f069/ 2 697 000 ₽
2903 https://auto.ru/cars/used/sale/bmw/x1/1105243755-e67e8d65/ 1 150 000 ₽
2904 https://auto.ru/cars/used/sale/bmw/x7/1104854669-5c3a18cc/ 7 097 000 ₽
2905 https://auto.ru/cars/used/sale/bmw/1er/1105479113-dfd7b3d6/ 545 000 ₽
2906 https://auto.ru/cars/used/sale/bmw/5er/1105647933-a4c79100/ 2 750 000 ₽
2907 https://auto.ru/cars/used/sale/bmw/x1/1105418011-ae8e4ad0/ 1 225 000 ₽
2909 https://auto.ru/cars/used/sale/bmw/i3/1105571549-316d8b55/ 2 200 000 ₽
2910 https://auto.ru/cars/used/sale/bmw/x5/1105334317-ef04fb19/ 2 459 000 ₽
2911 https://auto.ru/cars/used/sale/bmw/x5/1105177028-5016a9a2/ 2 899 000 ₽
2912 https://auto.ru/cars/used/sale/bmw/x4/1104756574-e171eaea/ 2 950 000 ₽
2913 https://auto.ru/cars/used/sale/bmw/x4/1105014678-53023c0d/ 2 010 762 ₽
291

3013 https://auto.ru/cars/used/sale/bmw/x5/1105257331-52cd3c80/ 3 748 000 ₽
3014 https://auto.ru/cars/used/sale/bmw/x4/1105036288-7da0a9ea/ 4 595 000 ₽
3015 https://auto.ru/cars/used/sale/bmw/7er/1105416435-eb375d05/ 7 387 000 ₽
3016 https://auto.ru/cars/used/sale/bmw/3er/1105416140-b88dc799/ 3 149 000 ₽
3017 https://auto.ru/cars/used/sale/bmw/3er/1105646589-4480fa13/ 1 000 050 ₽
3018 https://auto.ru/cars/used/sale/bmw/x5/1105416676-9a19c2e9/ 470 000 ₽
3019 https://auto.ru/cars/used/sale/bmw/x3/1105534494-bd6f5344/ 4 400 000 ₽
3020 https://auto.ru/cars/used/sale/bmw/5er/1104918586-eee58015/ 3 395 000 ₽
3021 https://auto.ru/cars/used/sale/bmw/5er/1105523688-7f0c4a0a/ 3 125 000 ₽
3022 https://auto.ru/cars/used/sale/bmw/x4/1105419964-c3f8ebe4/ 4 666 000 ₽
3023 https://auto.ru/cars/used/sale/bmw/5er/1104972882-849abea9/ 1 600 000 ₽
3024 https://auto.ru/cars/used/sale/bmw/x5/1105416054-c61f1aa5/ 555 000 ₽
3026 https://auto.ru/cars/used/sale/bmw/x6/1105372630-08f071cb/ 2 199 000 ₽
3027 https

3126 https://auto.ru/cars/used/sale/bmw/x4/1105393570-9a4bc278/ 2 167 000 ₽
3127 https://auto.ru/cars/used/sale/bmw/7er/1105511987-1e4bfb82/ 4 220 000 ₽
3128 https://auto.ru/cars/used/sale/bmw/x5/1105027265-5b914598/ 1 643 000 ₽
3129 https://auto.ru/cars/used/sale/bmw/2er/1105393569-ca0605a2/ 2 375 000 ₽
3130 https://auto.ru/cars/used/sale/bmw/4/1102683547-222e499e/ 1 715 000 ₽
3131 https://auto.ru/cars/used/sale/bmw/x6/1105426019-ed6a2b59/ 1 370 000 ₽
3132 https://auto.ru/cars/used/sale/bmw/3er/1105645509-90a296e7/ 1 600 000 ₽
3133 https://auto.ru/cars/used/sale/bmw/x3/1102571648-2a305e71/ 3 140 000 ₽
3134 https://auto.ru/cars/used/sale/bmw/3er/1105392142-7a84a76c/ 3 449 000 ₽
3135 https://auto.ru/cars/used/sale/bmw/5er/1105391192-d97ca7aa/ 1 489 000 ₽
3136 https://auto.ru/cars/used/sale/bmw/i3/1105392966-6acd85ed/ 2 969 000 ₽
3137 https://auto.ru/cars/used/sale/bmw/x3/1105391859-82fba64f/ 4 170 000 ₽
3138 https://auto.ru/cars/used/sale/bmw/x3/1105392776-2261fdc7/ 3 099 000 ₽
3139 htt

3235 https://auto.ru/cars/used/sale/bmw/4/1105363175-746ee56f/ 2 010 000 ₽
3236 https://auto.ru/cars/used/sale/bmw/x3/1105360985-fead01c2/ 1 379 000 ₽
3237 https://auto.ru/cars/used/sale/bmw/x3/1105592992-1bcf5919/ 580 000 ₽
3238 https://auto.ru/cars/used/sale/bmw/3er/1105360987-62bd7c33/ 1 716 000 ₽
3239 https://auto.ru/cars/used/sale/bmw/x6_m/1105365587-40485f7e/ 3 888 000 ₽
3240 https://auto.ru/cars/used/sale/bmw/x3/1103618263-86677cb3/ 3 499 000 ₽
3241 https://auto.ru/cars/used/sale/bmw/x3/1102635091-d35fc511/ 2 170 000 ₽
3242 https://auto.ru/cars/used/sale/bmw/x3/1105645401-5f40c323/ 2 030 000 ₽
3243 https://auto.ru/cars/used/sale/bmw/6er/1105329111-4da10a83/ 1 090 000 ₽
3244 https://auto.ru/cars/used/sale/bmw/x1/1105360986-721621d5/ 1 248 000 ₽
3245 https://auto.ru/cars/used/sale/bmw/x5/1105449077-ba5afd85/ 1 172 416 ₽
3246 https://auto.ru/cars/used/sale/bmw/x3/1105428417-42423167/ 4 100 000 ₽
3248 https://auto.ru/cars/used/sale/bmw/x3/1105240625-078399e2/ 4 125 000 ₽
3249 https:

3354 https://auto.ru/cars/used/sale/bmw/x5/1103017072-1e95a4d9/ 940 000 ₽
3355 https://auto.ru/cars/used/sale/bmw/5er/1105644717-02c894d6/ 370 000 ₽
3356 https://auto.ru/cars/used/sale/bmw/x3/1105340001-68846daf/ 895 000 ₽
3357 https://auto.ru/cars/used/sale/bmw/x5/1105567114-d54b34ee/ 1 180 000 ₽
3358 https://auto.ru/cars/used/sale/bmw/x5/1105345399-0477e368/ 859 000 ₽
3359 https://auto.ru/cars/used/sale/bmw/x5/1105341566-94e8fccd/ 3 589 000 ₽
3360 https://auto.ru/cars/used/sale/bmw/m2/1105335453-3abd7e51/ 3 600 000 ₽
3361 https://auto.ru/cars/used/sale/bmw/x3/1101989269-6c2c57b0/ 4 050 000 ₽
3362 https://auto.ru/cars/used/sale/bmw/5er/1105641245-a9857986/ 5 550 000 ₽
3363 https://auto.ru/cars/used/sale/bmw/5er/1105104764-681fc8a4/ 1 719 000 ₽
3364 https://auto.ru/cars/used/sale/bmw/x5/1105341131-1746e3cb/ 3 229 000 ₽
3365 https://auto.ru/cars/used/sale/bmw/5er/1105483868-f9e2ce9f/ 1 490 000 ₽
3366 https://auto.ru/cars/used/sale/bmw/7er/1091143224-aeb801c9/ 320 000 ₽
3367 https://auto

In [18]:
get_content("Nissan")

NISSAN: 3459 urls to parse
starting from the beginning...
creating CONTENT/NISSAN_content.csv
2 https://auto.ru/cars/used/sale/nissan/teana/1105642698-cd517e3f/ 765 000 ₽
3 https://auto.ru/cars/used/sale/nissan/x_trail/1105517820-3ca2a36a/ 955 000 ₽
4 https://auto.ru/cars/used/sale/nissan/murano/1105246757-46322321/ 465 000 ₽
6 https://auto.ru/cars/used/sale/nissan/x_trail/1105554223-6801f223/ 740 000 ₽
7 https://auto.ru/cars/used/sale/nissan/x_trail/1105662196-2ef4c296/ 469 000 ₽
8 https://auto.ru/cars/used/sale/nissan/np300/1104839069-5e1fa940/ 707 000 ₽
9 https://auto.ru/cars/used/sale/nissan/x_trail/1105666193-de2287b0/ 975 000 ₽
10 https://auto.ru/cars/used/sale/nissan/teana/1105659983-881dbc38/ 749 990 ₽
12 https://auto.ru/cars/used/sale/nissan/patrol/1105179346-fd1688cb/ 2 749 000 ₽
13 https://auto.ru/cars/used/sale/nissan/teana/1105465951-9b8c1d82/ 995 000 ₽
14 https://auto.ru/cars/used/sale/nissan/qashqai/1105663079-6df60886/ 534 000 ₽
15 https://auto.ru/cars/used/sale/nissan/

117 https://auto.ru/cars/used/sale/nissan/note/1105662445-4a34bd04/ 680 000 ₽
118 https://auto.ru/cars/used/sale/nissan/qashqai/1105628135-89137472/ 1 125 000 ₽
120 https://auto.ru/cars/used/sale/nissan/qashqai/1105646222-495706dc/ 919 000 ₽
122 https://auto.ru/cars/used/sale/nissan/juke/1105104504-273c2fbf/ 799 000 ₽
123 https://auto.ru/cars/used/sale/nissan/almera/1105620618-45ab4953/ 730 000 ₽
124 https://auto.ru/cars/used/sale/nissan/almera/1105534408-5b78a66d/ 566 400 ₽
125 https://auto.ru/cars/used/sale/nissan/primera/1105355215-e4f44c57/ 230 000 ₽
126 https://auto.ru/cars/used/sale/nissan/sentra/1105027396-8084dc53/ 769 000 ₽
127 https://auto.ru/cars/used/sale/nissan/micra/1105485720-d7b560aa/ 419 000 ₽
128 https://auto.ru/cars/used/sale/nissan/tiida/1104006310-1fba162e/ 510 000 ₽
129 https://auto.ru/cars/used/sale/nissan/qashqai/1105524390-176001ac/ 1 090 000 ₽
130 https://auto.ru/cars/used/sale/nissan/terrano/1105560152-d9fb80fe/ 1 397 000 ₽
131 https://auto.ru/cars/used/sale/

234 https://auto.ru/cars/used/sale/nissan/note/1105655342-bb629c81/ 599 000 ₽
235 https://auto.ru/cars/used/sale/nissan/note/1105654709-1c48e328/ 798 000 ₽
236 https://auto.ru/cars/used/sale/nissan/terrano/1105656220-fa69e47e/ 1 326 000 ₽
237 https://auto.ru/cars/used/sale/nissan/serena/1105655710-c55b7776/ 1 178 000 ₽
238 https://auto.ru/cars/used/sale/nissan/liberty/1105654615-d73dc358/ 350 000 ₽
239 https://auto.ru/cars/used/sale/nissan/juke/1105666051-e6ab78e8/ 690 000 ₽
240 https://auto.ru/cars/used/sale/nissan/x_trail/1105654415-6e3b332d/ 2 296 000 ₽
241 https://auto.ru/cars/used/sale/nissan/x_trail/1105532045-19b53908/ 1 470 000 ₽
242 https://auto.ru/cars/used/sale/nissan/teana/1105666050-8a314ceb/ 450 000 ₽
243 https://auto.ru/cars/used/sale/nissan/almera/1105666040-75c857bc/ 540 000 ₽
244 https://auto.ru/cars/used/sale/nissan/note/1105654696-2de5bfe9/ 790 000 ₽
245 https://auto.ru/cars/used/sale/nissan/serena/1105654690-eb09fe82/ 1 400 000 ₽
246 https://auto.ru/cars/used/sale/

342 https://auto.ru/cars/used/sale/nissan/tiida/1105647447-c76b0605/ 610 000 ₽
344 https://auto.ru/cars/used/sale/nissan/juke/1105649236-08342cb3/ 1 075 000 ₽
345 https://auto.ru/cars/used/sale/nissan/qashqai/1105560375-07be16ac/ 1 240 000 ₽
346 https://auto.ru/cars/used/sale/nissan/note/1105490439-ccb95f29/ 390 000 ₽
348 https://auto.ru/cars/used/sale/nissan/x_trail/1105548991-9cff38f2/ 709 000 ₽
349 https://auto.ru/cars/used/sale/nissan/terrano/1105650781-a01e39ca/ 1 400 000 ₽
350 https://auto.ru/cars/used/sale/nissan/qashqai/1105116940-377ce794/ 1 275 000 ₽
352 https://auto.ru/cars/used/sale/nissan/x_trail/1105589605-e6823f58/ 1 570 000 ₽
353 https://auto.ru/cars/used/sale/nissan/qashqai/1105455792-bc878ca2/ 1 900 000 ₽
354 https://auto.ru/cars/used/sale/nissan/teana/1105429727-1a1d6bb5/ 445 000 ₽
355 https://auto.ru/cars/used/sale/nissan/x_trail/1105646974-47824e40/ 1 227 000 ₽
356 https://auto.ru/cars/used/sale/nissan/note/1105185943-73eb9605/ 425 000 ₽
357 https://auto.ru/cars/us

458 https://auto.ru/cars/used/sale/nissan/juke/1105645098-44b4950e/ 910 000 ₽
460 https://auto.ru/cars/used/sale/nissan/almera/1105664936-f48c9217/ 274 000 ₽
461 https://auto.ru/cars/used/sale/nissan/x_trail/1105644885-1836fdd0/ 1 695 000 ₽
462 https://auto.ru/cars/used/sale/nissan/almera/1105644735-ebac18f7/ 707 000 ₽
463 https://auto.ru/cars/used/sale/nissan/qashqai/1105402732-257f9ed8/ 1 270 000 ₽
464 https://auto.ru/cars/used/sale/nissan/qashqai_plus_2/1105098530-0ca7e87b/ 732 000 ₽
465 https://auto.ru/cars/used/sale/nissan/teana/1105372183-dbc1f0e5/ 718 000 ₽
466 https://auto.ru/cars/used/sale/nissan/x_trail/1105421993-f6b3c59c/ 1 730 000 ₽
467 https://auto.ru/cars/used/sale/nissan/qashqai/1105454604-8b8ae9e7/ 1 518 000 ₽
468 https://auto.ru/cars/used/sale/nissan/juke/1105219266-d0f63557/ 837 000 ₽
469 https://auto.ru/cars/used/sale/nissan/almera_classic/1105369815-6fa7b2c5/ 329 000 ₽
470 https://auto.ru/cars/used/sale/nissan/x_trail/1105344861-9ef6ea3d/ 771 000 ₽
471 https://auto

...

2941 https://auto.ru/cars/used/sale/nissan/x_trail/1104786379-f80f37a0/ 1 824 000 ₽
2942 https://auto.ru/cars/used/sale/nissan/qashqai/1105175653-9431b814/ 1 145 000 ₽
2943 https://auto.ru/cars/used/sale/nissan/leaf/1105647923-9166fdb9/ 690 000 ₽
2944 https://auto.ru/cars/used/sale/nissan/x_trail/1104967153-ab886498/ 1 199 000 ₽
2945 https://auto.ru/cars/used/sale/nissan/murano/1105303570-5a84248a/ 2 200 000 ₽
2946 https://auto.ru/cars/used/sale/nissan/almera/1103220442-905e0289/ 529 000 ₽
2947 https://auto.ru/cars/used/sale/nissan/x_trail/1105435249-f77d0b61/ 1 198 000 ₽
2948 https://auto.ru/cars/used/sale/nissan/qashqai/1105647881-f925fd09/ 1 390 000 ₽
2949 https://auto.ru/cars/used/sale/nissan/x_trail/1105271785-2859f0f4/ 1 097 500 ₽
2950 https://auto.ru/cars/used/sale/nissan/x_trail/1105079238-f136deab/ 1 887 000 ₽
2951 https://auto.ru/cars/used/sale/nissan/qashqai/1105647764-67e89bc9/ 660 000 ₽
2952 https://auto.ru/cars/used/sale/nissan/qashqai/1104062951-6366db5f/ 839 000 ₽
2953 

3041 https://auto.ru/cars/used/sale/nissan/primera/1105647236-b5271266/ 220 000 ₽
3042 https://auto.ru/cars/used/sale/nissan/qashqai/1105423099-76d2a074/ 1 314 000 ₽
3043 https://auto.ru/cars/used/sale/nissan/almera_classic/1105647068-b27187f0/ 315 000 ₽
3044 https://auto.ru/cars/used/sale/nissan/x_trail/1105422050-3c82a258/ 1 056 049 ₽
3045 https://auto.ru/cars/used/sale/nissan/note/1105419130-4b7fb879/ 799 000 ₽
3046 https://auto.ru/cars/used/sale/nissan/qashqai/1105422897-4a206ede/ 789 000 ₽
3047 https://auto.ru/cars/used/sale/nissan/tiida/1101333391-6689c525/ 435 000 ₽
3048 https://auto.ru/cars/used/sale/nissan/x_trail/1105646922-c6a55ccf/ 1 400 000 ₽
3049 https://auto.ru/cars/used/sale/nissan/x_trail/1105410260-50577e3d/ 1 661 000 ₽
3050 https://auto.ru/cars/used/sale/nissan/terrano/1105422577-b969f8cc/ 1 489 000 ₽
3051 https://auto.ru/cars/used/sale/nissan/qashqai/1105456459-51400764/ 765 000 ₽
3052 https://auto.ru/cars/used/sale/nissan/juke/1105420901-b779963a/ 848 000 ₽
3054 ht

3151 https://auto.ru/cars/used/sale/nissan/terrano/1105150203-49aacf05/ 1 457 000 ₽
3152 https://auto.ru/cars/used/sale/nissan/ad/1105467429-dbd74d70/ 115 000 ₽
3153 https://auto.ru/cars/used/sale/nissan/qashqai/1105408455-7631ab34/ 675 000 ₽
3154 https://auto.ru/cars/used/sale/nissan/qashqai/1105646075-2e60cc17/ 970 000 ₽
3155 https://auto.ru/cars/used/sale/nissan/maxima/1105409647-7a5ba7cb/ 190 000 ₽
3156 https://auto.ru/cars/used/sale/nissan/qashqai/1105406920-1bc9ccdd/ 1 875 000 ₽
3157 https://auto.ru/cars/used/sale/nissan/almera/1105645983-15b20a38/ 750 000 ₽
3158 https://auto.ru/cars/used/sale/nissan/juke/1105128555-681a61db/ 930 000 ₽
3159 https://auto.ru/cars/used/sale/nissan/qashqai/1105406477-9cafe1df/ 980 000 ₽
3160 https://auto.ru/cars/used/sale/nissan/teana/1105409454-6b9c8369/ 1 129 000 ₽
3161 https://auto.ru/cars/used/sale/nissan/almera_classic/1105408622-496d78ab/ 249 000 ₽
3162 https://auto.ru/cars/used/sale/nissan/terrano/1105409723-14313059/ 1 279 000 ₽
3163 https://

3256 https://auto.ru/cars/used/sale/nissan/cefiro/1105412637-bc9457af/ 240 000 ₽
3257 https://auto.ru/cars/used/sale/nissan/qashqai/1105122890-646a90b7/ 1 209 000 ₽
3258 https://auto.ru/cars/used/sale/nissan/tiida/1105391644-00ff1319/ 529 000 ₽
3259 https://auto.ru/cars/used/sale/nissan/almera/1105389274-0f860efc/ 700 000 ₽
3260 https://auto.ru/cars/used/sale/nissan/terrano/1103614239-f54a04c3/ 599 000 ₽
3262 https://auto.ru/cars/used/sale/nissan/tiida/1105211876-440850b6/ 760 000 ₽
3263 https://auto.ru/cars/used/sale/nissan/terrano/1105160175-eb934070/ 599 000 ₽
3264 https://auto.ru/cars/used/sale/nissan/juke/1105122666-847907d3/ 661 000 ₽
3265 https://auto.ru/cars/used/sale/nissan/qashqai/1105107530-d1538c63/ 1 369 000 ₽
3266 https://auto.ru/cars/used/sale/nissan/qashqai/1104843948-75b22394/ 1 250 410 ₽
3267 https://auto.ru/cars/used/sale/nissan/x_trail/1105645499-a6e246fb/ 450 000 ₽
3268 https://auto.ru/cars/used/sale/nissan/x_trail/1105393216-655075f4/ 1 190 000 ₽
3269 https://auto

3367 https://auto.ru/cars/used/sale/nissan/x_trail/1105513178-1fcd3ab0/ 1 750 000 ₽
3368 https://auto.ru/cars/used/sale/nissan/serena/1105127479-03dbcc21/ 1 370 000 ₽
3369 https://auto.ru/cars/used/sale/nissan/x_trail/1105374127-ab791e81/ 1 863 000 ₽
3370 https://auto.ru/cars/used/sale/nissan/presea/1105645441-5f7f6329/ 355 000 ₽
3371 https://auto.ru/cars/used/sale/nissan/micra/1104848059-776fdc3b/ 285 000 ₽
3372 https://auto.ru/cars/used/sale/nissan/serena/1099440930-b016322b/ 900 000 ₽
3373 https://auto.ru/cars/used/sale/nissan/x_trail/1105372296-126669cb/ 1 704 000 ₽
3374 https://auto.ru/cars/used/sale/nissan/primera/1105089499-1dc20525/ 144 999 ₽
3376 https://auto.ru/cars/used/sale/nissan/rogue/1105366232-d658bfd1/ 1 750 000 ₽
3377 https://auto.ru/cars/used/sale/nissan/qashqai/1105097204-fa24129a/ 1 200 000 ₽
3378 https://auto.ru/cars/used/sale/nissan/note/1105370697-57d72a76/ 549 900 ₽
3379 https://auto.ru/cars/used/sale/nissan/almera/1105561261-d5c34651/ 389 999 ₽
3380 https://au

In [19]:
get_content("Infiniti")

INFINITI: 1464 urls to parse
starting from the beginning...
creating CONTENT/INFINITI_content.csv
1 https://auto.ru/cars/used/sale/infiniti/fx/1104637076-7cf06547/ 1 040 000 ₽
2 https://auto.ru/cars/used/sale/infiniti/g35/1105366628-c6cabf49/ 799 000 ₽
3 https://auto.ru/cars/used/sale/infiniti/qx56/1104982732-839007e9/ 900 000 ₽
4 https://auto.ru/cars/used/sale/infiniti/fx/1105083535-9b4b55a7/ 625 000 ₽
5 https://auto.ru/cars/used/sale/infiniti/fx/1105658577-c8313a42/ 515 000 ₽
6 https://auto.ru/cars/used/sale/infiniti/g35/1105409065-b38b4728/ 1 000 000 ₽
7 https://auto.ru/cars/used/sale/infiniti/qx56/1103759875-9099d077/ 740 000 ₽
8 https://auto.ru/cars/used/sale/infiniti/g35/1105651441-46bb0a0d/ 700 000 ₽
9 https://auto.ru/cars/used/sale/infiniti/fx/1105155362-33b9b0b8/ 1 400 000 ₽
10 https://auto.ru/cars/used/sale/infiniti/fx/1105208586-3682026d/ 950 000 ₽
11 https://auto.ru/cars/used/sale/infiniti/qx56/1105365612-1dbe4c6e/ 730 000 ₽
12 https://auto.ru/cars/used/sale/infiniti/fx/110

132 https://auto.ru/cars/used/sale/infiniti/qx70/1105210579-42398ce2/ 2 295 000 ₽
133 https://auto.ru/cars/used/sale/infiniti/qx4/1105631792-895b02ca/ 700 000 ₽
134 https://auto.ru/cars/used/sale/infiniti/qx50/1105651019-049e033c/ 3 199 000 ₽
135 https://auto.ru/cars/used/sale/infiniti/qx80/1105642434-2d30f385/ 3 134 000 ₽
136 https://auto.ru/cars/used/sale/infiniti/qx70/1104580644-349a9bfc/ 1 599 000 ₽
137 https://auto.ru/cars/used/sale/infiniti/q60/1105665577-01519fd5/ 3 120 000 ₽
138 https://auto.ru/cars/used/sale/infiniti/fx/1104733539-cef2f6e7/ 1 899 000 ₽
139 https://auto.ru/cars/used/sale/infiniti/q/1105394181-f7d22e4a/ 540 000 ₽
140 https://auto.ru/cars/used/sale/infiniti/q30/1105646182-994f817c/ 2 450 000 ₽
141 https://auto.ru/cars/used/sale/infiniti/qx50/1104765984-c3cfb075/ 2 690 000 ₽
142 https://auto.ru/cars/used/sale/infiniti/fx/1105357044-58e54c4e/ 1 409 000 ₽
143 https://auto.ru/cars/used/sale/infiniti/fx/1105644819-7271e7cb/ 700 000 ₽
144 https://auto.ru/cars/used/sale

248 https://auto.ru/cars/used/sale/infiniti/qx70/1105610117-4d651b58/ 2 290 000 ₽
249 https://auto.ru/cars/used/sale/infiniti/m/1105564994-1552a34a/ 1 450 000 ₽
250 https://auto.ru/cars/used/sale/infiniti/qx70/1104722435-9abe5153/ 2 590 000 ₽
251 https://auto.ru/cars/used/sale/infiniti/qx60/1105607205-18cf2892/ 3 427 000 ₽
252 https://auto.ru/cars/used/sale/infiniti/m/1105519620-66e9e672/ 1 349 000 ₽
253 https://auto.ru/cars/used/sale/infiniti/qx70/1105659402-87c514c6/ 1 750 000 ₽
254 https://auto.ru/cars/used/sale/infiniti/fx/1105659433-ca542d0f/ 890 000 ₽
255 https://auto.ru/cars/used/sale/infiniti/q50/1105607468-17a7c538/ 2 194 000 ₽
256 https://auto.ru/cars/used/sale/infiniti/qx70/1105659429-0937027c/ 2 050 000 ₽
257 https://auto.ru/cars/used/sale/infiniti/fx/1105659405-879ce192/ 550 000 ₽
258 https://auto.ru/cars/used/sale/infiniti/fx/1105659403-35523c0c/ 1 320 000 ₽
259 https://auto.ru/cars/used/sale/infiniti/fx/1105603856-dcfe6e15/ 1 480 000 ₽
260 https://auto.ru/cars/used/sale/

353 https://auto.ru/cars/used/sale/infiniti/qx80/1105029595-09a1c0f3/ 2 340 000 ₽
354 https://auto.ru/cars/used/sale/infiniti/qx80/1105047494-3ac30c3a/ 5 500 000 ₽
355 https://auto.ru/cars/used/sale/infiniti/m/1105076583-2dc35b0b/ 535 000 ₽
356 https://auto.ru/cars/used/sale/infiniti/qx80/1105204691-598d9a81/ 3 090 000 ₽
357 https://auto.ru/cars/used/sale/infiniti/qx50/1105512034-49a899dc/ 1 748 000 ₽
358 https://auto.ru/cars/used/sale/infiniti/qx56/1105537409-89a69b52/ 1 780 000 ₽
359 https://auto.ru/cars/used/sale/infiniti/q50/1105534334-e30f7883/ 1 735 000 ₽
360 https://auto.ru/cars/used/sale/infiniti/qx50/1105420334-5fa77c36/ 1 999 000 ₽
361 https://auto.ru/cars/used/sale/infiniti/fx/1105532879-1602cf7b/ 690 000 ₽
362 https://auto.ru/cars/used/sale/infiniti/qx70/1105530781-c8219f33/ 2 700 000 ₽
363 https://auto.ru/cars/used/sale/infiniti/q50/1103331675-5328ce6d/ 2 370 000 ₽
364 https://auto.ru/cars/used/sale/infiniti/fx/1105523709-4da51aef/ 769 000 ₽
365 https://auto.ru/cars/used/s

459 https://auto.ru/cars/used/sale/infiniti/qx50/1105425178-7c6ae236/ 1 980 000 ₽
460 https://auto.ru/cars/used/sale/infiniti/qx56/1105424306-3a577477/ 1 999 000 ₽
461 https://auto.ru/cars/used/sale/infiniti/qx80/1105422633-8c5c0c83/ 4 499 000 ₽
462 https://auto.ru/cars/used/sale/infiniti/qx60/1105418139-9b55b3b4/ 4 299 000 ₽
463 https://auto.ru/cars/used/sale/infiniti/q70/1104751802-24fa923b/ 1 580 000 ₽
464 https://auto.ru/cars/used/sale/infiniti/qx60/1105419937-02fa65ff/ 2 395 000 ₽
465 https://auto.ru/cars/used/sale/infiniti/qx50/1105418141-3d9e0e64/ 4 125 000 ₽
466 https://auto.ru/cars/used/sale/infiniti/qx50/1105418142-defc9e75/ 4 125 000 ₽
467 https://auto.ru/cars/used/sale/infiniti/qx60/1105150197-2129ebca/ 2 799 000 ₽
468 https://auto.ru/cars/used/sale/infiniti/qx56/1104094991-a9e3374d/ 1 450 000 ₽
469 https://auto.ru/cars/used/sale/infiniti/m/1105122716-a58e898b/ 1 189 000 ₽
470 https://auto.ru/cars/used/sale/infiniti/qx50/1105415884-84e9705e/ 2 077 000 ₽
471 https://auto.ru/

...

996 https://auto.ru/cars/used/sale/infiniti/qx50/1104775619-3fd64605/ 3 445 000 ₽
997 https://auto.ru/cars/used/sale/infiniti/g35/1105623795-7286c2b8/ 1 450 000 ₽
998 https://auto.ru/cars/used/sale/infiniti/q50/1098046180-f8859fe0/ 1 750 000 ₽
999 https://auto.ru/cars/used/sale/infiniti/qx80/1104009866-84a1ed44/ 3 270 000 ₽
1000 links parsed
adding to CONTENT/INFINITI_content.csv
1000 https://auto.ru/cars/used/sale/infiniti/q50/1105442191-11e7d026/ 2 500 000 ₽
1002 https://auto.ru/cars/used/sale/infiniti/fx/1103739622-0bd54316/ 550 000 ₽
1003 https://auto.ru/cars/used/sale/infiniti/qx80/1104912022-ed361e83/ 2 600 000 ₽
1004 https://auto.ru/cars/used/sale/infiniti/m/1104762405-15b0645b/ 1 215 000 ₽
1005 https://auto.ru/cars/used/sale/infiniti/fx/1104601469-e77f7ebc/ 950 000 ₽
1006 https://auto.ru/cars/used/sale/infiniti/fx/1101645169-9780ef61/ 1 150 000 ₽
1007 https://auto.ru/cars/used/sale/infiniti/fx/1105091921-2068c7fb/ 2 200 000 ₽
1010 https://auto.ru/cars/used/sale/infiniti/fx/1103

1108 https://auto.ru/cars/used/sale/infiniti/qx50/1104126662-ec3c75df/ 1 850 000 ₽
1109 https://auto.ru/cars/used/sale/infiniti/fx/1104985200-40f98ec4/ 575 000 ₽
1110 https://auto.ru/cars/used/sale/infiniti/qx56/1102258530-46cb47c4/ 1 100 000 ₽
1112 https://auto.ru/cars/used/sale/infiniti/qx70/1105607262-eb895372/ 2 600 000 ₽
1113 https://auto.ru/cars/used/sale/infiniti/fx/1105581179-c0505110/ 1 600 000 ₽
1114 https://auto.ru/cars/used/sale/infiniti/fx/1105496095-fb5e0c22/ 750 000 ₽
1115 https://auto.ru/cars/used/sale/infiniti/qx70/1104772484-de880b79/ 2 309 000 ₽
1116 https://auto.ru/cars/used/sale/infiniti/fx/1105427745-6ecac927/ 1 630 000 ₽
1117 https://auto.ru/cars/used/sale/infiniti/qx70/1105218417-12e38689/ 2 390 000 ₽
1118 https://auto.ru/cars/used/sale/infiniti/qx56/1101797648-afb59db4/ 2 700 000 ₽
1119 https://auto.ru/cars/used/sale/infiniti/fx/1105584407-18a01ee2/ 1 080 000 ₽
1120 https://auto.ru/cars/used/sale/infiniti/qx80/1105247864-feb44e7d/ 2 990 000 ₽
1121 https://auto.

1221 https://auto.ru/cars/used/sale/infiniti/qx80/1105534192-0a8c3cc7/ 2 950 000 ₽
1222 https://auto.ru/cars/used/sale/infiniti/g35/1103376926-026ba6a7/ 1 300 000 ₽
1223 https://auto.ru/cars/used/sale/infiniti/g35/1105406584-06a86afb/ 1 450 000 ₽
1224 https://auto.ru/cars/used/sale/infiniti/qx70/1105535905-59e085e2/ 1 650 000 ₽
1225 https://auto.ru/cars/used/sale/infiniti/m/1105235038-3b466604/ 1 150 000 ₽
1226 https://auto.ru/cars/used/sale/infiniti/fx/1104738299-6b30104f/ 670 000 ₽
1227 https://auto.ru/cars/used/sale/infiniti/fx/1105620872-f68cbd7c/ 550 000 ₽
1228 https://auto.ru/cars/used/sale/infiniti/g35/1105590189-a38c2031/ 949 000 ₽
1229 https://auto.ru/cars/used/sale/infiniti/g35/1105304520-49465a73/ 850 000 ₽
1231 https://auto.ru/cars/used/sale/infiniti/g35/1104779232-50bc9146/ 950 000 ₽
1232 https://auto.ru/cars/used/sale/infiniti/fx/1102143125-731c8f81/ 1 330 000 ₽
1233 https://auto.ru/cars/used/sale/infiniti/fx/1096327682-fcc5e589/ 1 770 000 ₽
1234 https://auto.ru/cars/used

1327 https://auto.ru/cars/used/sale/infiniti/m/1105419405-9bab7e26/ 800 000 ₽
1328 https://auto.ru/cars/used/sale/infiniti/fx/1105597472-7e462476/ 1 358 000 ₽
1329 https://auto.ru/cars/used/sale/infiniti/qx56/1101178580-fd515525/ 1 250 000 ₽
1330 https://auto.ru/cars/used/sale/infiniti/g35/1068584078-74d976ef/ 660 000 ₽
1331 https://auto.ru/cars/used/sale/infiniti/g35/1105214978-9f3bade1/ 1 250 000 ₽
1332 https://auto.ru/cars/used/sale/infiniti/g35/1105631935-a42e200e/ 580 000 ₽
1333 https://auto.ru/cars/used/sale/infiniti/fx/1105568386-e769c6ef/ 550 000 ₽
1335 https://auto.ru/cars/used/sale/infiniti/fx/1104152141-69bb48be/ 1 899 999 ₽
1336 https://auto.ru/cars/used/sale/infiniti/fx/1105313627-eccca4e7/ 1 280 000 ₽
1337 https://auto.ru/cars/used/sale/infiniti/fx/1104251084-bb1ecffe/ 650 000 ₽
1338 https://auto.ru/cars/used/sale/infiniti/g35/1105540328-54dd816d/ 1 150 000 ₽
1339 https://auto.ru/cars/used/sale/infiniti/fx/1098454366-58e9998e/ 1 149 000 ₽
1340 https://auto.ru/cars/used/sa

1432 https://auto.ru/cars/used/sale/infiniti/qx80/1105324503-03d75ea1/ 5 900 000 ₽
1433 https://auto.ru/cars/used/sale/infiniti/g35/1105178280-96547c3a/ 850 000 ₽
1434 https://auto.ru/cars/used/sale/infiniti/qx70/1103345728-23da91d0/ 2 170 000 ₽
1435 https://auto.ru/cars/used/sale/infiniti/qx4/1105082749-edd2fb38/ 550 000 ₽
1436 https://auto.ru/cars/used/sale/infiniti/q/1103633252-50e01e8f/ 450 000 ₽
1437 https://auto.ru/cars/used/sale/infiniti/fx/1105280278-efab0fb8/ 1 570 000 ₽
1438 https://auto.ru/cars/used/sale/infiniti/fx/1105563705-4fb2b9b0/ 1 450 000 ₽
1439 https://auto.ru/cars/used/sale/infiniti/qx4/1082655106-4d195719/ 600 000 ₽
1440 https://auto.ru/cars/used/sale/infiniti/fx/1105090012-4a2c2839/ 1 460 000 ₽
1441 https://auto.ru/cars/used/sale/infiniti/qx70/1104895445-f53f1c29/ 2 220 000 ₽
1442 https://auto.ru/cars/used/sale/infiniti/qx56/1105450798-ed4b4801/ 1 850 000 ₽
1443 https://auto.ru/cars/used/sale/infiniti/ex/1085221744-14319799/ 1 445 000 ₽
1444 https://auto.ru/cars/

In [20]:
get_content("Mercedes-Benz")

MERCEDES-BENZ: 3410 urls to parse
starting from the beginning...
creating CONTENT/MERCEDES-BENZ_content.csv
1 https://auto.ru/cars/used/sale/mercedes/gle_klasse_amg/1105179554-2af88eb6/ 4 490 000 ₽
2 https://auto.ru/cars/used/sale/mercedes/w128/1101388727-4e4e0593/ 14 628 455 ₽
5 https://auto.ru/cars/used/sale/mercedes/gls_klasse/1104906019-d7425770/ 3 790 000 ₽
6 https://auto.ru/cars/used/sale/mercedes/e_klasse/1105309258-9ff76181/ 1 045 000 ₽
7 https://auto.ru/cars/used/sale/mercedes/g_klasse_amg/1105101658-a77199f4/ 19 000 000 ₽
8 https://auto.ru/cars/used/sale/mercedes/m_klasse/1105607062-f37ad8d9/ 715 000 ₽
9 https://auto.ru/cars/used/sale/mercedes/g_klasse/1105532667-088ddc0c/ 2 400 000 ₽
10 https://auto.ru/cars/used/sale/mercedes/gl_klasse/1105285630-4df02889/ 2 349 000 ₽
11 https://auto.ru/cars/used/sale/mercedes/c_klasse/1105665277-6dbf7923/ 1 100 000 ₽
12 https://auto.ru/cars/used/sale/mercedes/e_klasse/1104722503-7eacc09a/ 995 000 ₽
13 https://auto.ru/cars/used/sale/mercedes

109 https://auto.ru/cars/used/sale/mercedes/e_klasse/1105665991-9946e7e1/ 3 099 000 ₽
110 https://auto.ru/cars/used/sale/mercedes/e_klasse/1105388309-5c83f815/ 4 397 000 ₽
111 https://auto.ru/cars/used/sale/mercedes/gle_klasse/1105186880-201fc5d3/ 2 976 000 ₽
112 https://auto.ru/cars/used/sale/mercedes/vito/1105607454-81059d3c/ 3 550 000 ₽
113 https://auto.ru/cars/used/sale/mercedes/e_klasse/1105578460-006af7af/ 3 387 000 ₽
114 https://auto.ru/cars/used/sale/mercedes/cla_klasse/1105632490-daedb3b9/ 1 479 000 ₽
115 https://auto.ru/cars/used/sale/mercedes/e_klasse/1105661022-3dec000c/ 1 379 000 ₽
116 https://auto.ru/cars/used/sale/mercedes/s_klasse/1105216276-ea058543/ 4 075 000 ₽
117 https://auto.ru/cars/used/sale/mercedes/a_klasse/1105514100-85947af7/ 595 000 ₽
118 https://auto.ru/cars/used/sale/mercedes/gl_klasse/1105662770-23186e61/ 1 888 000 ₽
119 https://auto.ru/cars/used/sale/mercedes/gls_klasse/1105442926-b4de6db1/ 4 140 000 ₽
120 https://auto.ru/cars/used/sale/mercedes/glc_klass

221 https://auto.ru/cars/used/sale/mercedes/glc_klasse_amg/1105218640-89d159a4/ 3 490 000 ₽
223 https://auto.ru/cars/used/sale/mercedes/e_klasse/1105495612-162e6b19/ 2 945 000 ₽
224 https://auto.ru/cars/used/sale/mercedes/glc_klasse/1105027212-6c38352b/ 3 100 000 ₽
225 https://auto.ru/cars/used/sale/mercedes/gl_klasse/1105090934-d3e6cbb4/ 839 000 ₽
226 https://auto.ru/cars/used/sale/mercedes/b_klasse/1105657534-fa9b7e06/ 1 699 000 ₽
228 https://auto.ru/cars/used/sale/mercedes/s_klasse/1105265371-402f4d10/ 2 939 000 ₽
229 https://auto.ru/cars/used/sale/mercedes/glc_klasse/1105581446-33bb91bc/ 4 100 000 ₽
230 https://auto.ru/cars/used/sale/mercedes/gle_klasse_coupe/1105334969-87184fe2/ 3 989 000 ₽
231 https://auto.ru/cars/used/sale/mercedes/s_klasse/1105337016-ba861c85/ 5 045 000 ₽
232 https://auto.ru/cars/used/sale/mercedes/gla_class/1104186198-b3d760a5/ 1 700 000 ₽
233 https://auto.ru/cars/used/sale/mercedes/g_klasse_amg/1104715948-77ab00c7/ 5 190 000 ₽
234 https://auto.ru/cars/used/sa

331 https://auto.ru/cars/used/sale/mercedes/cla_klasse/1105652195-e9aaa407/ 1 548 000 ₽
332 https://auto.ru/cars/used/sale/mercedes/m_klasse_amg/1099744294-02f51cb0/ 2 450 000 ₽
333 https://auto.ru/cars/used/sale/mercedes/glk_klasse/1105652700-b04b5bed/ 1 799 000 ₽
335 https://auto.ru/cars/used/sale/mercedes/glc_klasse/1105651548-d37c81be/ 4 580 000 ₽
336 https://auto.ru/cars/used/sale/mercedes/c_klasse/1102834485-a80569b0/ 245 000 ₽
337 https://auto.ru/cars/used/sale/mercedes/e_klasse/1105665830-99cba225/ 2 700 000 ₽
338 https://auto.ru/cars/used/sale/mercedes/cla_klasse/1105652191-ab8a5495/ 1 538 000 ₽
339 https://auto.ru/cars/used/sale/mercedes/glc_coupe/1105652704-0b0c2369/ 3 789 000 ₽
340 https://auto.ru/cars/used/sale/mercedes/cl_klasse_amg/1105577416-92759c12/ 1 500 000 ₽
341 https://auto.ru/cars/used/sale/mercedes/glk_klasse/1105653692-f18c929f/ 1 348 000 ₽
342 https://auto.ru/cars/used/sale/mercedes/cla_klasse/1105652203-add12bb7/ 1 498 000 ₽
343 https://auto.ru/cars/used/sale

437 https://auto.ru/cars/used/sale/mercedes/e_klasse/1105650099-48b14e73/ 2 836 000 ₽
438 https://auto.ru/cars/used/sale/mercedes/s_klasse/1105555758-81d8a21c/ 897 000 ₽
439 https://auto.ru/cars/used/sale/mercedes/vito/1105530869-cc4dbadc/ 1 137 000 ₽
440 https://auto.ru/cars/used/sale/mercedes/vito/1105322705-0ac8c45c/ 1 039 000 ₽
441 https://auto.ru/cars/used/sale/mercedes/glc_klasse/1105304770-cd9f88be/ 2 549 597 ₽
442 https://auto.ru/cars/used/sale/mercedes/g_klasse_amg/1104975484-907916a5/ 6 190 000 ₽
443 https://auto.ru/cars/used/sale/mercedes/viano/1105255333-1ad39f1c/ 2 275 000 ₽
444 https://auto.ru/cars/used/sale/mercedes/cla_klasse/1105210009-20f55597/ 1 527 000 ₽
445 https://auto.ru/cars/used/sale/mercedes/cla_klasse/1105210008-0ad8a86d/ 1 577 000 ₽
447 https://auto.ru/cars/used/sale/mercedes/s_klasse/1105211315-629a9540/ 4 200 000 ₽
448 https://auto.ru/cars/used/sale/mercedes/m_klasse/1105635208-44a02bb5/ 1 800 000 ₽
449 https://auto.ru/cars/used/sale/mercedes/gla_class/110

...

3001 https://auto.ru/cars/used/sale/mercedes/m_klasse/1105202142-d854f6c0/ 2 630 000 ₽
3002 https://auto.ru/cars/used/sale/mercedes/g_klasse/1104234087-8a67ea4a/ 4 500 000 ₽
3003 https://auto.ru/cars/used/sale/mercedes/s_klasse/1105405824-65f44a18/ 420 000 ₽
3004 https://auto.ru/cars/used/sale/mercedes/c_klasse/1105334305-b94eec12/ 2 552 000 ₽
3005 https://auto.ru/cars/used/sale/mercedes/gls_klasse/1105433482-e7038f26/ 4 099 000 ₽
3006 https://auto.ru/cars/used/sale/mercedes/e_klasse/1105172919-7bf79c5f/ 350 000 ₽
3007 https://auto.ru/cars/used/sale/mercedes/gl_klasse/1105171172-f0cd7606/ 1 820 000 ₽
3008 https://auto.ru/cars/used/sale/mercedes/gle_klasse/1105178842-eee12a9f/ 3 465 000 ₽
3009 https://auto.ru/cars/used/sale/mercedes/cla_klasse/1105431232-9d78c860/ 1 985 000 ₽
3010 https://auto.ru/cars/used/sale/mercedes/maybach_gls/1105430623-7da55138/ 25 999 000 ₽
3011 https://auto.ru/cars/used/sale/mercedes/slk_klasse/1105435520-91e6a89b/ 2 699 990 ₽
3012 https://auto.ru/cars/used/sal

3104 https://auto.ru/cars/used/sale/mercedes/v_klasse/1105646977-4273920a/ 5 400 000 ₽
3105 https://auto.ru/cars/used/sale/mercedes/m_klasse/1105422946-d6989e9d/ 1 557 000 ₽
3106 https://auto.ru/cars/used/sale/mercedes/w123/1103700161-d3af73f5/ 320 000 ₽
3107 https://auto.ru/cars/used/sale/mercedes/glc_klasse/1105287226-1e515c4f/ 3 300 000 ₽
3108 https://auto.ru/cars/used/sale/mercedes/gls_klasse/1105127132-c4e67c11/ 4 290 000 ₽
3109 https://auto.ru/cars/used/sale/mercedes/e_klasse/1105419307-a02c1cb9/ 2 490 000 ₽
3110 https://auto.ru/cars/used/sale/mercedes/gle_klasse/1104584214-4f18e027/ 3 100 000 ₽
3111 https://auto.ru/cars/used/sale/mercedes/sl_klasse/1103871954-5a10d4ea/ 1 300 000 ₽
3112 https://auto.ru/cars/used/sale/mercedes/c_klasse/1105419912-f4122baf/ 1 989 000 ₽
3113 https://auto.ru/cars/used/sale/mercedes/gl_klasse/1105419927-6a6bb0d3/ 3 249 000 ₽
3114 https://auto.ru/cars/used/sale/mercedes/gls_klasse/1105419308-3e65d1a3/ 5 862 150 ₽
3115 https://auto.ru/cars/used/sale/mer

3205 https://auto.ru/cars/used/sale/mercedes/cla_klasse/1105396486-2b54787e/ 3 049 000 ₽
3206 https://auto.ru/cars/used/sale/mercedes/c_klasse/1104751906-36e9935b/ 657 000 ₽
3207 https://auto.ru/cars/used/sale/mercedes/gls_klasse/1105391181-494796ac/ 4 649 000 ₽
3208 https://auto.ru/cars/used/sale/mercedes/gle_klasse/1105645642-20a2e8a8/ 4 000 000 ₽
3209 https://auto.ru/cars/used/sale/mercedes/g_klasse_amg/1105396381-a5d47c6e/ 8 200 000 ₽
3210 https://auto.ru/cars/used/sale/mercedes/e_klasse/1105397687-95c8b591/ 3 349 000 ₽
3211 https://auto.ru/cars/used/sale/mercedes/gle_klasse_coupe/1105394778-40c30ef9/ 4 490 000 ₽
3213 https://auto.ru/cars/used/sale/mercedes/glc_coupe/1105397209-bad21f19/ 3 670 000 ₽
3214 https://auto.ru/cars/used/sale/mercedes/glb_klasse/1105396634-b62610f9/ 3 829 000 ₽
3215 https://auto.ru/cars/used/sale/mercedes/glc_coupe/1105394314-9fa8dc32/ 3 390 000 ₽
3216 https://auto.ru/cars/used/sale/mercedes/m_klasse/1104740092-f829d91f/ 2 367 000 ₽
3217 https://auto.ru/ca

3305 https://auto.ru/cars/used/sale/mercedes/glc_klasse/1105368735-55366fec/ 3 075 000 ₽
3306 https://auto.ru/cars/used/sale/mercedes/viano/1105368896-ac747ae6/ 1 315 000 ₽
3307 https://auto.ru/cars/used/sale/mercedes/gle_klasse_coupe_amg/1105184574-4ac082ed/ 5 495 000 ₽
3308 https://auto.ru/cars/used/sale/mercedes/w124/1105645436-369952dc/ 300 000 ₽
3309 https://auto.ru/cars/used/sale/mercedes/gla_class_amg/1104908990-4b821296/ 2 411 000 ₽
3310 https://auto.ru/cars/used/sale/mercedes/glc_klasse/1105112646-deb66d48/ 3 324 000 ₽
3311 https://auto.ru/cars/used/sale/mercedes/e_klasse/1105368573-cdffc09e/ 2 400 000 ₽
3312 https://auto.ru/cars/used/sale/mercedes/s_klasse/1105366802-ce683565/ 7 750 000 ₽
3313 https://auto.ru/cars/used/sale/mercedes/gls_klasse/1105336588-ae4252cd/ 8 690 000 ₽
3314 https://auto.ru/cars/used/sale/mercedes/g_klasse_amg/1104872960-a8c90e22/ 7 199 000 ₽
3315 https://auto.ru/cars/used/sale/mercedes/e_klasse/1105368897-a564b134/ 3 139 864 ₽
3316 https://auto.ru/cars

3405 https://auto.ru/cars/used/sale/mercedes/viano/1105242733-a211e523/ 1 899 000 ₽
3406 https://auto.ru/cars/used/sale/mercedes/gla_class/1105348880-8467cdd9/ 2 130 000 ₽
3407 https://auto.ru/cars/used/sale/mercedes/a_klasse/1105143609-b49b1183/ 375 000 ₽
3408 https://auto.ru/cars/used/sale/mercedes/w124/1105521568-1eb944af/ 170 000 ₽
3409 https://auto.ru/cars/used/sale/mercedes/e_klasse/1105346256-aafc1208/ 1 649 897 ₽
3410 https://auto.ru/cars/used/sale/mercedes/vito/1105239357-55da0ec5/ 700 000 ₽
3410 links parsed
adding to CONTENT/MERCEDES-BENZ_content.csv


In [21]:
get_content("Toyota")

TOYOTA: 3350 urls to parse
starting from the beginning...
creating CONTENT/TOYOTA_content.csv
1 https://auto.ru/cars/used/sale/toyota/camry/1105398248-b47be3d9/ 2 750 000 ₽
2 https://auto.ru/cars/used/sale/toyota/camry/1105496628-72b506f9/ 3 250 000 ₽
3 https://auto.ru/cars/used/sale/toyota/yaris/1105598588-582963f8/ 250 000 ₽
5 https://auto.ru/cars/used/sale/toyota/corolla/1105629752-bf768a54/ 650 000 ₽
7 https://auto.ru/cars/used/sale/toyota/land_cruiser_prado/1105663544-0946f909/ 549 000 ₽
8 https://auto.ru/cars/used/sale/toyota/camry/1105601970-44883b7c/ 2 750 000 ₽
9 https://auto.ru/cars/used/sale/toyota/celica/1105666373-a104f49f/ 750 000 ₽
10 https://auto.ru/cars/used/sale/toyota/corolla/1105605477-4dc7102d/ 495 000 ₽
11 https://auto.ru/cars/used/sale/toyota/corolla/1105665481-b61ad4d0/ 617 000 ₽
12 https://auto.ru/cars/used/sale/toyota/camry/1104930519-d0695327/ 895 000 ₽
13 https://auto.ru/cars/used/sale/toyota/camry/1105354477-7cbaa5db/ 680 000 ₽
14 https://auto.ru/cars/used/

108 https://auto.ru/cars/used/sale/toyota/camry/1105664654-1d73b7d0/ 2 499 000 ₽
111 https://auto.ru/cars/used/sale/toyota/corolla/1105079596-2eb68794/ 1 689 000 ₽
112 https://auto.ru/cars/used/sale/toyota/camry/1105664973-338ee6f3/ 2 586 000 ₽
113 https://auto.ru/cars/used/sale/toyota/rav_4/1105660707-8e4730d0/ 1 710 000 ₽
114 https://auto.ru/cars/used/sale/toyota/avensis/1105010360-bd290b93/ 829 000 ₽
115 https://auto.ru/cars/used/sale/toyota/camry/1105660645-c3df9e55/ 1 701 000 ₽
116 https://auto.ru/cars/used/sale/toyota/fortuner/1103973350-5488490f/ 2 895 000 ₽
117 https://auto.ru/cars/used/sale/toyota/land_cruiser/1105088066-43e4123e/ 2 159 000 ₽
118 https://auto.ru/cars/used/sale/toyota/camry/1105665891-c0bc8ec7/ 2 557 000 ₽
119 https://auto.ru/cars/used/sale/toyota/venza/1105665533-b67d002f/ 1 748 000 ₽
120 https://auto.ru/cars/used/sale/toyota/camry/1105662996-84b812cb/ 2 199 000 ₽
121 https://auto.ru/cars/used/sale/toyota/camry/1105662039-3585ef60/ 1 578 000 ₽
122 https://auto

221 https://auto.ru/cars/used/sale/toyota/rav_4/1103509842-76c09288/ 1 576 000 ₽
222 https://auto.ru/cars/used/sale/toyota/corolla/1105655471-a7ce19d4/ 1 052 000 ₽
223 https://auto.ru/cars/used/sale/toyota/camry/1105655119-ca0a337e/ 1 330 000 ₽
224 https://auto.ru/cars/used/sale/toyota/rav_4/1103303544-c4247b37/ 1 420 000 ₽
225 https://auto.ru/cars/used/sale/toyota/celica/1100277700-6e9b2025/ 250 000 ₽
226 https://auto.ru/cars/used/sale/toyota/camry/1105656114-9f0096cd/ 1 587 000 ₽
227 https://auto.ru/cars/used/sale/toyota/rav_4/1105656267-78a4489d/ 1 679 000 ₽
229 https://auto.ru/cars/used/sale/toyota/rav_4/1105656251-87f4710b/ 1 727 000 ₽
230 https://auto.ru/cars/used/sale/toyota/c_hr/1103168000-4d94fba3/ 1 800 000 ₽
231 https://auto.ru/cars/used/sale/toyota/auris/1105657260-eb0cc0f1/ 645 000 ₽
232 https://auto.ru/cars/used/sale/toyota/camry/1105655407-83ebe05f/ 1 997 000 ₽
233 https://auto.ru/cars/used/sale/toyota/camry/1105654969-838f9e19/ 890 000 ₽
235 https://auto.ru/cars/used/sa

330 https://auto.ru/cars/used/sale/toyota/auris/1105650098-bd377b35/ 639 000 ₽
331 https://auto.ru/cars/used/sale/toyota/rav_4/1105650634-c811caaf/ 2 275 000 ₽
333 https://auto.ru/cars/used/sale/toyota/rav_4/1105665805-ca70440f/ 535 000 ₽
334 https://auto.ru/cars/used/sale/toyota/land_cruiser/1105629039-6309122f/ 2 570 000 ₽
335 https://auto.ru/cars/used/sale/toyota/land_cruiser/1105649639-76105b75/ 6 399 000 ₽
336 https://auto.ru/cars/used/sale/toyota/auris/1105452677-478bb008/ 795 000 ₽
337 https://auto.ru/cars/used/sale/toyota/starlet/1105665782-0000f492/ 75 000 ₽
338 https://auto.ru/cars/used/sale/toyota/rav_4/1105649296-c4e07834/ 2 807 000 ₽
339 https://auto.ru/cars/used/sale/toyota/starlet/1105624188-468e3195/ 160 000 ₽
340 https://auto.ru/cars/used/sale/toyota/land_cruiser/1105474514-f63df8f6/ 4 199 000 ₽
341 https://auto.ru/cars/used/sale/toyota/corolla/1105649556-2f5a4e12/ 480 000 ₽
342 https://auto.ru/cars/used/sale/toyota/corolla/1105208536-b8c404cc/ 1 590 000 ₽
343 https://

450 https://auto.ru/cars/used/sale/toyota/land_cruiser_prado/1105626406-5ab07141/ 2 180 000 ₽
451 https://auto.ru/cars/used/sale/toyota/land_cruiser_prado/1105625715-273fa3d6/ 1 470 000 ₽
452 https://auto.ru/cars/used/sale/toyota/land_cruiser_prado/1105626413-70b65448/ 3 150 000 ₽
453 https://auto.ru/cars/used/sale/toyota/rav_4/1105636572-9f41be89/ 1 628 000 ₽
454 https://auto.ru/cars/used/sale/toyota/corolla/1104200655-e78b6259/ 350 000 ₽
455 https://auto.ru/cars/used/sale/toyota/verso/1105625803-3c48c70c/ 877 000 ₽
456 https://auto.ru/cars/used/sale/toyota/highlander/1105234973-4e4ec291/ 3 795 000 ₽
457 https://auto.ru/cars/used/sale/toyota/camry/1105545356-32ba6424/ 2 900 000 ₽
458 https://auto.ru/cars/used/sale/toyota/corolla/1105636556-1e8ef369/ 145 000 ₽
exception: 
https://auto.ru/cars/used/sale/toyota/land_cruiser/1105636557-9243324b/
460 https://auto.ru/cars/used/sale/toyota/rav_4/1105037502-8dece41a/ 3 057 000 ₽
461 https://auto.ru/cars/used/sale/toyota/camry/1105624966-32bac

...

2845 https://auto.ru/cars/used/sale/toyota/corolla/1105490956-e997c1b3/ 364 000 ₽
2846 https://auto.ru/cars/used/sale/toyota/land_cruiser_prado/1105490156-28dc83b0/ 1 575 000 ₽
2847 https://auto.ru/cars/used/sale/toyota/corolla/1105492319-cbec4222/ 770 000 ₽
2848 https://auto.ru/cars/used/sale/toyota/camry/1105490615-e2e76758/ 1 258 000 ₽
2849 https://auto.ru/cars/used/sale/toyota/corolla/1105490897-e58d6d18/ 499 000 ₽
2850 https://auto.ru/cars/used/sale/toyota/highlander/1105491154-b37223f1/ 5 150 000 ₽
2851 https://auto.ru/cars/used/sale/toyota/lite_ace/1105653513-dc1a0530/ 280 000 ₽
2852 https://auto.ru/cars/used/sale/toyota/corolla/1105653518-a180e168/ 450 000 ₽
2853 https://auto.ru/cars/used/sale/toyota/camry/1105621832-f358a1d7/ 595 000 ₽
2855 https://auto.ru/cars/used/sale/toyota/hilux/1105489615-f971bcc9/ 2 139 000 ₽
2856 https://auto.ru/cars/used/sale/toyota/alphard/1105489668-ed5a8088/ 2 050 000 ₽
2857 https://auto.ru/cars/used/sale/toyota/alphard/1105491402-56b80e86/ 3 399 0

2943 https://auto.ru/cars/used/sale/toyota/rav_4/1105481744-51f28330/ 1 799 000 ₽
2944 https://auto.ru/cars/used/sale/toyota/corolla/1105478779-148c6166/ 475 000 ₽
2945 https://auto.ru/cars/used/sale/toyota/camry/1105477531-96b1d0ba/ 1 896 230 ₽
2946 https://auto.ru/cars/used/sale/toyota/lite_ace/1105475459-cdaa8690/ 175 000 ₽
2947 https://auto.ru/cars/used/sale/toyota/sprinter_carib/1105652585-23bed2cc/ 200 000 ₽
2948 https://auto.ru/cars/used/sale/toyota/land_cruiser/1105476968-3840e94b/ 2 610 000 ₽
2949 https://auto.ru/cars/used/sale/toyota/rav_4/1105478012-8178d505/ 1 916 000 ₽
2950 https://auto.ru/cars/used/sale/toyota/bb/1105652504-576ef046/ 355 000 ₽
2951 https://auto.ru/cars/used/sale/toyota/avensis/1105480496-94e9bb59/ 1 199 000 ₽
2952 https://auto.ru/cars/used/sale/toyota/highlander/1105652451-0a918693/ 1 450 000 ₽
2953 https://auto.ru/cars/used/sale/toyota/highlander/1105477981-8d603aa5/ 1 589 000 ₽
2954 https://auto.ru/cars/used/sale/toyota/land_cruiser_prado/1105480163-843

3046 https://auto.ru/cars/used/sale/toyota/corolla/1105467868-7314fadf/ 950 000 ₽
3047 https://auto.ru/cars/used/sale/toyota/rav_4/1105468687-aa6fdb53/ 2 149 000 ₽
3048 https://auto.ru/cars/used/sale/toyota/land_cruiser_prado/1105467100-e7bc7646/ 919 000 ₽
3049 https://auto.ru/cars/used/sale/toyota/rav_4/1105472885-647baab9/ 499 000 ₽
3050 https://auto.ru/cars/used/sale/toyota/land_cruiser/1105472107-a3021273/ 2 960 000 ₽
3051 https://auto.ru/cars/used/sale/toyota/yaris/1105651581-f0bdabfc/ 495 000 ₽
3052 https://auto.ru/cars/used/sale/toyota/land_cruiser/1105473316-54c9f48c/ 4 190 000 ₽
3053 https://auto.ru/cars/used/sale/toyota/land_cruiser/1105472908-bc8e1e20/ 2 200 000 ₽
3054 https://auto.ru/cars/used/sale/toyota/estima/1105470378-db58bc58/ 235 000 ₽
3055 https://auto.ru/cars/used/sale/toyota/land_cruiser_prado/1105466273-21e152f5/ 1 980 000 ₽
3056 https://auto.ru/cars/used/sale/toyota/avensis/1105651555-6ad60a78/ 265 000 ₽
3057 https://auto.ru/cars/used/sale/toyota/rav_4/110565155

3148 https://auto.ru/cars/used/sale/toyota/corolla/1105650546-dc8bf881/ 1 100 000 ₽
3149 https://auto.ru/cars/used/sale/toyota/rav_4/1105455960-8b884bbd/ 549 000 ₽
3150 https://auto.ru/cars/used/sale/toyota/land_cruiser/1105126891-44db48ba/ 3 299 000 ₽
3151 https://auto.ru/cars/used/sale/toyota/corolla/1105456578-d9617912/ 1 615 555 ₽
3153 https://auto.ru/cars/used/sale/toyota/rav_4/1105334181-1320f3b4/ 1 270 000 ₽
3154 https://auto.ru/cars/used/sale/toyota/hilux/1105185857-d95f0930/ 2 069 000 ₽
3155 https://auto.ru/cars/used/sale/toyota/camry/1105411711-35e7fdf0/ 1 300 000 ₽
3156 https://auto.ru/cars/used/sale/toyota/ractis/1104242622-856305b7/ 900 000 ₽
3157 https://auto.ru/cars/used/sale/toyota/rav_4/1105459180-f4840c98/ 2 106 000 ₽
3158 https://auto.ru/cars/used/sale/toyota/sequoia/1105165031-bb26eb2d/ 4 200 000 ₽
3159 https://auto.ru/cars/used/sale/toyota/tundra/1105324620-f4e719d5/ 1 750 000 ₽
3160 https://auto.ru/cars/used/sale/toyota/corolla/1105440712-489908d4/ 510 000 ₽
3161 

3258 https://auto.ru/cars/used/sale/toyota/rav_4/1105441677-071b9887/ 3 099 900 ₽
3259 https://auto.ru/cars/used/sale/toyota/corolla/1105249278-a7080307/ 595 000 ₽
3261 https://auto.ru/cars/used/sale/toyota/land_cruiser/1105446165-556c9282/ 3 390 000 ₽
3263 https://auto.ru/cars/used/sale/toyota/camry/1105445697-97a0d205/ 2 649 000 ₽
3264 https://auto.ru/cars/used/sale/toyota/altezza/1105649406-d16838f4/ 640 000 ₽
3265 https://auto.ru/cars/used/sale/toyota/sprinter/1105649385-ac03b1a3/ 75 000 ₽
3266 https://auto.ru/cars/used/sale/toyota/highlander/1105445031-12a2b5f2/ 3 189 000 ₽
3267 https://auto.ru/cars/used/sale/toyota/land_cruiser_prado/1105445693-c48dff75/ 2 999 000 ₽
3269 https://auto.ru/cars/used/sale/toyota/corolla/1105573019-e044e4ac/ 580 000 ₽
3270 https://auto.ru/cars/used/sale/toyota/hilux/1104990904-0d3d92ef/ 2 651 000 ₽
3271 https://auto.ru/cars/used/sale/toyota/highlander/1105446018-573b5074/ 648 000 ₽
3272 https://auto.ru/cars/used/sale/toyota/land_cruiser/1105440751-bf1

In [6]:
get_content("Lexus")

LEXUS: 3146 urls to parse
starting from the beginning...
creating CONTENT/LEXUS_content.csv
1 https://auto.ru/cars/used/sale/lexus/rx/1105212420-6869f423/ 4 000 000 ₽
2 https://auto.ru/cars/used/sale/lexus/lx/1105558235-8a99b613/ 2 515 000 ₽
3 https://auto.ru/cars/used/sale/lexus/is/1105661220-d3ce992d/ 670 000 ₽
4 https://auto.ru/cars/used/sale/lexus/lx/1105312379-7ac16379/ 7 140 000 ₽
6 https://auto.ru/cars/used/sale/lexus/lx/1105579452-e6d47ab6/ 6 250 000 ₽
7 https://auto.ru/cars/used/sale/lexus/es/1105401575-d5cc567d/ 1 417 000 ₽
8 https://auto.ru/cars/used/sale/lexus/gx/1102175761-9cdfce6e/ 3 550 000 ₽
9 https://auto.ru/cars/used/sale/lexus/es/1105470839-4d947aa6/ 1 980 000 ₽
11 https://auto.ru/cars/used/sale/lexus/gs/1105416823-4f0aa32a/ 170 000 ₽
12 https://auto.ru/cars/used/sale/lexus/lx/1105004758-6671553e/ 2 999 000 ₽
13 https://auto.ru/cars/used/sale/lexus/gs/1102166574-33de80d3/ 1 675 000 ₽
14 https://auto.ru/cars/used/sale/lexus/nx/1097475398-72dadea2/ 2 780 000 ₽
15 https

127 https://auto.ru/cars/used/sale/lexus/rx/1105509706-2df9d914/ 1 220 000 ₽
129 https://auto.ru/cars/used/sale/lexus/is/1105557137-a3b56fbf/ 1 290 000 ₽
130 https://auto.ru/cars/used/sale/lexus/ls/1105195012-b42307ab/ 1 500 000 ₽
131 https://auto.ru/cars/used/sale/lexus/nx/1105616767-4a212917/ 2 500 000 ₽
132 https://auto.ru/cars/used/sale/lexus/rx/1105439928-d0ea691f/ 4 550 000 ₽
133 https://auto.ru/cars/used/sale/lexus/lx/1103970131-332ca938/ 2 700 000 ₽
136 https://auto.ru/cars/used/sale/lexus/rx/1103151809-ea4eeac5/ 2 800 000 ₽
137 https://auto.ru/cars/used/sale/lexus/rx/1105439089-f5841cee/ 1 480 000 ₽
138 https://auto.ru/cars/used/sale/lexus/rx/1105551718-4c11e146/ 1 995 000 ₽
139 https://auto.ru/cars/used/sale/lexus/ls/1105243304-841c449f/ 2 750 000 ₽
140 https://auto.ru/cars/used/sale/lexus/rx/1105655207-cab4007b/ 2 830 000 ₽
141 https://auto.ru/cars/used/sale/lexus/lx/1105653723-4de4a6ef/ 2 449 000 ₽
142 https://auto.ru/cars/used/sale/lexus/nx/1105215978-9b8856e4/ 2 445 000 ₽

246 https://auto.ru/cars/used/sale/lexus/nx/1105412219-8b57ca72/ 2 737 957 ₽
247 https://auto.ru/cars/used/sale/lexus/rx/1105561462-8aef6136/ 567 000 ₽
248 https://auto.ru/cars/used/sale/lexus/nx/1105016153-b2df2413/ 3 050 000 ₽
250 https://auto.ru/cars/used/sale/lexus/gx/1105637984-9f63ff7a/ 6 899 000 ₽
251 https://auto.ru/cars/used/sale/lexus/lx/1104136960-06ede89c/ 7 700 000 ₽
252 https://auto.ru/cars/used/sale/lexus/es/1104950603-86160c8b/ 3 288 000 ₽
253 https://auto.ru/cars/used/sale/lexus/nx/1105638821-8fc512af/ 2 430 000 ₽
254 https://auto.ru/cars/used/sale/lexus/rx/1104951542-e72c805c/ 3 747 000 ₽
255 https://auto.ru/cars/used/sale/lexus/nx/1105637989-7e554cc7/ 2 957 000 ₽
256 https://auto.ru/cars/used/sale/lexus/lx/1101335714-172e18f5/ 5 550 000 ₽
257 https://auto.ru/cars/used/sale/lexus/gx/1105169327-2d0e8604/ 1 540 000 ₽
258 https://auto.ru/cars/used/sale/lexus/is/1105636890-1a07d1db/ 1 397 000 ₽
259 https://auto.ru/cars/used/sale/lexus/es/1105638365-6cf1a9e2/ 2 370 000 ₽
2

362 https://auto.ru/cars/used/sale/lexus/rx/1104836743-47ce081e/ 1 837 000 ₽
363 https://auto.ru/cars/used/sale/lexus/lx/1105205188-1c05088e/ 7 857 000 ₽
364 https://auto.ru/cars/used/sale/lexus/lx/1105623052-a0fac47e/ 2 549 000 ₽
365 https://auto.ru/cars/used/sale/lexus/lx/1104678229-e637e681/ 5 949 000 ₽
366 https://auto.ru/cars/used/sale/lexus/rx/1105499837-e8fb117f/ 875 000 ₽
367 https://auto.ru/cars/used/sale/lexus/lx/1104952246-1565ca0c/ 6 900 000 ₽
368 https://auto.ru/cars/used/sale/lexus/rc/1105621180-3220b011/ 4 200 000 ₽
369 https://auto.ru/cars/used/sale/lexus/is/1105621833-52a7b12c/ 750 000 ₽
371 https://auto.ru/cars/used/sale/lexus/es/1105514982-28ca69c7/ 1 635 000 ₽
372 https://auto.ru/cars/used/sale/lexus/rx/1105621664-bb1fa646/ 3 498 000 ₽
373 https://auto.ru/cars/used/sale/lexus/lx/1105618415-fa29f0d1/ 5 300 000 ₽
374 https://auto.ru/cars/used/sale/lexus/rx/1105661937-e12abc60/ 2 900 000 ₽
375 https://auto.ru/cars/used/sale/lexus/gs/1105570476-a6f44934/ 2 150 000 ₽
377

484 https://auto.ru/cars/used/sale/lexus/nx/1105601400-5ff8b77c/ 3 834 000 ₽
485 https://auto.ru/cars/used/sale/lexus/rx/1105599174-18c6b0a2/ 1 400 000 ₽
486 https://auto.ru/cars/used/sale/lexus/rx/1105600873-35996032/ 1 770 000 ₽
487 https://auto.ru/cars/used/sale/lexus/rx/1105601353-57ae8df0/ 3 665 000 ₽
488 https://auto.ru/cars/used/sale/lexus/lx/1104926412-51513718/ 2 190 000 ₽
489 https://auto.ru/cars/used/sale/lexus/rx/1105601401-26d260f8/ 4 634 000 ₽
490 https://auto.ru/cars/used/sale/lexus/rx/1105455276-726042f8/ 1 732 000 ₽
491 https://auto.ru/cars/used/sale/lexus/rx/1105601906-86a7ef79/ 5 989 000 ₽
492 https://auto.ru/cars/used/sale/lexus/lx/1105598818-65add7ff/ 6 099 000 ₽
493 https://auto.ru/cars/used/sale/lexus/rx/1105601832-7f0fd5e1/ 5 927 199 ₽
496 https://auto.ru/cars/used/sale/lexus/rx/1105154693-a15af24f/ 1 427 000 ₽
498 https://auto.ru/cars/used/sale/lexus/ls/1104925650-e03ebbe8/ 1 440 000 ₽
500 links parsed
adding to CONTENT/LEXUS_content.csv
500 https://auto.ru/car

...

2616 https://auto.ru/cars/used/sale/lexus/rx/1105460660-34db11db/ 2 150 000 ₽
2618 https://auto.ru/cars/used/sale/lexus/lx/1102072161-b49b7117/ 4 980 000 ₽
2619 https://auto.ru/cars/used/sale/lexus/gx/1104697953-da14b15e/ 2 320 000 ₽
2620 https://auto.ru/cars/used/sale/lexus/gs/1102811676-f6331cfb/ 2 020 000 ₽
2621 https://auto.ru/cars/used/sale/lexus/ls/1103707540-08214fe9/ 650 000 ₽
2622 https://auto.ru/cars/used/sale/lexus/is/1105359112-fa02e842/ 800 000 ₽
2623 https://auto.ru/cars/used/sale/lexus/rx/1104898291-c8062a50/ 1 550 000 ₽
2624 https://auto.ru/cars/used/sale/lexus/gs/1105562596-2bfd3565/ 2 055 000 ₽
2625 https://auto.ru/cars/used/sale/lexus/rx/1105051676-ff496a16/ 1 970 000 ₽
2626 https://auto.ru/cars/used/sale/lexus/rx/1105490763-22efe3e8/ 700 000 ₽
2628 https://auto.ru/cars/used/sale/lexus/lx/1105209376-39d3c2d6/ 980 000 ₽
2629 https://auto.ru/cars/used/sale/lexus/lx/1105622999-fcb5a24d/ 2 245 000 ₽
2630 https://auto.ru/cars/used/sale/lexus/gs/1105219265-2857a1c0/ 2 300 

2726 https://auto.ru/cars/used/sale/lexus/rx/1103424499-b7f5be48/ 450 000 ₽
2727 https://auto.ru/cars/used/sale/lexus/lx/1104789451-417572fc/ 6 550 000 ₽
2728 https://auto.ru/cars/used/sale/lexus/lx/1104784931-0e746e29/ 2 000 000 ₽
2729 https://auto.ru/cars/used/sale/lexus/lx/1104064808-28a34d2d/ 2 990 000 ₽
2730 https://auto.ru/cars/used/sale/lexus/rx/1104197195-2c210777/ 1 240 000 ₽
2731 https://auto.ru/cars/used/sale/lexus/rx/1105537436-152477b3/ 650 000 ₽
2732 https://auto.ru/cars/used/sale/lexus/es/1105263450-7d57c4be/ 1 050 000 ₽
2733 https://auto.ru/cars/used/sale/lexus/lx/1105561682-a57cc269/ 6 400 000 ₽
2734 https://auto.ru/cars/used/sale/lexus/lx/1104736595-d35aaa27/ 6 100 000 ₽
2735 https://auto.ru/cars/used/sale/lexus/lx/1104892896-0e1fc4e6/ 7 900 000 ₽
2736 https://auto.ru/cars/used/sale/lexus/lx/1103197760-499e6761/ 2 750 000 ₽
2737 https://auto.ru/cars/used/sale/lexus/rx/1079359308-57db06/ 1 000 000 ₽
2738 https://auto.ru/cars/used/sale/lexus/rx/1105486642-de33f55b/ 3 70

2842 https://auto.ru/cars/used/sale/lexus/lx/1104918328-626173c5/ 3 000 000 ₽
2843 https://auto.ru/cars/used/sale/lexus/gs/1105620695-87b344c3/ 820 000 ₽
2844 https://auto.ru/cars/used/sale/lexus/rx/1105458082-fd017659/ 1 199 000 ₽
2845 https://auto.ru/cars/used/sale/lexus/rx/1105241345-28f46066/ 2 340 000 ₽
2846 https://auto.ru/cars/used/sale/lexus/rx/1104186633-b608b4d4/ 800 000 ₽
2847 https://auto.ru/cars/used/sale/lexus/es/1091149982-ad3f729b/ 453 629 ₽
2848 https://auto.ru/cars/used/sale/lexus/gs/1105006593-bebb6fa9/ 848 000 ₽
2849 https://auto.ru/cars/used/sale/lexus/is/1104847704-5060b350/ 820 000 ₽
2850 https://auto.ru/cars/used/sale/lexus/lx/1104280680-9d25da23/ 9 600 000 ₽
2852 https://auto.ru/cars/used/sale/lexus/rx/1105585341-accf37b5/ 1 350 000 ₽
2853 https://auto.ru/cars/used/sale/lexus/lx/1105537480-d2c592d9/ 6 800 000 ₽
2854 https://auto.ru/cars/used/sale/lexus/lx/1105212328-e6856b96/ 1 520 000 ₽
2855 https://auto.ru/cars/used/sale/lexus/rx/1105046331-3b58b219/ 845 000 

2960 https://auto.ru/cars/used/sale/lexus/rx/1104868524-81591af4/ 540 000 ₽
2961 https://auto.ru/cars/used/sale/lexus/rx/1105301079-f36198c1/ 3 730 000 ₽
2962 https://auto.ru/cars/used/sale/lexus/is/1105551783-6e32af44/ 1 695 000 ₽
2963 https://auto.ru/cars/used/sale/lexus/lx/1105259826-fdec6549/ 7 900 000 ₽
2964 https://auto.ru/cars/used/sale/lexus/ls/1105635538-6cef9f26/ 870 000 ₽
2965 https://auto.ru/cars/used/sale/lexus/gx/1105250241-c47dce4d/ 1 480 000 ₽
2966 https://auto.ru/cars/used/sale/lexus/gs/1099301052-a77bc5a1/ 950 000 ₽
2967 https://auto.ru/cars/used/sale/lexus/nx/1103665407-8df37dee/ 2 550 000 ₽
2968 https://auto.ru/cars/used/sale/lexus/rx/1071780154-cbb46b/ 1 200 000 ₽
2969 https://auto.ru/cars/used/sale/lexus/lx/1105404569-eeddb8c6/ 900 000 ₽
2970 https://auto.ru/cars/used/sale/lexus/rx/1105432521-4a1f5761/ 960 000 ₽
2971 https://auto.ru/cars/used/sale/lexus/is/1105348456-fed05892/ 860 000 ₽
2972 https://auto.ru/cars/used/sale/lexus/gx/1105468735-5dc72004/ 1 540 000 ₽


3067 https://auto.ru/cars/used/sale/lexus/rx/1104046003-03e13a90/ 4 150 000 ₽
3068 https://auto.ru/cars/used/sale/lexus/rx/1104796845-d76ce061/ 950 000 ₽
3069 https://auto.ru/cars/used/sale/lexus/rx/1105387341-b80bc229/ 1 850 000 ₽
3070 https://auto.ru/cars/used/sale/lexus/nx/1105065198-d9ad895e/ 2 670 000 ₽
3071 https://auto.ru/cars/used/sale/lexus/rx/1105464563-20247a8e/ 1 170 000 ₽
3072 https://auto.ru/cars/used/sale/lexus/gx/1104856398-ec04ef9b/ 1 450 000 ₽
3073 https://auto.ru/cars/used/sale/lexus/lx/1105451589-4083506a/ 6 350 000 ₽
3074 https://auto.ru/cars/used/sale/lexus/lx/1105154859-3bd9cb84/ 6 350 000 ₽
3075 https://auto.ru/cars/used/sale/lexus/lx/1102243939-aa20e498/ 5 200 000 ₽
3076 https://auto.ru/cars/used/sale/lexus/rx/1103704654-2d8fd563/ 1 599 000 ₽
3077 https://auto.ru/cars/used/sale/lexus/nx/1093371068-f02e6dde/ 2 300 000 ₽
3078 https://auto.ru/cars/used/sale/lexus/rx/1105426296-e9909c21/ 1 830 000 ₽
3079 https://auto.ru/cars/used/sale/lexus/lx/1105441076-8d334812/ 

In [6]:
get_content("Volkswagen")

VOLKSWAGEN: 3477 urls to parse
starting from the beginning...
creating CONTENT/VOLKSWAGEN_content.csv
1 https://auto.ru/cars/used/sale/volkswagen/teramont/1099421594-a856546e/ 3 350 000 ₽
3 https://auto.ru/cars/used/sale/volkswagen/golf/1105475254-9e0d39a9/ 120 000 ₽
6 https://auto.ru/cars/used/sale/volkswagen/golf/1105238312-15a481a1/ 323 000 ₽
7 https://auto.ru/cars/used/sale/volkswagen/golf_plus/1104766542-6161ac61/ 470 000 ₽
10 https://auto.ru/cars/used/sale/volkswagen/transporter/1105423682-8fdebdfa/ 130 000 ₽
12 https://auto.ru/cars/used/sale/volkswagen/passat/1105666345-13891f6c/ 145 000 ₽
13 https://auto.ru/cars/used/sale/volkswagen/touran/1105364606-d40aa586/ 799 000 ₽
14 https://auto.ru/cars/used/sale/volkswagen/tiguan/1105066665-d4ccb707/ 795 000 ₽
15 https://auto.ru/cars/used/sale/volkswagen/transporter/1105009344-c2bbad45/ 322 000 ₽
16 https://auto.ru/cars/used/sale/volkswagen/amarok/1105415420-c16d1f4e/ 949 000 ₽
18 https://auto.ru/cars/used/sale/volkswagen/tiguan/1104952

126 https://auto.ru/cars/used/sale/volkswagen/polo/1105666261-23bc010d/ 1 029 000 ₽
127 https://auto.ru/cars/used/sale/volkswagen/golf/1105598627-7e4aef49/ 635 000 ₽
128 https://auto.ru/cars/used/sale/volkswagen/caravelle/1105664102-65d8c0b9/ 2 400 000 ₽
129 https://auto.ru/cars/used/sale/volkswagen/polo/1105666293-d7a09718/ 879 000 ₽
131 https://auto.ru/cars/used/sale/volkswagen/golf/1105606005-8f3409a3/ 787 000 ₽
132 https://auto.ru/cars/used/sale/volkswagen/caravelle/1105662950-9d5c7b5f/ 2 375 000 ₽
133 https://auto.ru/cars/used/sale/volkswagen/caravelle/1105664770-49dce837/ 3 680 000 ₽
134 https://auto.ru/cars/used/sale/volkswagen/tiguan/1105665898-c8f4f9f0/ 2 229 000 ₽
135 https://auto.ru/cars/used/sale/volkswagen/polo/1105518618-86ccd76c/ 1 180 000 ₽
136 https://auto.ru/cars/used/sale/volkswagen/polo/1105664117-9e8a9af5/ 1 490 000 ₽
137 https://auto.ru/cars/used/sale/volkswagen/tiguan/1105348955-c4f31b9e/ 2 099 900 ₽
138 https://auto.ru/cars/used/sale/volkswagen/caddy/1105374360-

243 https://auto.ru/cars/used/sale/volkswagen/caravelle/1105561039-85bb6f16/ 2 390 000 ₽
244 https://auto.ru/cars/used/sale/volkswagen/polo/1105415218-c1bcf028/ 795 000 ₽
245 https://auto.ru/cars/used/sale/volkswagen/polo/1105269169-89fd84b7/ 630 000 ₽
246 https://auto.ru/cars/used/sale/volkswagen/touareg/1105322759-c57800f7/ 1 095 000 ₽
247 https://auto.ru/cars/used/sale/volkswagen/tiguan/1105509006-b76dea91/ 2 297 000 ₽
248 https://auto.ru/cars/used/sale/volkswagen/passat/1105212775-f6fb0978/ 530 000 ₽
249 https://auto.ru/cars/used/sale/volkswagen/polo/1105488145-bfba6ea6/ 555 000 ₽
250 https://auto.ru/cars/used/sale/volkswagen/polo/1105378724-ab8d6203/ 600 000 ₽
251 https://auto.ru/cars/used/sale/volkswagen/passat/1105657045-054bb0e4/ 599 000 ₽
252 https://auto.ru/cars/used/sale/volkswagen/polo/1105659960-9c0f84ed/ 1 359 990 ₽
253 https://auto.ru/cars/used/sale/volkswagen/polo/1105655830-25c5c872/ 774 000 ₽
254 https://auto.ru/cars/used/sale/volkswagen/jetta/1105658194-3a3777f0/ 685

356 https://auto.ru/cars/used/sale/volkswagen/tiguan/1105443000-e96b0ae6/ 1 750 000 ₽
357 https://auto.ru/cars/used/sale/volkswagen/polo/1105653610-475bd1b6/ 822 000 ₽
358 https://auto.ru/cars/used/sale/volkswagen/polo/1105299422-393f422a/ 1 025 000 ₽
359 https://auto.ru/cars/used/sale/volkswagen/polo/1105322675-5f2e2738/ 904 000 ₽
361 https://auto.ru/cars/used/sale/volkswagen/polo/1105653611-a769a217/ 855 000 ₽
362 https://auto.ru/cars/used/sale/volkswagen/polo/1105410984-b44bc352/ 1 010 000 ₽
363 https://auto.ru/cars/used/sale/volkswagen/polo/1105410975-fbc9164a/ 829 000 ₽
364 https://auto.ru/cars/used/sale/volkswagen/passat/1105651394-2a626ba8/ 1 650 000 ₽
365 https://auto.ru/cars/used/sale/volkswagen/multivan/1105653775-1a6a0da7/ 3 270 000 ₽
367 https://auto.ru/cars/used/sale/volkswagen/jetta/1105262929-2afa1770/ 555 000 ₽
368 https://auto.ru/cars/used/sale/volkswagen/polo/1105653500-f8822292/ 749 000 ₽
369 https://auto.ru/cars/used/sale/volkswagen/tiguan/1105652958-67d1c914/ 2 297

490 https://auto.ru/cars/used/sale/volkswagen/caddy/1105314728-6d2fa918/ 955 000 ₽
491 https://auto.ru/cars/used/sale/volkswagen/multivan/1104227146-7b2229e1/ 4 450 000 ₽
492 https://auto.ru/cars/used/sale/volkswagen/passat/1105601633-0dde2cd7/ 610 000 ₽
493 https://auto.ru/cars/used/sale/volkswagen/teramont/1105027277-b2cd370f/ 3 565 000 ₽
494 https://auto.ru/cars/used/sale/volkswagen/golf/1105600478-c4dcee11/ 877 000 ₽
496 https://auto.ru/cars/used/sale/volkswagen/passat/1105665675-ba7c4657/ 190 000 ₽
497 https://auto.ru/cars/used/sale/volkswagen/tiguan/1105518347-d1b528ef/ 1 050 000 ₽
498 https://auto.ru/cars/used/sale/volkswagen/tiguan/1105233588-02e72c2e/ 1 397 000 ₽
500 links parsed
adding to CONTENT/VOLKSWAGEN_content.csv
500 https://auto.ru/cars/used/sale/volkswagen/touareg/1104800804-74c74da4/ 3 190 000 ₽
501 https://auto.ru/cars/used/sale/volkswagen/tiguan/1104752980-0b6a1533/ 1 290 000 ₽
502 https://auto.ru/cars/used/sale/volkswagen/tiguan/1105544081-33fc890c/ 2 785 000 ₽
50

...

718 https://auto.ru/cars/used/sale/volkswagen/tiguan/1105510513-a0cb3ca2/ 2 330 000 ₽
719 https://auto.ru/cars/used/sale/volkswagen/tiguan/1105638526-5266cc9e/ 2 399 000 ₽
721 https://auto.ru/cars/used/sale/volkswagen/polo/1105591419-be019b6e/ 1 624 000 ₽
722 https://auto.ru/cars/used/sale/volkswagen/golf/1104740445-b6cc566b/ 360 000 ₽
723 https://auto.ru/cars/used/sale/volkswagen/jetta/1105533803-94cfeced/ 1 325 000 ₽
724 https://auto.ru/cars/used/sale/volkswagen/jetta/1105533793-59037f7f/ 1 385 000 ₽
725 https://auto.ru/cars/used/sale/volkswagen/passat/1105479799-54b0e76b/ 1 497 000 ₽
726 https://auto.ru/cars/used/sale/volkswagen/sharan/1104909881-1ed3c44e/ 369 000 ₽
727 https://auto.ru/cars/used/sale/volkswagen/polo/1105587433-950fac7c/ 680 000 ₽
728 https://auto.ru/cars/used/sale/volkswagen/polo/1105637352-beb51d01/ 829 000 ₽
729 https://auto.ru/cars/used/sale/volkswagen/polo/1105640508-4bb565ad/ 850 000 ₽
730 https://auto.ru/cars/used/sale/volkswagen/polo/1105664686-5f56e57f/ 500 

826 https://auto.ru/cars/used/sale/volkswagen/golf/1105663628-b9c259d6/ 1 890 000 ₽
827 https://auto.ru/cars/used/sale/volkswagen/caddy/1105200373-9c80d576/ 555 000 ₽
828 https://auto.ru/cars/used/sale/volkswagen/tiguan/1105505234-00e3bd01/ 1 070 000 ₽
829 https://auto.ru/cars/used/sale/volkswagen/caddy/1105212268-c067e2fa/ 549 000 ₽
830 https://auto.ru/cars/used/sale/volkswagen/multivan/1105388221-ee0b30be/ 4 135 000 ₽
831 https://auto.ru/cars/used/sale/volkswagen/tiguan/1105613952-c7d3924d/ 2 267 000 ₽
832 https://auto.ru/cars/used/sale/volkswagen/touareg/1103814930-a34329de/ 2 767 000 ₽
833 https://auto.ru/cars/used/sale/volkswagen/polo/1105613947-94e69c25/ 719 000 ₽
834 https://auto.ru/cars/used/sale/volkswagen/jetta/1104918528-0b113e33/ 1 237 000 ₽
835 https://auto.ru/cars/used/sale/volkswagen/tiguan/1105589766-a90cdfb1/ 1 335 000 ₽
836 https://auto.ru/cars/used/sale/volkswagen/passat/1105663863-2c522e2d/ 270 000 ₽
837 https://auto.ru/cars/used/sale/volkswagen/polo/1105635494-59c1

930 https://auto.ru/cars/used/sale/volkswagen/tiguan/1105439885-9304e01c/ 2 784 000 ₽
931 https://auto.ru/cars/used/sale/volkswagen/golf/1105633674-d1e70e74/ 869 000 ₽
932 https://auto.ru/cars/used/sale/volkswagen/polo/1105262715-04d1a5d9/ 825 000 ₽
933 https://auto.ru/cars/used/sale/volkswagen/passat/1105663642-95bfdf38/ 210 000 ₽
934 https://auto.ru/cars/used/sale/volkswagen/polo/1105630325-bcf410f9/ 1 177 000 ₽
935 https://auto.ru/cars/used/sale/volkswagen/polo/1105429739-afd71dcc/ 315 000 ₽
936 https://auto.ru/cars/used/sale/volkswagen/polo/1105579246-3d0e5109/ 885 000 ₽
937 https://auto.ru/cars/used/sale/volkswagen/polo/1105631621-d6a138ca/ 989 000 ₽
938 https://auto.ru/cars/used/sale/volkswagen/polo/1105490312-7710ab65/ 1 339 000 ₽
939 https://auto.ru/cars/used/sale/volkswagen/polo/1105398661-7dd42b2b/ 1 329 000 ₽
940 https://auto.ru/cars/used/sale/volkswagen/tiguan/1105634371-0439ecdd/ 1 984 400 ₽
941 https://auto.ru/cars/used/sale/volkswagen/tiguan/1105073699-37a8fcca/ 1 929 00

1039 https://auto.ru/cars/used/sale/volkswagen/polo/1105492495-d49f5da2/ 1 425 000 ₽
1042 https://auto.ru/cars/used/sale/volkswagen/jetta/1105632649-c5f1daa7/ 780 000 ₽
1043 https://auto.ru/cars/used/sale/volkswagen/passat/1105630626-5d3a14b7/ 630 000 ₽
1044 https://auto.ru/cars/used/sale/volkswagen/tiguan/1105142683-9f008a7d/ 1 539 000 ₽
1045 https://auto.ru/cars/used/sale/volkswagen/tiguan/1105575178-aec328f1/ 1 499 000 ₽
1046 https://auto.ru/cars/used/sale/volkswagen/amarok/1105277250-38025f5b/ 1 490 000 ₽
1047 https://auto.ru/cars/used/sale/volkswagen/jetta/1105628625-a4a6e273/ 1 240 000 ₽
1048 https://auto.ru/cars/used/sale/volkswagen/tiguan/1105582999-a4d34976/ 890 000 ₽
1049 https://auto.ru/cars/used/sale/volkswagen/polo/1105403125-6e0bff67/ 1 009 000 ₽
1050 https://auto.ru/cars/used/sale/volkswagen/tiguan/1104650866-1b255006/ 1 368 000 ₽
1051 https://auto.ru/cars/used/sale/volkswagen/touareg/1105405358-29da4c06/ 635 000 ₽
1052 https://auto.ru/cars/used/sale/volkswagen/passat/11

KeyError: "name='from_lifetime', domain=None, path=None"

In [7]:
get_content("Volkswagen", 1000)

VOLKSWAGEN: 3477 urls to parse
starting from ...1000
creating CONTENT/VOLKSWAGEN_1000_content.csv
1001 https://auto.ru/cars/used/sale/volkswagen/touran/1105330239-0b796d67/ 800 000 ₽
1002 https://auto.ru/cars/used/sale/volkswagen/tiguan/1105538854-6cfab37b/ 1 020 000 ₽
1003 https://auto.ru/cars/used/sale/volkswagen/passat_cc/1105631922-3f44fe44/ 950 000 ₽
1004 https://auto.ru/cars/used/sale/volkswagen/polo/1105630721-a1c3aa8c/ 897 000 ₽
1005 https://auto.ru/cars/used/sale/volkswagen/polo/1105631697-487e9d97/ 1 121 900 ₽
1007 https://auto.ru/cars/used/sale/volkswagen/jetta/1105631140-80713faf/ 709 900 ₽
1009 https://auto.ru/cars/used/sale/volkswagen/tiguan/1105545336-49ad6f8a/ 2 480 000 ₽
1010 https://auto.ru/cars/used/sale/volkswagen/passat/1105414347-53602b2f/ 860 000 ₽
1011 https://auto.ru/cars/used/sale/volkswagen/polo/1105631827-a392df64/ 465 000 ₽
1012 https://auto.ru/cars/used/sale/volkswagen/tiguan/1105630612-ed501190/ 1 194 000 ₽
1013 https://auto.ru/cars/used/sale/volkswagen/m

1110 https://auto.ru/cars/used/sale/volkswagen/polo/1105630596-892f71a2/ 867 000 ₽
1112 https://auto.ru/cars/used/sale/volkswagen/jetta/1105626736-dc918fa3/ 643 000 ₽
1113 https://auto.ru/cars/used/sale/volkswagen/multivan/1101824915-05376101/ 1 099 000 ₽
1114 https://auto.ru/cars/used/sale/volkswagen/tiguan/1105337557-8cd04254/ 1 160 000 ₽
1115 https://auto.ru/cars/used/sale/volkswagen/golf/1105660139-282b8f1c/ 170 000 ₽
1116 https://auto.ru/cars/used/sale/volkswagen/jetta/1105563510-2e4eb1c9/ 440 000 ₽
1117 https://auto.ru/cars/used/sale/volkswagen/amarok/1105099430-aba1dc9a/ 1 379 599 ₽
1118 https://auto.ru/cars/used/sale/volkswagen/tiguan/1105369605-df056fd5/ 1 547 000 ₽
1119 https://auto.ru/cars/used/sale/volkswagen/tiguan/1105004451-f564f6fe/ 1 175 000 ₽
1120 https://auto.ru/cars/used/sale/volkswagen/polo/1105354795-2de6c863/ 727 000 ₽
1121 https://auto.ru/cars/used/sale/volkswagen/polo/1105663134-435c548b/ 970 000 ₽
1122 https://auto.ru/cars/used/sale/volkswagen/polo/1105630851-

1221 https://auto.ru/cars/used/sale/volkswagen/caravelle/1105554434-bd4e2f14/ 2 750 000 ₽
1222 https://auto.ru/cars/used/sale/volkswagen/tiguan/1105343618-c9f9ba5f/ 1 409 000 ₽
1223 https://auto.ru/cars/used/sale/volkswagen/tiguan/1105625354-fbc8a69c/ 2 220 000 ₽
1224 https://auto.ru/cars/used/sale/volkswagen/polo/1105624898-396c4aab/ 1 069 000 ₽
1225 https://auto.ru/cars/used/sale/volkswagen/tiguan/1105624731-91ce4351/ 1 387 000 ₽
1227 https://auto.ru/cars/used/sale/volkswagen/teramont/1105623605-6d5d7b62/ 3 711 115 ₽
1229 https://auto.ru/cars/used/sale/volkswagen/passat/1105662274-cb0c29a1/ 450 000 ₽
1230 https://auto.ru/cars/used/sale/volkswagen/jetta/1105623535-f78be9d0/ 499 000 ₽
1231 https://auto.ru/cars/used/sale/volkswagen/touareg/1104250452-ac63403e/ 1 779 000 ₽
1232 https://auto.ru/cars/used/sale/volkswagen/touareg/1105550256-48a6ebb4/ 3 499 000 ₽
1233 https://auto.ru/cars/used/sale/volkswagen/jetta/1104903450-c36accfa/ 1 517 000 ₽
1234 https://auto.ru/cars/used/sale/volkswag

1330 https://auto.ru/cars/used/sale/volkswagen/passat/1105142225-398dda0e/ 1 247 000 ₽
1331 https://auto.ru/cars/used/sale/volkswagen/tiguan/1105619320-d32919af/ 1 237 957 ₽
1332 https://auto.ru/cars/used/sale/volkswagen/polo/1104935161-2e10dfe8/ 205 000 ₽
1333 https://auto.ru/cars/used/sale/volkswagen/polo/1105332731-f891e9c5/ 943 000 ₽
1334 https://auto.ru/cars/used/sale/volkswagen/polo/1105661669-a41c4d5d/ 325 000 ₽
1335 https://auto.ru/cars/used/sale/volkswagen/tiguan/1105509918-673bea77/ 1 190 000 ₽
1337 https://auto.ru/cars/used/sale/volkswagen/passat/1105622332-2743494d/ 1 598 000 ₽
1338 https://auto.ru/cars/used/sale/volkswagen/polo/1105271294-75302dec/ 1 174 000 ₽
1339 https://auto.ru/cars/used/sale/volkswagen/tiguan/1105402869-76b147c5/ 1 460 000 ₽
1340 https://auto.ru/cars/used/sale/volkswagen/polo/1105308692-d30cac97/ 1 098 000 ₽
1341 https://auto.ru/cars/used/sale/volkswagen/jetta/1104971766-806740aa/ 570 000 ₽
1342 https://auto.ru/cars/used/sale/volkswagen/tiguan/11056212

1431 https://auto.ru/cars/used/sale/volkswagen/passat/1105586009-de3241ef/ 335 000 ₽
1433 https://auto.ru/cars/used/sale/volkswagen/polo/1105616627-fb54dabc/ 985 000 ₽
1434 https://auto.ru/cars/used/sale/volkswagen/tiguan/1105660943-0053aba3/ 1 300 000 ₽
1435 https://auto.ru/cars/used/sale/volkswagen/passat/1105412237-ce884304/ 105 000 ₽
1436 https://auto.ru/cars/used/sale/volkswagen/touareg/1105661037-0ad43275/ 725 000 ₽
1437 https://auto.ru/cars/used/sale/volkswagen/touran/1105615499-7f83b37b/ 689 000 ₽
1439 https://auto.ru/cars/used/sale/volkswagen/passat/1105179893-b3e8e789/ 1 400 000 ₽
1440 https://auto.ru/cars/used/sale/volkswagen/polo/1105615341-8afc1669/ 770 000 ₽
1441 https://auto.ru/cars/used/sale/volkswagen/caddy/1105390072-e204269a/ 619 000 ₽
1442 https://auto.ru/cars/used/sale/volkswagen/tiguan/1105661017-fc3db3ef/ 1 975 000 ₽
1444 https://auto.ru/cars/used/sale/volkswagen/polo/1105297615-4535ecb6/ 625 000 ₽
1445 https://auto.ru/cars/used/sale/volkswagen/touareg/1105614432

...

2869 https://auto.ru/cars/used/sale/volkswagen/tiguan/1105488925-207fe43b/ 1 080 000 ₽
2870 https://auto.ru/cars/used/sale/volkswagen/teramont/1105492709-adfcc8f5/ 3 658 000 ₽
2871 https://auto.ru/cars/used/sale/volkswagen/polo/1100157000-e9939049/ 750 000 ₽
2872 https://auto.ru/cars/used/sale/volkswagen/caddy/1105492813-41474731/ 698 000 ₽
2873 https://auto.ru/cars/used/sale/volkswagen/jetta/1105148805-1859d8e6/ 365 000 ₽
2874 https://auto.ru/cars/used/sale/volkswagen/touareg/1105492877-b08bb4cf/ 3 249 000 ₽
2875 https://auto.ru/cars/used/sale/volkswagen/golf/1105490229-3d28445d/ 1 213 800 ₽
2876 https://auto.ru/cars/used/sale/volkswagen/polo/1104747228-f44a6ea1/ 800 000 ₽
2877 https://auto.ru/cars/used/sale/volkswagen/jetta/1105490751-b69c86b1/ 719 000 ₽
2878 https://auto.ru/cars/used/sale/volkswagen/caravelle/1105653492-efa1e41f/ 2 750 000 ₽
2880 https://auto.ru/cars/used/sale/volkswagen/polo/1105212316-440b5a60/ 728 200 ₽
2883 https://auto.ru/cars/used/sale/volkswagen/golf_plus/110

2975 https://auto.ru/cars/used/sale/volkswagen/passat/1105652562-cfd19b7e/ 835 000 ₽
2977 https://auto.ru/cars/used/sale/volkswagen/touareg/1105652365-ee62e0e8/ 720 000 ₽
2979 https://auto.ru/cars/used/sale/volkswagen/tiguan/1105652300-fe0ee467/ 1 150 000 ₽
2980 https://auto.ru/cars/used/sale/volkswagen/polo/1105478013-d8ac1845/ 738 000 ₽
2981 https://auto.ru/cars/used/sale/volkswagen/transporter/1105652353-4ac4ca37/ 175 000 ₽
2982 https://auto.ru/cars/used/sale/volkswagen/caddy/1105477985-c1d08b81/ 1 998 000 ₽
2983 https://auto.ru/cars/used/sale/volkswagen/touareg/1105652363-aa8d6aac/ 2 350 000 ₽
2984 https://auto.ru/cars/used/sale/volkswagen/tiguan/1105652366-5d89b0af/ 830 000 ₽
2986 https://auto.ru/cars/used/sale/volkswagen/tiguan/1105476593-71c7eb9c/ 2 319 000 ₽
2987 https://auto.ru/cars/used/sale/volkswagen/polo/1105478264-4bebf4d9/ 710 000 ₽
2988 https://auto.ru/cars/used/sale/volkswagen/golf/1105652311-0d2d3b96/ 90 000 ₽
2989 https://auto.ru/cars/used/sale/volkswagen/tiguan/1105

3092 https://auto.ru/cars/used/sale/volkswagen/bora/1104031379-d4d0705e/ 175 000 ₽
3095 https://auto.ru/cars/used/sale/volkswagen/polo/1105463868-b9c82f87/ 780 000 ₽
3096 https://auto.ru/cars/used/sale/volkswagen/tiguan/1105462370-640d0375/ 3 090 000 ₽
3097 https://auto.ru/cars/used/sale/volkswagen/polo/1105463052-be7bf4c6/ 959 000 ₽
3098 https://auto.ru/cars/used/sale/volkswagen/tiguan/1105651154-4e488d13/ 1 020 000 ₽
3099 https://auto.ru/cars/used/sale/volkswagen/touareg/1105462905-1b81ad3e/ 2 379 000 ₽
3100 links parsed
adding to CONTENT/VOLKSWAGEN_1000_content.csv
3100 https://auto.ru/cars/used/sale/volkswagen/caddy/1105535016-f5e748db/ 780 000 ₽
3101 https://auto.ru/cars/used/sale/volkswagen/passat/1105651200-f6adf099/ 830 000 ₽
3102 https://auto.ru/cars/used/sale/volkswagen/jetta/1105463059-4f464ab4/ 1 270 000 ₽
3103 https://auto.ru/cars/used/sale/volkswagen/touareg/1105651137-5af79390/ 3 250 000 ₽
3104 https://auto.ru/cars/used/sale/volkswagen/golf/1105491814-43fcfb05/ 575 000 ₽

3198 https://auto.ru/cars/used/sale/volkswagen/polo/1104859380-270d93be/ 760 000 ₽
3199 https://auto.ru/cars/used/sale/volkswagen/caddy/1105650326-e7f6b5e8/ 1 320 000 ₽
3200 links parsed
adding to CONTENT/VOLKSWAGEN_1000_content.csv
3200 https://auto.ru/cars/used/sale/volkswagen/passat/1105298065-62d54181/ 570 000 ₽
3201 https://auto.ru/cars/used/sale/volkswagen/polo/1105455312-f675963c/ 699 000 ₽
3202 https://auto.ru/cars/used/sale/volkswagen/passat_cc/1105408214-d6868e2e/ 650 000 ₽
3203 https://auto.ru/cars/used/sale/volkswagen/golf/1105007167-50980661/ 340 000 ₽
3204 https://auto.ru/cars/used/sale/volkswagen/touareg/1105451631-41dbae37/ 3 427 900 ₽
3205 https://auto.ru/cars/used/sale/volkswagen/tiguan/1105453258-a4150dd6/ 1 500 000 ₽
3206 https://auto.ru/cars/used/sale/volkswagen/tiguan/1105453018-43285b40/ 1 283 000 ₽
3207 https://auto.ru/cars/used/sale/volkswagen/polo/1105453039-ce011557/ 770 000 ₽
3208 https://auto.ru/cars/used/sale/volkswagen/polo/1105451357-7192d9cd/ 909 000 ₽


ConnectionError: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))

In [8]:
get_content("Volkswagen", 3200)

VOLKSWAGEN: 3477 urls to parse
starting from ...3200
creating CONTENT/VOLKSWAGEN_3200_content.csv
3201 https://auto.ru/cars/used/sale/volkswagen/polo/1105455312-f675963c/ 699 000 ₽
3202 https://auto.ru/cars/used/sale/volkswagen/passat_cc/1105408214-d6868e2e/ 650 000 ₽
3203 https://auto.ru/cars/used/sale/volkswagen/golf/1105007167-50980661/ 340 000 ₽
3204 https://auto.ru/cars/used/sale/volkswagen/touareg/1105451631-41dbae37/ 3 427 900 ₽
3205 https://auto.ru/cars/used/sale/volkswagen/tiguan/1105453258-a4150dd6/ 1 500 000 ₽
3206 https://auto.ru/cars/used/sale/volkswagen/tiguan/1105453018-43285b40/ 1 283 000 ₽
3207 https://auto.ru/cars/used/sale/volkswagen/polo/1105453039-ce011557/ 770 000 ₽
3208 https://auto.ru/cars/used/sale/volkswagen/polo/1105451357-7192d9cd/ 909 000 ₽
3210 https://auto.ru/cars/used/sale/volkswagen/passat/1105650189-22ec253f/ 400 000 ₽
3212 https://auto.ru/cars/used/sale/volkswagen/tiguan/1105449402-f185443c/ 1 360 000 ₽
3215 https://auto.ru/cars/used/sale/volkswagen/p

3318 https://auto.ru/cars/used/sale/volkswagen/golf_plus/1105648938-db935162/ 390 000 ₽
3319 https://auto.ru/cars/used/sale/volkswagen/transporter/1105648936-ea1f9aa9/ 320 000 ₽
3320 https://auto.ru/cars/used/sale/volkswagen/polo/1105438149-43791bac/ 745 000 ₽
3321 https://auto.ru/cars/used/sale/volkswagen/jetta/1105648840-f0277859/ 600 000 ₽
3322 https://auto.ru/cars/used/sale/volkswagen/passat/1105406853-54eba2ea/ 155 000 ₽
3323 https://auto.ru/cars/used/sale/volkswagen/multivan/1105436466-eb84359f/ 4 599 000 ₽
3324 https://auto.ru/cars/used/sale/volkswagen/passat/1105648779-03c8e168/ 550 000 ₽
3325 https://auto.ru/cars/used/sale/volkswagen/polo/1105438462-44f78a7e/ 1 288 000 ₽
3326 https://auto.ru/cars/used/sale/volkswagen/tiguan/1105648725-fe3a50f0/ 1 045 000 ₽
3327 https://auto.ru/cars/used/sale/volkswagen/polo/1105436982-4e6b3f57/ 779 000 ₽
3328 https://auto.ru/cars/used/sale/volkswagen/tiguan/1105648763-8bcca081/ 3 350 000 ₽
3329 https://auto.ru/cars/used/sale/volkswagen/touareg

3425 https://auto.ru/cars/used/sale/volkswagen/jetta/1105087744-b91dbd12/ 610 000 ₽
3426 https://auto.ru/cars/used/sale/volkswagen/polo/1105426224-e2eed6c3/ 539 000 ₽
3427 https://auto.ru/cars/used/sale/volkswagen/polo/1105432313-d5cf423e/ 877 000 ₽
3428 https://auto.ru/cars/used/sale/volkswagen/amarok/1105426175-885b31fb/ 2 899 000 ₽
3429 https://auto.ru/cars/used/sale/volkswagen/caddy/1105410032-3c4c3737/ 1 471 074 ₽
3430 https://auto.ru/cars/used/sale/volkswagen/polo/1105429701-096b6fa2/ 764 900 ₽
3431 https://auto.ru/cars/used/sale/volkswagen/polo/1105428819-bc4a4b7a/ 535 000 ₽
3432 https://auto.ru/cars/used/sale/volkswagen/caravelle/1105647832-4a2ca828/ 3 400 000 ₽
3433 https://auto.ru/cars/used/sale/volkswagen/caddy/1104719165-6f6085bf/ 740 000 ₽
3434 https://auto.ru/cars/used/sale/volkswagen/caddy/1105429634-87fe0fee/ 799 000 ₽
3435 https://auto.ru/cars/used/sale/volkswagen/polo/1105430794-d05c00c4/ 879 000 ₽
3437 https://auto.ru/cars/used/sale/volkswagen/passat_cc/1105647889-61

In [9]:
get_content("Mitsubishi")


MITSUBISHI: 3484 urls to parse
starting from the beginning...
creating CONTENT/MITSUBISHI_content.csv
1 https://auto.ru/cars/used/sale/mitsubishi/lancer/1105585572-b92a9db9/ 320 000 ₽
3 https://auto.ru/cars/used/sale/mitsubishi/lancer/1104947672-5fb8ccba/ 503 000 ₽
4 https://auto.ru/cars/used/sale/mitsubishi/outlander/1105393775-ee824430/ 530 000 ₽
5 https://auto.ru/cars/used/sale/mitsubishi/outlander/1104795540-f5e1e713/ 650 000 ₽
6 https://auto.ru/cars/used/sale/mitsubishi/asx/1105557030-bc9e4f81/ 680 000 ₽
7 https://auto.ru/cars/used/sale/mitsubishi/pajero_sport/1104036017-06bb6a59/ 729 000 ₽
8 https://auto.ru/cars/used/sale/mitsubishi/pajero/1105339817-280d04eb/ 875 000 ₽
9 https://auto.ru/cars/used/sale/mitsubishi/pajero/1105222515-ecfaf705/ 315 000 ₽
10 https://auto.ru/cars/used/sale/mitsubishi/montero_sport/1105659630-82d039ae/ 250 000 ₽
11 https://auto.ru/cars/used/sale/mitsubishi/outlander/1105447681-6371f73a/ 690 000 ₽
12 https://auto.ru/cars/used/sale/mitsubishi/pajero/11052

112 https://auto.ru/cars/used/sale/mitsubishi/asx/1105665382-70736394/ 899 000 ₽
114 https://auto.ru/cars/used/sale/mitsubishi/pajero/1105665025-e161a671/ 1 700 000 ₽
115 https://auto.ru/cars/used/sale/mitsubishi/outlander/1105664255-bbea6605/ 1 119 000 ₽
117 https://auto.ru/cars/used/sale/mitsubishi/outlander/1105299766-c3f5e5ac/ 1 650 000 ₽
118 https://auto.ru/cars/used/sale/mitsubishi/asx/1105130778-5a0da2bd/ 920 000 ₽
119 https://auto.ru/cars/used/sale/mitsubishi/lancer/1105344626-2aad5cbc/ 295 000 ₽
120 https://auto.ru/cars/used/sale/mitsubishi/outlander/1104625039-ec05686c/ 1 540 000 ₽
121 https://auto.ru/cars/used/sale/mitsubishi/outlander/1105642193-86456cd5/ 1 699 990 ₽
122 https://auto.ru/cars/used/sale/mitsubishi/lancer/1105310284-802b0722/ 350 000 ₽
123 https://auto.ru/cars/used/sale/mitsubishi/pajero_sport/1105575394-e1182223/ 2 700 000 ₽
124 https://auto.ru/cars/used/sale/mitsubishi/pajero_sport/1105065658-e96c54ae/ 2 900 000 ₽
125 https://auto.ru/cars/used/sale/mitsubish

218 https://auto.ru/cars/used/sale/mitsubishi/eclipse/1105336592-99717827/ 700 000 ₽
219 https://auto.ru/cars/used/sale/mitsubishi/outlander/1103686898-ee60a2cc/ 1 600 000 ₽
220 https://auto.ru/cars/used/sale/mitsubishi/pajero_sport/1105467692-9b95b536/ 3 400 000 ₽
221 https://auto.ru/cars/used/sale/mitsubishi/pajero/1105458300-5db91205/ 1 117 000 ₽
222 https://auto.ru/cars/used/sale/mitsubishi/outlander/1105656896-ecd289f0/ 1 250 000 ₽
224 https://auto.ru/cars/used/sale/mitsubishi/outlander/1105409593-d0d78208/ 1 550 000 ₽
225 https://auto.ru/cars/used/sale/mitsubishi/asx/1105658303-3f32eb4b/ 879 000 ₽
226 https://auto.ru/cars/used/sale/mitsubishi/l200/1105057552-112505ba/ 250 000 ₽
227 https://auto.ru/cars/used/sale/mitsubishi/lancer/1105546155-d99f22fe/ 500 000 ₽
228 https://auto.ru/cars/used/sale/mitsubishi/lancer/1104135006-fe2ce9b2/ 530 999 ₽
229 https://auto.ru/cars/used/sale/mitsubishi/pajero/1105453231-a33af430/ 750 000 ₽
231 https://auto.ru/cars/used/sale/mitsubishi/outlander

334 https://auto.ru/cars/used/sale/mitsubishi/outlander/1103451767-59f549b2/ 1 090 000 ₽
336 https://auto.ru/cars/used/sale/mitsubishi/pajero/1105253197-16fd16c6/ 1 980 000 ₽
338 https://auto.ru/cars/used/sale/mitsubishi/galant/1105405362-a27c858d/ 75 000 ₽
339 https://auto.ru/cars/used/sale/mitsubishi/outlander/1105654045-3dd22684/ 1 478 000 ₽
340 https://auto.ru/cars/used/sale/mitsubishi/lancer/1105544800-d5297609/ 450 000 ₽
341 https://auto.ru/cars/used/sale/mitsubishi/l200/1105653841-00d7e1c0/ 520 000 ₽
342 https://auto.ru/cars/used/sale/mitsubishi/carisma/1105493973-a2ecf14a/ 50 000 ₽
344 https://auto.ru/cars/used/sale/mitsubishi/pajero_sport/1105652257-2356dcb4/ 649 000 ₽
346 https://auto.ru/cars/used/sale/mitsubishi/pajero/1105435513-156f7740/ 1 378 000 ₽
347 https://auto.ru/cars/used/sale/mitsubishi/pajero/1105653800-2be43bb0/ 1 666 170 ₽
348 https://auto.ru/cars/used/sale/mitsubishi/colt/1105653416-524baa90/ 250 000 ₽
350 https://auto.ru/cars/used/sale/mitsubishi/pajero/110532

458 https://auto.ru/cars/used/sale/mitsubishi/pajero/1104155267-4788ccde/ 300 000 ₽
460 https://auto.ru/cars/used/sale/mitsubishi/lancer/1105213029-4934a490/ 470 000 ₽
461 https://auto.ru/cars/used/sale/mitsubishi/outlander/1105537126-c8ccd5ee/ 770 000 ₽
462 https://auto.ru/cars/used/sale/mitsubishi/outlander/1105227472-872ebe25/ 1 735 000 ₽
464 https://auto.ru/cars/used/sale/mitsubishi/lancer/1105647613-765c2ce4/ 229 000 ₽
465 https://auto.ru/cars/used/sale/mitsubishi/colt/1105647138-def498e8/ 50 000 ₽
466 https://auto.ru/cars/used/sale/mitsubishi/outlander/1105648482-c0b23fe7/ 1 389 000 ₽
467 https://auto.ru/cars/used/sale/mitsubishi/asx/1105126565-e788d1f3/ 795 000 ₽
469 https://auto.ru/cars/used/sale/mitsubishi/outlander/1105452977-136993a4/ 1 497 000 ₽
470 https://auto.ru/cars/used/sale/mitsubishi/outlander/1105278350-cef17b09/ 1 359 000 ₽
471 https://auto.ru/cars/used/sale/mitsubishi/lancer/1105452250-529b2de7/ 262 000 ₽
472 https://auto.ru/cars/used/sale/mitsubishi/pajero/110564

...

1884 https://auto.ru/cars/used/sale/mitsubishi/outlander/1105493000-f7f8bcd3/ 2 449 000 ₽
1885 https://auto.ru/cars/used/sale/mitsubishi/outlander/1105489552-0e3a1eba/ 2 160 000 ₽
1886 https://auto.ru/cars/used/sale/mitsubishi/l200/1105488067-c2334e9a/ 1 170 000 ₽
1887 https://auto.ru/cars/used/sale/mitsubishi/pajero/1105492220-662562fd/ 1 049 900 ₽
1888 https://auto.ru/cars/used/sale/mitsubishi/outlander/1105488951-ea367fd2/ 1 358 000 ₽
1890 https://auto.ru/cars/used/sale/mitsubishi/carisma/1105653515-1f129549/ 260 000 ₽
1892 https://auto.ru/cars/used/sale/mitsubishi/lancer/1105489585-2aaa7e70/ 359 000 ₽
1894 https://auto.ru/cars/used/sale/mitsubishi/outlander/1105487797-b6954513/ 1 669 000 ₽
1895 https://auto.ru/cars/used/sale/mitsubishi/asx/1105328029-ee61dadf/ 845 000 ₽
1896 https://auto.ru/cars/used/sale/mitsubishi/diamante/1105653437-d37aa6ef/ 240 000 ₽
1897 https://auto.ru/cars/used/sale/mitsubishi/lancer/1105653456-f1f3c878/ 380 000 ₽
1898 https://auto.ru/cars/used/sale/mitsubi

1985 https://auto.ru/cars/used/sale/mitsubishi/outlander/1104913283-177d1ac1/ 870 000 ₽
1986 https://auto.ru/cars/used/sale/mitsubishi/colt/1103339265-6ab378cb/ 500 000 ₽
1987 https://auto.ru/cars/used/sale/mitsubishi/lancer/1105435160-038ed3dd/ 260 000 ₽
1988 https://auto.ru/cars/used/sale/mitsubishi/pajero_sport/1105651892-3dbef690/ 900 000 ₽
1989 https://auto.ru/cars/used/sale/mitsubishi/outlander/1105471977-33928983/ 1 250 000 ₽
1990 https://auto.ru/cars/used/sale/mitsubishi/pajero_sport/1105470792-9485ead1/ 2 589 000 ₽
1991 https://auto.ru/cars/used/sale/mitsubishi/outlander/1105651867-18e35eb1/ 530 000 ₽
1992 https://auto.ru/cars/used/sale/mitsubishi/lancer/1105073452-4a0a470e/ 313 000 ₽
1993 https://auto.ru/cars/used/sale/mitsubishi/outlander/1105472196-d01ba67e/ 2 165 000 ₽
1995 https://auto.ru/cars/used/sale/mitsubishi/l200/1105651822-be0cc260/ 1 300 000 ₽
1996 https://auto.ru/cars/used/sale/mitsubishi/carisma/1105473533-b54b3a9c/ 259 000 ₽
1997 https://auto.ru/cars/used/sale/

2090 https://auto.ru/cars/used/sale/mitsubishi/pajero_sport/1105649832-491b2a17/ 3 000 000 ₽
2091 https://auto.ru/cars/used/sale/mitsubishi/l200/1105649773-cce94d91/ 820 000 ₽
2092 https://auto.ru/cars/used/sale/mitsubishi/pajero_sport/1105149384-b5a1fcba/ 2 950 000 ₽
2093 https://auto.ru/cars/used/sale/mitsubishi/debonair/1105649569-3236ce75/ 600 000 ₽
2094 https://auto.ru/cars/used/sale/mitsubishi/lancer/1105426764-17f9eb0d/ 210 000 ₽
2095 https://auto.ru/cars/used/sale/mitsubishi/delica_d_5/1105436404-d785f7b0/ 1 790 000 ₽
2096 https://auto.ru/cars/used/sale/mitsubishi/pajero_sport/1105447464-e4c18668/ 2 115 000 ₽
2097 https://auto.ru/cars/used/sale/mitsubishi/lancer/1105444693-3b244222/ 437 000 ₽
2098 https://auto.ru/cars/used/sale/mitsubishi/outlander/1105443230-585eb63b/ 1 125 000 ₽
2099 https://auto.ru/cars/used/sale/mitsubishi/outlander/1105206980-44850efa/ 1 239 000 ₽
2100 links parsed
adding to CONTENT/MITSUBISHI_content.csv
2100 https://auto.ru/cars/used/sale/mitsubishi/asx/

2196 https://auto.ru/cars/used/sale/mitsubishi/asx/1105647397-a564ffb2/ 1 750 000 ₽
2197 https://auto.ru/cars/used/sale/mitsubishi/outlander/1105647674-5c361e5c/ 850 000 ₽
2198 https://auto.ru/cars/used/sale/mitsubishi/l200/1105647639-be50a315/ 720 000 ₽
2199 https://auto.ru/cars/used/sale/mitsubishi/pajero/1105647102-177e1420/ 150 000 ₽
2200 links parsed
adding to CONTENT/MITSUBISHI_content.csv
2201 https://auto.ru/cars/used/sale/mitsubishi/lancer/1105647417-4b9d229e/ 170 000 ₽
2202 https://auto.ru/cars/used/sale/mitsubishi/outlander/1105419417-072f99c4/ 1 620 000 ₽
2203 https://auto.ru/cars/used/sale/mitsubishi/lancer/1105647349-0be8fffc/ 330 000 ₽
2204 https://auto.ru/cars/used/sale/mitsubishi/lancer/1105421983-0b16ddc5/ 575 000 ₽
2205 https://auto.ru/cars/used/sale/mitsubishi/colt/1105420135-0875db92/ 350 000 ₽
2206 https://auto.ru/cars/used/sale/mitsubishi/pajero/1105647311-1afe9690/ 330 000 ₽
2207 https://auto.ru/cars/used/sale/mitsubishi/outlander/1105284612-3704c3bc/ 1 609 897 

KeyError: "name='from_lifetime', domain=None, path=None"

In [10]:
get_content("Mitsubishi", 2200)

MITSUBISHI: 3484 urls to parse
starting from ...2200
creating CONTENT/MITSUBISHI_2200_content.csv
2201 https://auto.ru/cars/used/sale/mitsubishi/lancer/1105647417-4b9d229e/ 170 000 ₽
2202 https://auto.ru/cars/used/sale/mitsubishi/outlander/1105419417-072f99c4/ 1 620 000 ₽
2203 https://auto.ru/cars/used/sale/mitsubishi/lancer/1105647349-0be8fffc/ 330 000 ₽
2204 https://auto.ru/cars/used/sale/mitsubishi/lancer/1105421983-0b16ddc5/ 575 000 ₽
2205 https://auto.ru/cars/used/sale/mitsubishi/colt/1105420135-0875db92/ 350 000 ₽
2206 https://auto.ru/cars/used/sale/mitsubishi/pajero/1105647311-1afe9690/ 330 000 ₽
2207 https://auto.ru/cars/used/sale/mitsubishi/outlander/1105284612-3704c3bc/ 1 609 897 ₽
2208 https://auto.ru/cars/used/sale/mitsubishi/pajero/1105647256-7d544e90/ 680 000 ₽
2210 https://auto.ru/cars/used/sale/mitsubishi/pajero_sport/1105423283-f4410845/ 1 189 000 ₽
2211 https://auto.ru/cars/used/sale/mitsubishi/pajero_sport/1105422943-1e0d2b75/ 660 000 ₽
2212 https://auto.ru/cars/used

2307 https://auto.ru/cars/used/sale/mitsubishi/outlander/1105046040-d1e0ae25/ 2 340 000 ₽
2309 https://auto.ru/cars/used/sale/mitsubishi/asx/1105635957-0403f50b/ 1 000 000 ₽
2310 https://auto.ru/cars/used/sale/mitsubishi/outlander/1105082740-1fe2f94b/ 1 370 000 ₽
2311 https://auto.ru/cars/used/sale/mitsubishi/asx/1105080400-7b450ea0/ 1 610 000 ₽
2312 https://auto.ru/cars/used/sale/mitsubishi/lancer/1105590932-181dd4da/ 499 000 ₽
2313 https://auto.ru/cars/used/sale/mitsubishi/galant/1101904409-a593304e/ 60 000 ₽
2314 https://auto.ru/cars/used/sale/mitsubishi/outlander/1105406004-d84ed012/ 880 000 ₽
2316 https://auto.ru/cars/used/sale/mitsubishi/pajero/1105494395-228873a7/ 270 000 ₽
2317 https://auto.ru/cars/used/sale/mitsubishi/lancer/1105481412-4993ecaf/ 299 000 ₽
2318 https://auto.ru/cars/used/sale/mitsubishi/l300/1098574834-3259804a/ 170 000 ₽
2319 https://auto.ru/cars/used/sale/mitsubishi/pajero/1104987971-e6cc2433/ 259 199 ₽
2320 https://auto.ru/cars/used/sale/mitsubishi/galant/109

2409 https://auto.ru/cars/used/sale/mitsubishi/outlander/1099351142-d1d204f1/ 1 969 224 ₽
2410 https://auto.ru/cars/used/sale/mitsubishi/asx/1105229244-33b8f39a/ 830 000 ₽
2411 https://auto.ru/cars/used/sale/mitsubishi/pajero/1105367113-9963767b/ 1 599 000 ₽
2412 https://auto.ru/cars/used/sale/mitsubishi/pajero_sport/1105364885-979e0f6b/ 1 376 000 ₽
2413 https://auto.ru/cars/used/sale/mitsubishi/pajero_sport/1088429770-bd940b8b/ 815 000 ₽
2414 https://auto.ru/cars/used/sale/mitsubishi/pajero/1105367109-9cfdac3a/ 2 049 000 ₽
2415 https://auto.ru/cars/used/sale/mitsubishi/outlander/1105364838-8cf23125/ 1 279 000 ₽
2417 https://auto.ru/cars/used/sale/mitsubishi/outlander/1096117918-29b130d9/ 1 200 000 ₽
2418 https://auto.ru/cars/used/sale/mitsubishi/pajero_sport/1105367134-c14d5a21/ 1 198 000 ₽
2419 https://auto.ru/cars/used/sale/mitsubishi/pajero_sport/1104639720-1896eff2/ 900 000 ₽
2422 https://auto.ru/cars/used/sale/mitsubishi/lancer/1104069586-82404884/ 300 000 ₽
2423 https://auto.ru/

2513 https://auto.ru/cars/used/sale/mitsubishi/outlander/1104870459-24aaeaaf/ 580 000 ₽
2514 https://auto.ru/cars/used/sale/mitsubishi/lancer/1105408969-d4e7b9b4/ 250 000 ₽
2515 https://auto.ru/cars/used/sale/mitsubishi/outlander/1104996945-fe322da0/ 820 000 ₽
2516 https://auto.ru/cars/used/sale/mitsubishi/carisma/1105085142-27cfe479/ 200 000 ₽
2517 https://auto.ru/cars/used/sale/mitsubishi/space_wagon/1104705846-26812171/ 100 000 ₽
2518 https://auto.ru/cars/used/sale/mitsubishi/lancer/1105436938-ab36aa3e/ 85 000 ₽
2519 https://auto.ru/cars/used/sale/mitsubishi/colt/1105532915-eca9423d/ 310 000 ₽
2520 https://auto.ru/cars/used/sale/mitsubishi/asx/1103806749-bbe503d6/ 850 000 ₽
2522 https://auto.ru/cars/used/sale/mitsubishi/outlander/1105329912-cfe1102e/ 1 418 000 ₽
2523 https://auto.ru/cars/used/sale/mitsubishi/carisma/1084761547-35a12167/ 150 000 ₽
2524 https://auto.ru/cars/used/sale/mitsubishi/lancer/1099646840-3be0a919/ 357 000 ₽
2525 https://auto.ru/cars/used/sale/mitsubishi/3000_g

2615 https://auto.ru/cars/used/sale/mitsubishi/lancer/1105512861-b9cab1dd/ 165 000 ₽
2617 https://auto.ru/cars/used/sale/mitsubishi/outlander/1105321083-51630308/ 899 000 ₽
2618 https://auto.ru/cars/used/sale/mitsubishi/outlander/1105218205-1c095eda/ 1 010 000 ₽
2619 https://auto.ru/cars/used/sale/mitsubishi/delica/1102812808-78284209/ 160 000 ₽
2620 https://auto.ru/cars/used/sale/mitsubishi/asx/1105644116-175d2b69/ 1 015 000 ₽
2621 https://auto.ru/cars/used/sale/mitsubishi/l200/1105518305-80084244/ 1 010 000 ₽
2622 https://auto.ru/cars/used/sale/mitsubishi/outlander/1105328311-74de658f/ 1 439 000 ₽
2623 https://auto.ru/cars/used/sale/mitsubishi/asx/1105013155-4f8c19fc/ 740 000 ₽
2624 https://auto.ru/cars/used/sale/mitsubishi/pajero_sport/1105625238-baa3cdca/ 500 000 ₽
2625 https://auto.ru/cars/used/sale/mitsubishi/galant/1101867987-23597718/ 140 000 ₽
2626 https://auto.ru/cars/used/sale/mitsubishi/outlander/1105644341-080c1df8/ 400 000 ₽
2627 https://auto.ru/cars/used/sale/mitsubishi/

...

3017 https://auto.ru/cars/used/sale/mitsubishi/outlander/1105220398-0c12543e/ 1 790 000 ₽
3018 https://auto.ru/cars/used/sale/mitsubishi/outlander/1104862327-cb8fddd0/ 1 220 000 ₽
3019 https://auto.ru/cars/used/sale/mitsubishi/legnum/1105099225-584ed6b1/ 240 000 ₽
3020 https://auto.ru/cars/used/sale/mitsubishi/pajero/1102980387-5eed928b/ 1 780 000 ₽
3021 https://auto.ru/cars/used/sale/mitsubishi/legnum/1105499870-a440187c/ 285 000 ₽
3022 https://auto.ru/cars/used/sale/mitsubishi/outlander/1105245693-336b664b/ 1 089 700 ₽
3023 https://auto.ru/cars/used/sale/mitsubishi/asx/1105540834-d577af75/ 790 000 ₽
3024 https://auto.ru/cars/used/sale/mitsubishi/eclipse_cross/1105449015-dfaa83c7/ 1 413 703 ₽
3025 https://auto.ru/cars/used/sale/mitsubishi/outlander/1104899726-55bb7f55/ 1 367 000 ₽
3026 https://auto.ru/cars/used/sale/mitsubishi/pajero/1092761454-007d0975/ 2 600 000 ₽
3028 https://auto.ru/cars/used/sale/mitsubishi/libero/1105606728-5987c95d/ 199 000 ₽
3029 https://auto.ru/cars/used/sale

3119 https://auto.ru/cars/used/sale/mitsubishi/delica_d_5/1101650615-0451fabd/ 2 000 000 ₽
3120 https://auto.ru/cars/used/sale/mitsubishi/outlander/1103759965-d405b4ba/ 1 110 000 ₽
3121 https://auto.ru/cars/used/sale/mitsubishi/pajero/1105501493-d3f81644/ 1 650 000 ₽
3122 https://auto.ru/cars/used/sale/mitsubishi/galant/1105198028-49bd9672/ 89 499 ₽
3123 https://auto.ru/cars/used/sale/mitsubishi/outlander/1103973647-c50ea936/ 1 069 999 ₽
3124 https://auto.ru/cars/used/sale/mitsubishi/outlander/1105258229-2ac5f1e4/ 1 289 990 ₽
3125 https://auto.ru/cars/used/sale/mitsubishi/lancer/1104881296-d7d8efb1/ 250 000 ₽
3126 https://auto.ru/cars/used/sale/mitsubishi/space_star/1101450140-1921bcde/ 135 000 ₽
3127 https://auto.ru/cars/used/sale/mitsubishi/lancer/1105439901-af00e972/ 249 000 ₽
3128 https://auto.ru/cars/used/sale/mitsubishi/carisma/1105095814-4165f58e/ 258 000 ₽
3129 https://auto.ru/cars/used/sale/mitsubishi/pajero_junior/1090608020-36fe10e4/ 3 000 000 ₽
3130 https://auto.ru/cars/use

3219 https://auto.ru/cars/used/sale/mitsubishi/outlander/1090793536-48770680/ 1 900 000 ₽
3220 https://auto.ru/cars/used/sale/mitsubishi/outlander/1105229341-2985da63/ 1 357 000 ₽
3221 https://auto.ru/cars/used/sale/mitsubishi/colt/1105496154-91de281b/ 320 000 ₽
3222 https://auto.ru/cars/used/sale/mitsubishi/asx/1105582361-62c70623/ 870 000 ₽
3223 https://auto.ru/cars/used/sale/mitsubishi/outlander/1103493588-802141ef/ 1 420 000 ₽
3224 https://auto.ru/cars/used/sale/mitsubishi/asx/1105279751-eba2b6c5/ 800 000 ₽
3225 https://auto.ru/cars/used/sale/mitsubishi/delica_d_5/1105187853-08e3981e/ 1 430 000 ₽
3226 https://auto.ru/cars/used/sale/mitsubishi/outlander/1105297410-d02eceea/ 919 899 ₽
3227 https://auto.ru/cars/used/sale/mitsubishi/lancer/1105301733-fb91ac0b/ 255 000 ₽
3228 https://auto.ru/cars/used/sale/mitsubishi/lancer/1104872671-ddd1e76e/ 330 000 ₽
3229 https://auto.ru/cars/used/sale/mitsubishi/pajero/1105534063-51d613dd/ 650 000 ₽
3230 https://auto.ru/cars/used/sale/mitsubishi/ou

3318 https://auto.ru/cars/used/sale/mitsubishi/montero/1102877547-e5a59424/ 250 000 ₽
3319 https://auto.ru/cars/used/sale/mitsubishi/lancer/1105282240-3bd134e2/ 535 000 ₽
3320 https://auto.ru/cars/used/sale/mitsubishi/outlander/1105565849-dcc6b89b/ 970 000 ₽
3321 https://auto.ru/cars/used/sale/mitsubishi/outlander/1093080978-b3f162ab/ 410 000 ₽
3322 https://auto.ru/cars/used/sale/mitsubishi/outlander/1103293237-ab333e1e/ 2 090 000 ₽
3323 https://auto.ru/cars/used/sale/mitsubishi/outlander/1103483490-0a073236/ 945 000 ₽
3324 https://auto.ru/cars/used/sale/mitsubishi/lancer/1103978863-4501499d/ 300 000 ₽
3325 https://auto.ru/cars/used/sale/mitsubishi/outlander/1105023515-5ec52ff2/ 1 050 000 ₽
3326 https://auto.ru/cars/used/sale/mitsubishi/outlander/1092988302-734e80d8/ 1 300 000 ₽
3327 https://auto.ru/cars/used/sale/mitsubishi/lancer/1104944998-e5042ecb/ 297 000 ₽
3328 https://auto.ru/cars/used/sale/mitsubishi/outlander/1105253563-5a2e493a/ 950 000 ₽
3329 https://auto.ru/cars/used/sale/m

3419 https://auto.ru/cars/used/sale/mitsubishi/lancer/1105555600-aaf44b18/ 265 000 ₽
3420 https://auto.ru/cars/used/sale/mitsubishi/lancer/1104270458-551c83f4/ 400 000 ₽
3421 https://auto.ru/cars/used/sale/mitsubishi/outlander/1003205882-7ea5b/ 1 080 000 ₽
3422 https://auto.ru/cars/used/sale/mitsubishi/outlander/1105525811-044d6c0c/ 1 250 000 ₽
3425 https://auto.ru/cars/used/sale/mitsubishi/chariot/1105151601-62d371d0/ 185 000 ₽
3426 https://auto.ru/cars/used/sale/mitsubishi/pajero/1105200019-5a944cf6/ 1 389 000 ₽
3427 https://auto.ru/cars/used/sale/mitsubishi/outlander/1105514750-b6cdc83d/ 790 000 ₽
3428 https://auto.ru/cars/used/sale/mitsubishi/outlander/1105078689-1f759a33/ 1 750 000 ₽
3429 https://auto.ru/cars/used/sale/mitsubishi/pajero_sport/1104793266-cd0c29dc/ 1 520 000 ₽
3430 https://auto.ru/cars/used/sale/mitsubishi/outlander/1105611139-8741883e/ 1 200 000 ₽
3431 https://auto.ru/cars/used/sale/mitsubishi/pajero/1100091030-b18e21e8/ 1 490 000 ₽
3432 https://auto.ru/cars/used/s